In [ ]:
!mkdir -p /content/drive/MyDrive/hindi_dfake/{raw,processed,metadata,scripts,checkpoints}

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os, glob
MOUNT_PATH = "/content/drive" if os.path.isdir("/content/drive/MyDrive") else "/content/gdrive"
print("Drive is already mounted here →", MOUNT_PATH)

Drive is already mounted here → /content/drive


In [ ]:
ROOT_DIR = f"{MOUNT_PATH}/MyDrive/hindi_dfake"
OUT_DIR  = f"{ROOT_DIR}/raw/real_clean/ivr"
CSV_PATH = f"{ROOT_DIR}/metadata/master_real.csv"

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

# **REAL**  **REAL** **REAL** **REAL** **REAL** **REAL** **REAL** **REAL** **REAL** **REAL** **REAL** **REAL** **REAL** **REAL** **REAL** **REAL** **REAL**




In [ ]:
!pip install -Uq pandas==2.2.2 requests==2.32.3

In [ ]:
!pip -q install --force-reinstall --no-deps "datasets==2.19.2"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 15.2 MB/s eta 0:00:00


In [ ]:
import os
print([p for p in os.listdir() if p == "datasets" or p.startswith("datasets.py")])

[]


In [ ]:
!pip -q install pyarrow-hotfix "fsspec[http]==2024.3.1" --no-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 5.4 MB/s eta 0:00:00


In [ ]:
# --- minimal env patch required by current 'datasets' builds ---
!pip -q install pyarrow-hotfix "fsspec[http]==2024.3.1" --no-deps >/dev/null

# --- clear any shadowing / partial import of 'datasets' ---
import sys, os, importlib

# If you ever had a variable named `datasets` in a cell, drop it:
if 'datasets' in globals():
    del datasets

# Remove partially-initialized modules from sys.modules so import is clean
for k in list(sys.modules):
    if k == "datasets" or k.startswith("datasets."):
        sys.modules.pop(k, None)

# Detect local files that would shadow the library (rename/delete them if printed)
offenders = [p for p in os.listdir() if p == "datasets" or p.startswith("datasets.py")]
if offenders:
    print("⚠️ Local files/folders shadowing the library:", offenders,
          "\nRename/remove them (e.g. 'datasets_local.py'), then re-run this cell.")
else:
    print("✅ No local shadowing detected. Proceeding…")

# Force a clean reinstall of the library itself (no deps to keep colab pins)
!pip -q install --force-reinstall --no-deps "datasets==2.19.2" >/dev/null
print("✅ datasets reinstalled cleanly.")


✅ No local shadowing detected. Proceeding…
✅ datasets reinstalled cleanly.


IndicVoices

In [ ]:
import os, csv, tqdm, hashlib, soundfile as sf
from datasets import load_dataset, Audio

HOURS_TO_SAVE = 30
ROOT_DIR      = "/content/drive/MyDrive/hindi_dfake"
OUT_DIR       = f"{ROOT_DIR}/raw/real_clean/ivr"
CSV_PATH      = f"{ROOT_DIR}/metadata/master_real.csv"
TEXT_KEY      = "text"

# create the folders once
os.makedirs(OUT_DIR,      exist_ok=True)
os.makedirs(os.path.dirname(CSV_PATH), exist_ok=True)

# streaming iterator
ds_iter = (
    load_dataset("ai4bharat/indicvoices_r", "Hindi",
                 split="train", streaming=True)
    .cast_column("audio", Audio(sampling_rate=16_000))
    .filter(lambda x: len(x[TEXT_KEY].strip()) > 0)
)

secs_cap  = HOURS_TO_SAVE * 3600
secs_done = 0

with open(CSV_PATH, "w", newline='') as csv_f, \
     tqdm.tqdm(total=secs_cap, unit="s") as bar:

    writer = csv.DictWriter(csv_f,
            fieldnames=["utt_id", "path", "duration", "speaker_id",
                        "gender", "text", "label"])
    writer.writeheader()

    for ex in ds_iter:
        arr, sr = ex["audio"]["array"], ex["audio"]["sampling_rate"]
        dur     = len(arr) / sr
        utt_id  = hashlib.sha1(ex["audio"]["path"].encode()).hexdigest()[:16]
        out_wav = f"{OUT_DIR}/{utt_id}.wav"

        sf.write(out_wav, arr, sr)
        writer.writerow({
            "utt_id"     : utt_id,
            "path"       : out_wav,
            "duration"   : dur,
            "speaker_id" : ex["speaker_id"],
            "gender"     : ex["gender"],
            "text"       : ex[TEXT_KEY],
            "label"      : "real"
        })

        secs_done += dur
        bar.update(dur)
        if secs_done >= secs_cap:
            break

print(f"\nSaved ≈ {secs_done/3600:.1f} h of audio to Drive:")
print("  WAVs →", OUT_DIR)
print("  CSV  →", CSV_PATH)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/246 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/108000 [00:17<?, ?s/s]


KeyboardInterrupt: 

IndicTTS

In [ ]:
# ==============================================================
#  IndicTTS-Hindi  →  ~10 h studio-quality real speech
#  • Writes WAVs to  .../raw/real_clean/indictts
#  • Appends rows to .../metadata/master_real.csv
# ==============================================================

# ---------- paths (edit ONLY if your mount path differs) -------
MOUNT_PATH   = "/content/drive"              # or "/content/gdrive"
ROOT_DIR     = f"{MOUNT_PATH}/MyDrive/hindi_dfake"
OUT_DIR      = f"{ROOT_DIR}/raw/real_clean/indictts"
CSV_PATH     = f"{ROOT_DIR}/metadata/master_real.csv"

# ---------- constants ------------------------------------------
TARGET_HOURS = 10.5           # whole dataset ≈10.3 h (leave as-is)
GENDER_MAP   = {0: "m", 1: "f", 2: "o"}      # fallback "u"
TEXT_KEY     = "text"         # transcript field in IndicTTS

# ---------- imports & dirs -------------------------------------
import os, csv, hashlib, tqdm, soundfile as sf
from datasets import load_dataset, Audio
os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(os.path.dirname(CSV_PATH), exist_ok=True)

# ---------- helper: write header only once ---------------------
need_header = not os.path.isfile(CSV_PATH)

# ---------- streaming iterator ---------------------------------
tts_iter = (load_dataset("SPRINGLab/IndicTTS-Hindi",
                         split="train", streaming=True)
            .cast_column("audio", Audio(sampling_rate=16_000)))

secs_cap  = TARGET_HOURS * 3600
secs_done = 0

with open(CSV_PATH, "a", newline='') as csv_f, \
     tqdm.tqdm(total=secs_cap, unit="s", desc="IndicTTS DL") as bar:

    writer = csv.DictWriter(csv_f,
        fieldnames=["utt_id","path","duration","speaker_id",
                    "gender","text","label"])

    if need_header:
        writer.writeheader()

    for ex in tts_iter:
        arr, sr = ex["audio"]["array"], ex["audio"]["sampling_rate"]
        dur     = len(arr) / sr
        utt_id  = hashlib.sha1(ex["audio"]["path"].encode()).hexdigest()[:16]

        # gender mapping (numeric → letter)
        g_val   = ex.get("gender", "u")
        g_letter = GENDER_MAP.get(g_val, "u")

        out_wav = f"{OUT_DIR}/{utt_id}.wav"
        sf.write(out_wav, arr, sr)

        writer.writerow({
            "utt_id"    : utt_id,
            "path"      : out_wav,
            "duration"  : dur,
            "speaker_id": f"{g_letter}_{utt_id[:4]}",
            "gender"    : g_letter,
            "text"      : ex[TEXT_KEY],
            "label"     : "real"
        })

        secs_done += dur
        bar.update(dur)
        if secs_done >= secs_cap:
            break

print(f"\n✓ IndicTTS download finished — {secs_done/3600:.2f} h saved to {OUT_DIR}")
print("  Metadata appended to →", CSV_PATH)


IndicTTS DL:   0%|          | 33.165875/37800.0 [00:07<2:24:01,  4.37s/s]


KeyboardInterrupt: 

common voice

In [ ]:
CV_REPO = "mozilla-foundation/common_voice_17_0"   # ← change to 17_0 if you logged in
CV_LANG = "hi"                                     # Hindi config name
TOKEN_KW = {}                                      # populated only if you logged in
# If you ran `from huggingface_hub import login; login("YOUR_TOKEN")`
# and want v17, uncomment the next line:
# TOKEN_KW = {"token": True, "trust_remote_code": True}

# ─── local paths ────────────────────────────────────────────────────────
MOUNT_PATH  = "/content/drive"                      # or "/content/gdrive"
ROOT_DIR    = f"{MOUNT_PATH}/MyDrive/hindi_dfake"
OUT_DIR     = f"{ROOT_DIR}/raw/real_clean/commonvoice"
CSV_PATH    = f"{ROOT_DIR}/metadata/master_real.csv"

# ─── constants ──────────────────────────────────────────────────────────
TARGET_HOURS = 15
TEXT_KEY     = "sentence"
MIN_UVOTES   = 1           # ≥1 community up-vote
DUR_RANGE    = (1.0, 8.0)  # 1–8 s clips
ENERGY_TH    = 1e-4        # discard near-silence
GENDER_MAP   = {"male":"m","female":"f","other":"o"}

# ─── imports & setup ────────────────────────────────────────────────────
import os, csv, tqdm, hashlib, numpy as np, soundfile as sf
from datasets import load_dataset, Audio
os.makedirs(OUT_DIR, exist_ok=True)
need_header = not os.path.isfile(CSV_PATH)

# helper: simple RMS energy
def rms(arr): return np.sqrt(np.mean(arr.astype(np.float32)**2))

# filter for duration, up-votes, energy
def cv_filter(ex):
    dur = ex["audio"]["array"].shape[-1] / 16_000
    if not (DUR_RANGE[0] <= dur <= DUR_RANGE[1]): return False
    if ex.get("up_votes",0) < MIN_UVOTES:         return False
    if rms(ex["audio"]["array"]) < ENERGY_TH:     return False
    return True

# streaming iterator
cv_iter = (
    load_dataset(CV_REPO, CV_LANG,
                 split="validated", streaming=True,
                 **TOKEN_KW)
    .cast_column("audio", Audio(sampling_rate=16_000))
    .filter(cv_filter)
)

# ─── download loop ──────────────────────────────────────────────────────
secs_cap  = TARGET_HOURS * 3600
secs_done = 0

with open(CSV_PATH, "a", newline='') as csv_f, \
     tqdm.tqdm(total=secs_cap, unit="s", desc="CommonVoice DL") as bar:

    writer = csv.DictWriter(csv_f,
        fieldnames=["utt_id","path","duration","speaker_id",
                    "gender","text","label"])
    if need_header: writer.writeheader()

    for ex in cv_iter:
        arr = ex["audio"]["array"]; sr = 16_000
        dur = len(arr) / sr
        utt_id  = hashlib.sha1(ex["path"].encode()).hexdigest()[:16]
        gender  = GENDER_MAP.get(str(ex.get("gender","")).lower(), "u")

        out_wav = f"{OUT_DIR}/{utt_id}.wav"
        sf.write(out_wav, arr, sr)

        writer.writerow({
            "utt_id"   : utt_id,
            "path"     : out_wav,
            "duration" : dur,
            "speaker_id": ex.get("client_id","anon"),
            "gender"   : gender,
            "text"     : ex[TEXT_KEY],
            "label"    : "real"
        })

        secs_done += dur
        bar.update(dur)
        if secs_done >= secs_cap: break

print(f"\n✓ Common Voice download finished — {secs_done/3600:.2f} h saved to {OUT_DIR}")
print("  Metadata appended to →", CSV_PATH)


/usr/local/lib/python3.11/dist-packages/datasets/load.py:1491: FutureWarning: The repository for mozilla-foundation/common_voice_17_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_17_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


CommonVoice DL:   0%|          | 0/54000 [00:00<?, ?s/s]
Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 1it [00:00,  2.49it/s]
Reading metadata...: 10329it [00:00, 18452.01it/s]
CommonVoice DL:  84%|████████▍ | 45447.37199999915/54000 [03:38<00:41, 208.03s/s]


✓ Common Voice download finished — 12.62 h saved to /content/drive/MyDrive/hindi_dfake/raw/real_clean/commonvoice
  Metadata appended to → /content/drive/MyDrive/hindi_dfake/metadata/master_real.csv


In [ ]:
# === Colab one-cell: grab 2 real Hindi samples from FLEURS (hi_in) ===
# Safe: writes ONLY to a new folder under ROOT; does not modify your existing CSVs.

# 1) Config
ROOT = "/content/drive/MyDrive/hindi_dfake"    # change if your Drive path differs
OUT_DIR = f"{ROOT}/thirdparty/real_new_test/fleurs_hi_in"
N_SAMPLES = 2
RANDOM_SEED = 1337

# 2) Setup
import os, io, csv, random, pathlib, soundfile as sf, numpy as np
from pathlib import Path

# Mount Drive if needed
if not os.path.exists("/content/drive"):
    from google.colab import drive
    drive.mount("/content/drive")

os.makedirs(OUT_DIR, exist_ok=True)

# 3) Minimal deps
!pip -q install datasets==2.19.0 > /dev/null

from datasets import load_dataset, Audio

# 4) Load FLEURS (Hindi, India). License: CC-BY 4.0 (real speech).
# We'll take from train + validation + test to get a broad speaker pool.
ds = load_dataset("google/fleurs", "hi_in", split="train+validation+test")
# Ensure 16k mono
ds = ds.cast_column("audio", Audio(sampling_rate=16_000))

# 5) Filter: keep clips within a reasonable duration range (e.g., 1–8s)
MIN_S = 1.0
MAX_S = 8.0
def good_ex(ex):
    arr = ex["audio"]["array"]
    sr  = ex["audio"]["sampling_rate"]
    if arr is None: return False
    dur = len(arr) / float(sr)
    return (MIN_S <= dur <= MAX_S)

ds = ds.filter(good_ex)

# 6) Prefer unique speakers (if speaker_id exists)
if "speaker_id" in ds.column_names:
    seen = set()
    keep_idx = []
    for i, sid in enumerate(ds["speaker_id"]):
        sid = str(sid)
        if sid in seen:
            continue
        seen.add(sid)
        keep_idx.append(i)
    ds = ds.select(keep_idx)

# 7) Deterministic sample
random.seed(RANDOM_SEED)
idxs = list(range(len(ds)))
random.shuffle(idxs)
idxs = idxs[:N_SAMPLES]

# 8) Save WAV + a tiny, standalone metadata CSV (does NOT touch your main CSVs)
meta_path = Path(OUT_DIR) / "fleurs_hi_in_samples.meta.csv"
with open(meta_path, "w", newline="", encoding="utf-8") as f:
    w = csv.writer(f)
    w.writerow(["utt_id","speaker_id","path","seconds","text","source","license"])

    for i in idxs:
        ex  = ds[i]
        arr = ex["audio"]["array"]
        sr  = ex["audio"]["sampling_rate"]   # 16000 after cast
        # force mono float32 in [-1,1]
        if arr.ndim > 1:
            arr = arr.mean(axis=1)
        arr = np.clip(arr.astype(np.float32), -1.0, 1.0)
        seconds = round(len(arr) / float(sr), 3)

        spk = str(ex.get("speaker_id", f"fleurs_hi_spk{i}"))
        # FLEURS has "transcription" and "raw_transcription" fields; prefer "transcription"
        text = ex.get("transcription") or ex.get("raw_transcription") or ""

        # Build a deterministic utt_id
        utt_id = f"fleurs_hi_{spk}_{i}"
        out_wav = Path(OUT_DIR) / f"{utt_id}.wav"
        sf.write(out_wav, arr, sr, subtype="PCM_16")

        w.writerow([utt_id, spk, str(out_wav).replace("\\","/"), seconds, text, "google/fleurs:hi_in", "CC-BY-4.0"])

print(f"[done] Wrote {N_SAMPLES} WAVs + metadata:")
print(f"  OUT_DIR = {OUT_DIR}")
print(f"  META    = {meta_path}")
print("\nFirst few files:")
for p in sorted(Path(OUT_DIR).glob("*.wav"))[:10]:
    print(" ", p.name)

/usr/local/lib/python3.12/dist-packages/datasets/load.py:1486: FutureWarning: The repository for google/fleurs contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/google/fleurs
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Filter:   0%|          | 0/2777 [00:00<?, ? examples/s]

[done] Wrote 2 WAVs + metadata:
  OUT_DIR = /content/drive/MyDrive/hindi_dfake/thirdparty/real_new_test/fleurs_hi_in
  META    = /content/drive/MyDrive/hindi_dfake/thirdparty/real_new_test/fleurs_hi_in/fleurs_hi_in_samples.meta.csv

First few files:
  fleurs_hi_fleurs_hi_spk354_354.wav
  fleurs_hi_fleurs_hi_spk36_36.wav


In [ ]:
# === Colab one-cell: Bulk real Hindi (FLEURS hi_in) ~3000 clips ===
# Safe: writes ONLY to new folders; does NOT modify your existing CSVs.

# ---------- config ----------
ROOT         = "/content/drive/MyDrive/hindi_dfake"
OUT_DIR      = f"{ROOT}/raw/real_clean/thirdparty/fleurs_hi_in"
CSV_OUT      = f"{ROOT}/metadata/thirdparty_real_test.fleurs.csv"
N_TARGET     = 3000
RANDOM_SEED  = 1337
DUR_RANGE_S  = (1.0, 8.0)        # keep 1–8s
SOURCE_TAG   = "google/fleurs:hi_in"
LICENSE_TAG  = "CC-BY-4.0"

# ---------- setup ----------
import os, csv, random, hashlib, numpy as np, soundfile as sf
from pathlib import Path

# Mount Drive if needed
if not os.path.exists("/content/drive"):
    from google.colab import drive
    drive.mount("/content/drive")

os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(Path(CSV_OUT).parent, exist_ok=True)

# clean install datasets (keeps Colab pins)
!pip -q install datasets==2.19.0 > /dev/null

from datasets import load_dataset, Audio

# ---------- load & filter ----------
print("[load] FLEURS hi_in (train+validation+test)…")
ds = load_dataset("google/fleurs", "hi_in", split="train+validation+test")
ds = ds.cast_column("audio", Audio(sampling_rate=16_000))

MIN_S, MAX_S = DUR_RANGE_S
def good_ex(ex):
    a = ex["audio"]["array"]; sr = ex["audio"]["sampling_rate"]
    if a is None: return False
    d = len(a) / float(sr)
    return (MIN_S <= d <= MAX_S)

ds = ds.filter(good_ex)
print(f"[info] after dur filter: {len(ds)} rows")

# ---------- prefer unique speakers first ----------
rng = random.Random(RANDOM_SEED)
idx_all = list(range(len(ds)))
# build buckets by speaker if present
spk_col = "speaker_id" if "speaker_id" in ds.column_names else None
if spk_col:
    from collections import defaultdict
    by_spk = defaultdict(list)
    for i in idx_all:
        by_spk[str(ds[i].get(spk_col, f"spk?"))].append(i)
    speakers = list(by_spk.keys())
    rng.shuffle(speakers)

    chosen_idxs = []
    # 1) take one per speaker (unique speakers pass)
    for s in speakers:
        chosen_idxs.append(by_spk[s][0])
        if len(chosen_idxs) >= N_TARGET:
            break

    # 2) backfill (allow multiple per speaker) if needed
    if len(chosen_idxs) < N_TARGET:
        leftovers = []
        for s in speakers:
            leftovers.extend(by_spk[s][1:])  # remaining clips per spk
        rng.shuffle(leftovers)
        need = N_TARGET - len(chosen_idxs)
        chosen_idxs.extend(leftovers[:need])

else:
    # no speaker ids → just random sample up to N_TARGET
    rng.shuffle(idx_all)
    chosen_idxs = idx_all[:N_TARGET]

print(f"[pick] selected {len(chosen_idxs)} clips (target={N_TARGET})")

# ---------- save WAVs + CSV (standalone) ----------
def norm_text(ex):
    # FLEURS has "transcription" and "raw_transcription"
    return ex.get("transcription") or ex.get("raw_transcription") or ""

def to_wav_and_row(ex, idx):
    arr = ex["audio"]["array"]; sr = ex["audio"]["sampling_rate"]  # 16k after cast
    if arr.ndim > 1:
        arr = arr.mean(axis=1)
    arr = np.clip(arr.astype(np.float32), -1.0, 1.0)
    secs = round(len(arr)/float(sr), 3)

    spk = str(ex.get(spk_col, f"spk_{idx}"))
    # stable utt_id from original path if available, else from content hash
    src_path = str(ex.get("path", f"fleurs_hi_in_{idx}"))
    uid_base = hashlib.sha1(src_path.encode("utf-8")).hexdigest()[:16]
    utt_id = f"fleurs_hi_{uid_base}"

    out_wav = Path(OUT_DIR) / f"{utt_id}.wav"
    sf.write(str(out_wav), arr, sr, subtype="PCM_16")

    row = {
        "utt_id": utt_id,
        "path": str(out_wav).replace("\\","/"),
        "duration": secs,
        "speaker_id": spk,
        "gender": "u",
        "text": norm_text(ex),
        "label": "real",
        "source": SOURCE_TAG,
        "license": LICENSE_TAG
    }
    return row

rows = []
kept = 0
seen_paths = set()
for i in chosen_idxs:
    ex = ds[i]
    r = to_wav_and_row(ex, i)
    # avoid accidental duplicate paths
    if r["path"] in seen_paths:
        continue
    seen_paths.add(r["path"])
    rows.append(r); kept += 1

print(f"[save] writing CSV → {CSV_OUT}")
with open(CSV_OUT, "w", newline="", encoding="utf-8") as f:
    w = csv.DictWriter(f, fieldnames=[
        "utt_id","path","duration","speaker_id","gender","text","label","source","license"
    ])
    w.writeheader()
    w.writerows(rows)

# ---------- summary ----------
total_sec = sum(r["duration"] for r in rows)
print("\n[summary]")
print(f"  WAVs saved : {kept}")
print(f"  Hours      : {total_sec/3600:.2f}")
print(f"  Out folder : {OUT_DIR}")
print(f"  CSV        : {CSV_OUT}")

# show a few examples
from itertools import islice
print("\n[examples]")
for r in islice(rows, 5):
    print(" •", Path(r["path"]).name, "|", r["speaker_id"], "|", r["duration"], "s")

[load] FLEURS hi_in (train+validation+test)…


/usr/local/lib/python3.12/dist-packages/datasets/load.py:1486: FutureWarning: The repository for google/fleurs contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/google/fleurs
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Filter:   0%|          | 0/2777 [00:00<?, ? examples/s]

[info] after dur filter: 604 rows
[pick] selected 604 clips (target=3000)
[save] writing CSV → /content/drive/MyDrive/hindi_dfake/metadata/thirdparty_real_test.fleurs.csv

[summary]
  WAVs saved : 604
  Hours      : 1.10
  Out folder : /content/drive/MyDrive/hindi_dfake/raw/real_clean/thirdparty/fleurs_hi_in
  CSV        : /content/drive/MyDrive/hindi_dfake/metadata/thirdparty_real_test.fleurs.csv

[examples]
 • fleurs_hi_8b63ce7acd912273.wav | spk_354 | 7.56 s
 • fleurs_hi_6c47be90259fefd1.wav | spk_36 | 7.68 s
 • fleurs_hi_cb0357a131aedb4f.wav | spk_275 | 6.54 s
 • fleurs_hi_9f5518aed01136d1.wav | spk_284 | 6.18 s
 • fleurs_hi_f973483f16007db9.wav | spk_255 | 5.76 s


In [ ]:
# === FLEURS audit + build a dedicated test-real CSV (FLEURS-only for now) ===

import os, pandas as pd, soundfile as sf
from pathlib import Path

ROOT      = "/content/drive/MyDrive/hindi_dfake"
FLEURS_CSV= f"{ROOT}/metadata/thirdparty_real_test.fleurs.csv"   # from your earlier one-cell
TEST_REAL = f"{ROOT}/metadata/test_real.csv"                      # dedicated CSV we’ll build/refresh
REPORT_MISS = True                                                # log missing files if any

assert os.path.exists(os.path.dirname(TEST_REAL)), "metadata folder not found"

if not os.path.isfile(FLEURS_CSV):
    raise FileNotFoundError(f"FLEURS CSV not found at: {FLEURS_CSV}")

fleurs = pd.read_csv(FLEURS_CSV)

# Optional: verify files exist and durations are plausible
exists_mask = fleurs["path"].apply(lambda p: os.path.isfile(p))
missing = fleurs[~exists_mask]
present = fleurs[exists_mask].copy()

# If duration column exists, keep it; else compute quickly (reading headers only)
def probe_dur(p):
    try:
        info = sf.info(p)
        return float(info.duration)
    except Exception:
        return None

if "duration" not in present.columns or present["duration"].isnull().any():
    present["duration"] = present["path"].apply(probe_dur)

# Summary
n_total   = len(fleurs)
n_present = len(present)
n_missing = len(missing)
hrs_total = (present["duration"].fillna(0).sum() / 3600.0)

print("🔎 FLEURS audit")
print(f"  rows in CSV        : {n_total}")
print(f"  files present      : {n_present}")
print(f"  files missing      : {n_missing}")
print(f"  unique speakers    : {present['speaker_id'].astype(str).nunique()}")
print(f"  hours (present)    : {hrs_total:.2f} h")

if REPORT_MISS and n_missing:
    print("\n⚠️ Missing files (showing up to 10):")
    for p in missing["path"].head(10).tolist():
        print("  -", p)

# Build/refresh a dedicated test-real CSV with just FLEURS for now
cols = ["utt_id","path","duration","speaker_id","gender","text","label","source","license"]
for c in cols:
    if c not in present.columns:
        present[c] = "" if c != "duration" else 0.0

present = present[cols].copy()
present["label"] = "real"
present["source"] = present.get("source", "").replace("", "google/fleurs:hi_in")
present["license"] = present.get("license", "").replace("", "CC-BY-4.0")

present.to_csv(TEST_REAL, index=False)
print(f"\n✅ Wrote dedicated test-real CSV (FLEURS only) → {TEST_REAL}")

🔎 FLEURS audit
  rows in CSV        : 604
  files present      : 604
  files missing      : 0
  unique speakers    : 604
  hours (present)    : 1.10 h

✅ Wrote dedicated test-real CSV (FLEURS only) → /content/drive/MyDrive/hindi_dfake/metadata/test_real.csv


In [ ]:
from huggingface_hub import login
login()

In [ ]:
import pandas as pd

CSV = "/content/drive/MyDrive/hindi_dfake/metadata/master_real.csv"
df = pd.read_csv(CSV)

def detect_source(p):
    p = str(p).lower()
    if "indictts" in p: return "IndicTTS"
    if "ivr" in p or "indicvoices" in p: return "IndicVoices_R"
    if "commonvoice" in p: return "CommonVoice"
    return "Other"

df["source"] = df["path"].apply(detect_source)
df["speaker_id"] = df["speaker_id"].astype(str)

stats = (
    df.groupby("source")
      .agg(total_clips=("speaker_id","count"),
           distinct_speakers=("speaker_id","nunique"))
      .reset_index()
)
stats["avg_clips_per_speaker"] = (stats["total_clips"] / stats["distinct_speakers"]).round(2)
print(stats)

          source  total_clips  distinct_speakers  avg_clips_per_speaker
0    CommonVoice        10088                302                  33.40
1       IndicTTS         4703               4548                   1.03
2  IndicVoices_R        11174                344                  32.48


In [ ]:
# === Verify uniqueness “is real” for CommonVoice & IndicVoices_R (from your master_real.csv) ===

import pandas as pd

ROOT      = "/content/drive/MyDrive/hindi_dfake"
MASTER    = f"{ROOT}/metadata/master_real.csv"

df = pd.read_csv(MASTER)
df["speaker_id"] = df["speaker_id"].astype(str)

def detect_source(p):
    p = str(p).lower()
    if "indictts" in p: return "IndicTTS"
    if "ivr" in p or "indicvoices" in p: return "IndicVoices_R"
    if "commonvoice" in p: return "CommonVoice"
    return "Other"

df["source"] = df["path"].apply(detect_source)

# Focus only on CommonVoice & IndicVoices_R
sub = df[df["source"].isin(["CommonVoice","IndicVoices_R"])].copy()

# Basic stats
stats = (
    sub.groupby("source")
       .agg(total_clips=("utt_id","count"),
            distinct_speakers=("speaker_id","nunique"))
       .reset_index()
)
stats["avg_clips_per_speaker"] = (stats["total_clips"]/stats["distinct_speakers"]).round(2)
print("📊 Diversity by source (from your saved data):")
print(stats.to_string(index=False))

# Show sample speakers with multiple clips (proves not minted-per-file)
def show_multi(source_name, k=5):
    print(f"\n🔍 Speakers with multiple clips in {source_name}:")
    grp = (sub[sub["source"].eq(source_name)]
           .value_counts("speaker_id").reset_index(name="n")
           .sort_values("n", ascending=False))
    multi = grp[grp["n"] > 1].head(k)
    if multi.empty:
        print("  (none >1 — then IDs may be per-file in this source copy)")
        return
    print(multi.to_string(index=False))

show_multi("CommonVoice", k=10)
show_multi("IndicVoices_R", k=10)

# Optional: sanity—list how many speakers appear exactly once vs >1
for src in ["CommonVoice","IndicVoices_R"]:
    g = (sub[sub["source"].eq(src)]
         .value_counts("speaker_id").reset_index(name="n"))
    once = (g["n"]==1).sum()
    more = (g["n"]>1).sum()
    print(f"\n{src}: speakers appearing once={once}, more_than_once={more}")

📊 Diversity by source (from your saved data):
       source  total_clips  distinct_speakers  avg_clips_per_speaker
  CommonVoice        10088                302                  33.40
IndicVoices_R        11174                344                  32.48

🔍 Speakers with multiple clips in CommonVoice:
                                                                                                                      speaker_id    n
3c47037b539064a039337a1c897e88029bf7cfb906524864e55c38b90aa9efe57730742089a7f2a1b508952b6272c39cd50f7d701230bde9d0afdf4fefc05d8a 2066
d5880664e0b201cf24b4972ea9890b801afe05af05cd4302d95952f06934198872328efbf6eb11bf51e169d4c62a0e8756088bbe001fc353a9b6d080ed9ee3dc 1696
0f018a99663f33afbb7d38aee281fb1afcfd07f9e7acd00383f604e1e17c38d6ed8adf1bd2ccbf927a52c5adefb8ac4b158ce27a7c2ed9581e71202eb302dfb3  943
ec293248c910c28091014fec13d47bb69cc84f0b265ad0536d7e39390f3de510ab879fa3c23797defe85d8c846a4e148c971e28da6a90a647a9bc37eb03362bc  717
76ae0779e8b42a6cebddc96f1c8be

In [ ]:
# === Build ~3,000 REAL test set ===
# - Keep existing FLEURS (~600) as-is (needs preprocessing/features later)
# - Move the rest (~2,400) by WHOLE speakers from CV + IVR
# - Ascending by speaker size, round-robin between sources
# - Non-destructive: write sidecar CSVs

import os
import pandas as pd
from collections import deque
from pathlib import Path

ROOT = "/content/drive/MyDrive/hindi_dfake"
MASTER_REAL = f"{ROOT}/metadata/master_real.csv"
FLEURS_CSV  = f"{ROOT}/metadata/thirdparty_real_test.fleurs.csv"   # from your FLEURS cell
OUT_TEST_FROM_TRAIN = f"{ROOT}/metadata/test_real.from_train.roundrobin_least_damage.csv"
OUT_TEST_UNION      = f"{ROOT}/metadata/test_real.union.csv"
OUT_TRAIN_AFTER     = f"{ROOT}/metadata/train_real.after_handoff.roundrobin_least_damage.csv"

TARGET_TEST_TOTAL = 3000
ALLOW_OVERSHOOT   = 0.05   # allow up to +5% because we move whole speakers
SMALL_SPK_MAX_N   = 40     # cap for "small" speakers to avoid mega-speakers
MIN_SPK_N         = 2      # ignore singletons if you prefer (set to 1 to allow)

def detect_source(p):
    p = str(p).lower()
    if "indictts" in p: return "IndicTTS"
    if "ivr" in p or "indicvoices" in p: return "IndicVoices_R"
    if "commonvoice" in p: return "CommonVoice"
    if "fleurs" in p: return "FLEURS"
    return "Other"

# 1) Load master + fleurs (real rows only)
df = pd.read_csv(MASTER_REAL)
df["label"] = df["label"].astype(str).str.lower()
df["source"] = df["path"].apply(detect_source)
df["speaker_id"] = df["speaker_id"].astype(str)
df_real = df[df["label"].eq("real")].copy()

fleurs = pd.DataFrame()
if os.path.isfile(FLEURS_CSV):
    fleurs = pd.read_csv(FLEURS_CSV)
    fleurs["speaker_id"] = fleurs["speaker_id"].astype(str)
    fleurs["source"] = fleurs.get("source", "FLEURS")
    fleurs = fleurs.copy()
else:
    print("⚠️ FLEURS CSV not found; proceeding with 0 FLEURS rows.")

n_fleurs = len(fleurs)
need_from_train = max(0, TARGET_TEST_TOTAL - n_fleurs)
allow_max_total = int(TARGET_TEST_TOTAL * (1 + ALLOW_OVERSHOOT))

print(f"FLEURS present: {n_fleurs} rows")
print(f"Target total test: {TARGET_TEST_TOTAL} → need from CV+IVR: {need_from_train} rows (max allowed total={allow_max_total})")

# 2) Candidate pool = CV + IVR rows from your master (already preprocessed/features exist)
pool = df_real[df_real["source"].isin(["CommonVoice","IndicVoices_R"])].copy()

# per-speaker counts and source
g = (pool.groupby(["speaker_id","source"])
          .agg(n=("utt_id","count"))
          .reset_index())

# filter to "small" speakers first to minimize disruption
g_small = g[(g["n"] >= MIN_SPK_N) & (g["n"] <= SMALL_SPK_MAX_N)].copy()

# build round-robin queues, smallest-first inside each source
queues = {
    "CommonVoice": deque(g_small[g_small["source"].eq("CommonVoice")].sort_values("n")["speaker_id"].tolist()),
    "IndicVoices_R": deque(g_small[g_small["source"].eq("IndicVoices_R")].sort_values("n")["speaker_id"].tolist()),
}

# selection loop: alternate sources to keep mix; stop when need_from_train met
order = ["CommonVoice", "IndicVoices_R"]
chosen_spk = []
chosen_rows = 0
need = need_from_train

# quick map for n per speaker
n_per_spk = { (row["speaker_id"]): int(row["n"]) for _, row in g_small.iterrows() }

while need > 0 and (queues["CommonVoice"] or queues["IndicVoices_R"]):
    for src in order:
        if need <= 0: break
        if queues[src]:
            spk = queues[src].popleft()
            chosen_spk.append(spk)
            chosen_rows += n_per_spk[spk]
            need -= n_per_spk[spk]

# If still under target (e.g., not enough small speakers), backfill with larger speakers (still round-robin)
if chosen_rows < need_from_train:
    remainder = g[(~g["speaker_id"].isin(chosen_spk)) & (g["n"] > SMALL_SPK_MAX_N)].copy()
    queues_big = {
        "CommonVoice": deque(remainder[remainder["source"].eq("CommonVoice")].sort_values("n")["speaker_id"].tolist()),
        "IndicVoices_R": deque(remainder[remainder["source"].eq("IndicVoices_R")].sort_values("n")["speaker_id"].tolist()),
    }
    need2 = need_from_train - chosen_rows
    while need2 > 0 and (queues_big["CommonVoice"] or queues_big["IndicVoices_R"]):
        for src in order:
            if need2 <= 0: break
            if queues_big[src]:
                spk = queues_big[src].popleft()
                chosen_spk.append(spk)
                chosen_rows += int(g.loc[g["speaker_id"].eq(spk), "n"].iloc[0])
                need2 -= int(g.loc[g["speaker_id"].eq(spk), "n"].iloc[0])

print(f"\nSelected speakers: {len(chosen_spk)}")
print(f"Selected rows from CV+IVR: {chosen_rows}  (target from CV+IVR={need_from_train})")

# guard overshoot of TOTAL (FLEURS + moved)
total_after = n_fleurs + chosen_rows
if total_after > allow_max_total:
    print(f"⚠️ Overshoot: total test would be {total_after} (> {allow_max_total}). "
          f"Consider lowering SMALL_SPK_MAX_N or TARGET_TEST_TOTAL.")
else:
    print(f"Projected total test (FLEURS + moved): {total_after} rows")

# 3) Build the splits
move_mask = pool["speaker_id"].isin(chosen_spk)
df_test_from_train = pool[move_mask].copy()
df_train_after     = pd.concat([df_real[~df_real["source"].isin(["CommonVoice","IndicVoices_R"])],  # keep IndicTTS, Other
                               pool[~move_mask]], ignore_index=True)

# union test (FLEURS + moved)
cols = ["utt_id","path","duration","speaker_id","gender","text","label","source","license"]
def ensure_cols(x):
    for c in cols:
        if c not in x.columns:
            x[c] = "" if c != "duration" else 0.0
    return x[cols].copy()

df_test_from_train = ensure_cols(df_test_from_train)
df_test_from_train["label"] = "real"

fleurs_aligned = pd.DataFrame()
if len(fleurs):
    fleurs_aligned = ensure_cols(fleurs.copy())
    fleurs_aligned["label"] = "real"

df_test_union = pd.concat([fleurs_aligned, df_test_from_train], ignore_index=True)

# 4) Save sidecars
Path(OUT_TEST_FROM_TRAIN).parent.mkdir(parents=True, exist_ok=True)
df_test_from_train.to_csv(OUT_TEST_FROM_TRAIN, index=False)
df_test_union.to_csv(OUT_TEST_UNION, index=False)
df_train_after.to_csv(OUT_TRAIN_AFTER, index=False)

# 5) Report + safety
def brief(name, d):
    return {k:int(v) for k,v in d.value_counts().to_dict().items()}

print("\n=== SUMMARY ===")
print(f"FLEURS test (already in CSV): {n_fleurs}")
print(f"Moved from train → test: {len(df_test_from_train)}")
print(f"Total test (union): {len(df_test_union)}")
print(f"Train after handoff: {len(df_train_after)}")
print("\nBy source in moved-to-test:", brief("src", df_test_from_train["source"]))
print("By source in train-after  :", brief("src", df_train_after["source"]))

# Speaker-disjointness check (train-after vs test-union)
assert set(df_test_union["speaker_id"]).isdisjoint(set(df_train_after["speaker_id"])), "Not speaker-disjoint!"
print("\n✅ Speaker-disjointness verified.")

FLEURS present: 604 rows
Target total test: 3000 → need from CV+IVR: 2396 rows (max allowed total=3150)

Selected speakers: 266
Selected rows from CV+IVR: 2418  (target from CV+IVR=2396)
Projected total test (FLEURS + moved): 3022 rows

=== SUMMARY ===
FLEURS test (already in CSV): 604
Moved from train → test: 2418
Total test (union): 3022
Train after handoff: 23547

By source in moved-to-test: {'IndicVoices_R': 1950, 'CommonVoice': 468}
By source in train-after  : {'CommonVoice': 9620, 'IndicVoices_R': 9224, 'IndicTTS': 4703}

✅ Speaker-disjointness verified.


# **FAKE FAKE FAKE FAKE FAKE FAKE FAKE FAKE FAKE FAKE FAKE FAKE FAKE FAKE FAKE FAKE**

In [ ]:
!pip install -q kaggle

# 1️⃣  Import the helper
from google.colab import files

# 2️⃣  Upload kaggle.json
files.upload()          # pick the kaggle.json from your laptop

# 3️⃣  Move & set permissions
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle.json


In [ ]:
!apt-get -qq install ffmpeg

In [ ]:
!pip install -q kagglehub

In [ ]:
!kaggle config view

Configuration values from /root/.kaggle
- username: anishdb
- path: None
- proxy: None
- competition: None


In [ ]:
!kaggle datasets list -s titanic | head

ref                                  title                                                size  lastUpdated                 downloadCount  voteCount  usabilityRating  
-----------------------------------  ---------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
heptapod/titanic                     Titanic                                             11090  2017-05-16 08:14:22.210000         121068       1671  0.7058824        
brendan45774/test-file               Titanic dataset                                     11514  2021-12-02 16:11:42.367000         187964       1472  1.0              
yasserh/titanic-dataset              Titanic Dataset                                     22564  2021-12-24 14:53:06.913000         178881        562  1.0              
azeembootwala/titanic                Titanic                                             12406  2017-06-05 12:14:37.477000          24100        199  0.8235294 

In [ ]:
!kaggle datasets files -d naman3007/hav-df-dataset

Next Page Token = CfDJ8H3GHv39YMxDjEpKbXZX9wGfVKYaffZLXcsQbF-cfTnLie2tQcrku0h6R2qUgSnjcrmFTP8LRdj7k9zqeLldno0
name                                    size  creationDate                
---------------------------------  ---------  --------------------------  
HAV-DF/test_videos/09c5dpye7s.mp4    2062867  2025-05-29 18:32:15.677000  
HAV-DF/test_videos/0icpg7s3wk.mp4   33566960  2025-05-29 18:32:17.203000  
HAV-DF/test_videos/2jfpk79in9.mp4     761426  2025-05-29 18:32:15.916000  
HAV-DF/test_videos/4pf3zf7jkl.mp4    1318054  2025-05-29 18:32:15.913000  
HAV-DF/test_videos/5n0npxwhr0.mp4     779350  2025-05-29 18:32:15.961000  
HAV-DF/test_videos/8lk8jqj2x1.mp4   38418818  2025-05-29 18:32:16.429000  
HAV-DF/test_videos/afdvusntas.mp4   35965813  2025-05-29 18:32:16.516000  
HAV-DF/test_videos/akfmhpsggq.mp4   33566960  2025-05-29 18:32:16.450000  
HAV-DF/test_videos/akqllxlxsb.mp4    8760159  2025-05-29 18:32:16.185000  
HAV-DF/test_videos/amfdrorkqd.mp4    2008473  2025-05-29 18:32:16

In [ ]:
import json, itertools, textwrap, pprint

meta_path = "/content/hav-df-dataset-metadata.json"    # your croissant file
with open(meta_path) as f:
    meta = json.load(f)

print(meta.keys())               # top-level keys
# show first 3 resources
for res in itertools.islice(meta.get("resources", []), 3):
    pprint.pprint(res)
    print("-"*60)

dict_keys(['@context', 'alternateName', 'citeAs', 'conformsTo', 'license', 'distribution', 'recordSet', 'version', 'keywords', 'identifier', 'isAccessibleForFree', 'includedInDataCatalog', 'creator', 'publisher', 'thumbnailUrl', 'dateModified', 'datePublished', '@type', 'name', 'url', 'description'])


In [ ]:
# ─── CONFIG ─────────────────────────────────────────────────────────────
HOURS_TARGET  = 25                                 # change if you want more
HAVDF_ROOT    = path                               # path from kagglehub download
META_CSV_SRC  = "/content/video_metadata.csv"     # your uploaded CSV
ROOT_DIR      = "/content/drive/MyDrive/hindi_dfake"
RAW_DIR       = f"{ROOT_DIR}/raw/fake_havdf"
META_OUT      = f"{ROOT_DIR}/metadata/master_fake.csv"

# ─── DEPS ────────────────────────────────────────────────────────────────
!apt-get -qq install ffmpeg >/dev/null
!pip -q install soundfile >/dev/null

import os, glob, csv, hashlib, tqdm, subprocess
import pandas as pd, numpy as np, soundfile as sf

os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(os.path.dirname(META_OUT), exist_ok=True)

# ─── 1. Load label CSV and collect FAKE filenames ───────────────────────
df = pd.read_csv(META_CSV_SRC)
fake_set = set(df.loc[df.label.str.upper() == "FAKE", "video_name"])
print(f"{len(fake_set)} clips labelled FAKE in metadata CSV.")

# ─── 2. Locate those MP4 files on disk ──────────────────────────────────
mp4_list = [f for f in glob.glob(f"{HAVDF_ROOT}/**/*.mp4", recursive=True)
            if os.path.basename(f) in fake_set]
print(f"Found {len(mp4_list)} FAKE MP4 files on disk.")

# ─── 3. Extraction loop (resume-safe) ───────────────────────────────────
secs_cap  = HOURS_TARGET * 3600
secs_done = 0
need_head = not os.path.isfile(META_OUT)

with open(META_OUT, "a", newline="") as csv_f, \
     tqdm.tqdm(total=secs_cap, unit="s") as bar:

    writer = csv.DictWriter(csv_f,
        fieldnames=["utt_id","path","duration","speaker_id",
                    "gender","text","label","fake_type"])
    if need_head:
        writer.writeheader()

    for mp4_path in mp4_list:
        if secs_done >= secs_cap:
            break

        fname   = os.path.basename(mp4_path)
        utt_id  = "hav_" + hashlib.sha1(fname.encode()).hexdigest()[:16]
        out_wav = f"{RAW_DIR}/{utt_id}.wav"

        # ---- resume safety ----
        if os.path.isfile(out_wav):
            d, sr = sf.read(out_wav); secs = len(d)/sr
            secs_done += secs; bar.update(secs); continue

        # ---- strip audio to 16 kHz mono PCM-16 ----
        subprocess.run(
            ["ffmpeg","-loglevel","quiet","-y",
             "-i", mp4_path, "-vn",
             "-ac","1","-ar","16000","-sample_fmt","s16", out_wav],
            check=True)

        data, sr = sf.read(out_wav)
        dur = len(data) / sr
        if dur < 1.0 or np.abs(data).mean() < 1e-4:
            os.remove(out_wav); continue  # skip silence / errors

        # ─── optional windowing (uncomment to split into 6 s chunks) ───
        # WIN = 96000        # 6 s × 16 kHz
        # STEP = 64000       # 4 s hop (2 s overlap)
        # for i in range(0, len(data)-WIN+1, STEP):
        #     seg = data[i:i+WIN]
        #     seg_id = f"{utt_id}_{i:04d}"
        #     seg_path = f"{RAW_DIR}/{seg_id}.wav"
        #     sf.write(seg_path, seg, sr)
        #     writer.writerow({"utt_id":seg_id,"path":seg_path,
        #                      "duration":WIN/sr,"speaker_id":"havdf",
        #                      "gender":"u","text":"",
        #                      "label":"fake","fake_type":"havdf_win"})
        # secs = len(data)
        # secs_done += secs / sr
        # bar.update(secs / sr)
        # continue   # comment this line if you keep full-length tracks

        # ---- full-track metadata  --------------------------------------
        writer.writerow({
            "utt_id"    : utt_id,
            "path"      : out_wav,
            "duration"  : dur,
            "speaker_id": "havdf",
            "gender"    : "u",
            "text"      : "",
            "label"     : "fake",
            "fake_type" : "havdf"
        })

        secs_done += dur
        bar.update(dur)

print(f"\n✓ Extracted ≈ {secs_done/3600:.1f} h of HAV-DF audio → {RAW_DIR}")
print("  Metadata appended →", META_OUT)


322 clips labelled FAKE in metadata CSV.
Found 322 FAKE MP4 files on disk.


  5%|▌         | 4808.759625000002/90000 [03:27<1:01:15, 23.18s/s]


✓ Extracted ≈ 1.3 h of HAV-DF audio → /content/drive/MyDrive/hindi_dfake/raw/fake_havdf
  Metadata appended → /content/drive/MyDrive/hindi_dfake/metadata/master_fake.csv


Edge TTS test run

In [ ]:
# =================== HINDI TTS VIA EDGE-TTS (Colab-safe, resume-safe) ===================
# Output WAVs:  /content/drive/MyDrive/hindi_dfake/raw/fake_tts
# Metadata:     /content/drive/MyDrive/hindi_dfake/metadata/master_fake.csv
# Reads texts:  /content/drive/MyDrive/hindi_dfake/metadata/master_real.csv
# ========================================================================================

# --- CONFIG ---
TARGET_HOURS   = 2             # start small; re-run later to accumulate hours
MOUNT_PATH     = "/content/drive"
ROOT_DIR       = f"{MOUNT_PATH}/MyDrive/hindi_dfake"
REAL_CSV       = f"{ROOT_DIR}/metadata/master_real.csv"
OUT_DIR        = f"{ROOT_DIR}/raw/fake_tts"
FAKE_CSV       = f"{ROOT_DIR}/metadata/master_fake.csv"

SEED           = 2025
MIN_CHARS, MAX_CHARS = 6, 180
MAX_CONCURRENT = 2             # keep low to avoid throttling (1–3 is safe)
VOICE_RATE     = "+0%"         # e.g., "+10%" or "-10%" to vary speed

# --- INSTALLS (idempotent) ---
!pip -q install edge-tts tqdm soundfile pandas numpy nest_asyncio
!apt-get -qq install ffmpeg >/dev/null

# --- IMPORTS ---
import os, csv, glob, hashlib, random, tempfile, subprocess
from pathlib import Path
import pandas as pd, numpy as np, soundfile as sf
from tqdm.auto import tqdm

import asyncio, nest_asyncio, edge_tts
nest_asyncio.apply()
loop = asyncio.get_event_loop()

random.seed(SEED); np.random.seed(SEED)
os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(Path(FAKE_CSV).parent, exist_ok=True)

# --- 1) Load texts from your real CSV ---
def load_texts(csv_path):
    df = pd.read_csv(csv_path)
    col = next((c for c in ["text","transcript","sentence"] if c in df.columns), None)
    assert col, "master_real.csv must have text/transcript/sentence column."
    texts = (df[col].astype(str).str.replace("\n"," ").str.strip()
             .dropna().loc[lambda s: s.str.len().between(MIN_CHARS, MAX_CHARS)]
             .drop_duplicates().tolist())
    random.shuffle(texts)
    return texts

texts = load_texts(REAL_CSV)
print(f"Loaded {len(texts)} unique sentences.")

# --- 2) Discover Hindi voices (Colab-safe) ---
async def get_hi_voices():
    try:
        voices = await edge_tts.list_voices()
        return [v["ShortName"] for v in voices if str(v.get("Locale","")).lower().startswith("hi-")]
    except Exception:
        return []

hi_voices = loop.run_until_complete(get_hi_voices())
if not hi_voices:
    # Fallback to common voices if listing fails
    hi_voices = ["hi-IN-MadhurNeural", "hi-IN-SwaraNeural"]
print("Hindi voices:", hi_voices)

# --- 3) Resume accounting (seconds already on disk) ---
secs_cap  = TARGET_HOURS * 3600
secs_done = 0.0
for fn in os.listdir(OUT_DIR):
    if fn.endswith(".wav"):
        try:
            d, sr = sf.read(os.path.join(OUT_DIR, fn))
            secs_done += len(d)/sr
        except: pass
print(f"Already on disk: {secs_done/3600:.2f} h")

# --- 4) Metadata prep ---
need_header = not os.path.isfile(FAKE_CSV)
fields = ["utt_id","path","duration","speaker_id","gender","text","label",
          "fake_type","tts_model","voice","rate","seed"]

# --- 5) Async synth helpers (no asyncio.run anywhere) ---
async def synth_one(text, voice, sem, pbar):
    # Wrap each call with a semaphore to throttle concurrency
    async with sem:
        key = f"{text}|{voice}|{VOICE_RATE}|{SEED}"
        uid = "tts_edge_" + hashlib.sha1(key.encode("utf-8")).hexdigest()[:16]
        out_wav = os.path.join(OUT_DIR, f"{uid}.wav")

        # Resume: if file exists, count it and return
        if os.path.isfile(out_wav):
            try:
                d, sr = sf.read(out_wav); dur = len(d)/sr
                pbar.update(dur);
                await asyncio.sleep(0.05)
                return uid, out_wav, dur, text, voice
            except: pass

        # edge-tts returns compressed; save to temp mp3 first
        tmp_mp3 = os.path.join(tempfile.gettempdir(), f"{uid}.mp3")
        try:
            await edge_tts.Communicate(text=text, voice=voice, rate=VOICE_RATE).save(tmp_mp3)
        except Exception:
            # Network hiccup or throttle; back off a bit
            await asyncio.sleep(0.8)
            return None

        # Convert to 16 kHz mono WAV
        try:
            subprocess.run(
                ["ffmpeg","-loglevel","quiet","-y","-i", tmp_mp3, "-ac","1","-ar","16000", out_wav],
                check=True
            )
        except Exception:
            try: os.remove(tmp_mp3)
            except: pass
            return None
        finally:
            try: os.remove(tmp_mp3)
            except: pass

        # Quality gate
        try:
            data, sr = sf.read(out_wav); dur = len(data)/sr
            if dur < 0.8 or float(np.abs(data).mean()) < 1e-4:
                os.remove(out_wav);
                await asyncio.sleep(0.05)
                return None
        except Exception:
            try: os.remove(out_wav)
            except: pass
            return None

        pbar.update(dur)
        # Tiny delay to be gentle with the service
        await asyncio.sleep(0.1)
        return uid, out_wav, dur, text, voice

async def run_batch():
    global secs_done
    sem = asyncio.Semaphore(MAX_CONCURRENT)

    # endless shuffled text stream
    def text_stream(items):
        while True:
            for t in items: yield t
            random.shuffle(items)
    ts = text_stream(texts)

    with open(FAKE_CSV, "a", newline="", encoding="utf-8") as fcsv:
        writer = csv.DictWriter(fcsv, fieldnames=fields)
        if need_header: writer.writeheader()

        with tqdm(total=max(0, secs_cap - secs_done), unit="s", desc="Edge-TTS hours") as pbar:
            while secs_done < secs_cap:
                # Submit a small wave of tasks (keeps memory small)
                tasks = [synth_one(next(ts), random.choice(hi_voices), sem, pbar)
                         for _ in range(MAX_CONCURRENT)]
                results = await asyncio.gather(*tasks)
                for res in results:
                    if not res:
                        continue
                    uid, out_wav, dur, text, voice = res
                    secs_done += dur
                    writer.writerow({
                        "utt_id": uid, "path": out_wav, "duration": round(dur,3),
                        "speaker_id": "edge_tts", "gender": "u", "text": text,
                        "label": "fake", "fake_type": "tts_edge",
                        "tts_model": "edge-tts", "voice": voice, "rate": VOICE_RATE,
                        "seed": SEED
                    })
                    print(f"made {uid} | {dur:.2f}s | total {secs_done/3600:.2f}h")
                    if secs_done >= secs_cap:
                        break

# ---- GO (Colab-safe loop driving) ----
loop.run_until_complete(run_batch())
print(f"\n✓ Done — total now ≈ {secs_done/3600:.2f} h in {OUT_DIR}")
print("Metadata →", FAKE_CSV)

Loaded 23551 unique sentences.
Hindi voices: ['hi-IN-MadhurNeural', 'hi-IN-SwaraNeural']
Already on disk: 0.00 h


Edge-TTS hours:   0%|          | 0/7200.0 [00:00<?, ?s/s]

made tts_edge_b183f7a0bb78a442 | 8.54s | total 0.00h
made tts_edge_e936abe720ba572c | 3.58s | total 0.00h
made tts_edge_0bb2dd6800953134 | 10.97s | total 0.01h
made tts_edge_38e94eed70f2b2ff | 3.12s | total 0.01h
made tts_edge_3f8c05effa8e2c50 | 6.02s | total 0.01h
made tts_edge_e0da6e0c3716990f | 4.97s | total 0.01h
made tts_edge_65bd59ae8069e007 | 2.45s | total 0.01h
made tts_edge_f586f3b3d3e4e66c | 4.78s | total 0.01h
made tts_edge_9881463a3d1a40da | 13.92s | total 0.02h
made tts_edge_c7df70e17d1cba47 | 13.82s | total 0.02h
made tts_edge_574a8460a1f4153d | 3.53s | total 0.02h
made tts_edge_ac9500f130bcc177 | 2.26s | total 0.02h
made tts_edge_33980b0ff01b0a5f | 5.45s | total 0.02h
made tts_edge_202f11cfaaa1c378 | 6.60s | total 0.03h
made tts_edge_f449322634fc3724 | 11.35s | total 0.03h
made tts_edge_8f389be320744402 | 5.57s | total 0.03h
made tts_edge_a35e38bd9e4bfb8b | 3.41s | total 0.03h
made tts_edge_a4f19038913c89bf | 4.39s | total 0.03h
made tts_edge_d369b1be3698aafe | 10.73s | 

Edge TTS bulk

In [ ]:
# =================== OPTIMIZED HINDI TTS VIA EDGE-TTS ===================
# Output WAVs:  /content/drive/MyDrive/hindi_dfake/raw/fake_tts
# Metadata:     /content/drive/MyDrive/hindi_dfake/metadata/master_fake.csv
# Reads texts:  /content/drive/MyDrive/hindi_dfake/metadata/master_real.csv
# =========================================================================

# --- OPTIMIZED CONFIG ---
TARGET_HOURS   = 55
MOUNT_PATH     = "/content/drive"
ROOT_DIR       = f"{MOUNT_PATH}/MyDrive/hindi_dfake"
REAL_CSV       = f"{ROOT_DIR}/metadata/master_real.csv"
OUT_DIR        = f"{ROOT_DIR}/raw/fake_tts"
FAKE_CSV       = f"{ROOT_DIR}/metadata/master_fake.csv"

SEED           = 2025
MIN_CHARS, MAX_CHARS = 6, 180
MAX_CONCURRENT = 15            # OPTIMIZED: Up from 2
BATCH_SIZE     = 30            # OPTIMIZED: Process larger batches
VOICE_RATE     = "+0%"

# --- INSTALLS (idempotent) ---
!pip -q install edge-tts tqdm soundfile pandas numpy nest_asyncio
!apt-get -qq install ffmpeg >/dev/null

# --- IMPORTS ---
import os, csv, glob, hashlib, random, tempfile, subprocess
from pathlib import Path
import pandas as pd, numpy as np, soundfile as sf
from tqdm.auto import tqdm

import asyncio, nest_asyncio, edge_tts
nest_asyncio.apply()
loop = asyncio.get_event_loop()

random.seed(SEED); np.random.seed(SEED)
os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(Path(FAKE_CSV).parent, exist_ok=True)

# --- 1) Load texts from your real CSV ---
def load_texts(csv_path):
    df = pd.read_csv(csv_path)
    col = next((c for c in ["text","transcript","sentence"] if c in df.columns), None)
    assert col, "master_real.csv must have text/transcript/sentence column."
    texts = (df[col].astype(str).str.replace("\n"," ").str.strip()
             .dropna().loc[lambda s: s.str.len().between(MIN_CHARS, MAX_CHARS)]
             .drop_duplicates().tolist())
    random.shuffle(texts)
    return texts

texts = load_texts(REAL_CSV)
print(f"Loaded {len(texts)} unique sentences.")

# --- 2) Discover Hindi voices (Colab-safe) ---
async def get_hi_voices():
    try:
        voices = await edge_tts.list_voices()
        return [v["ShortName"] for v in voices if str(v.get("Locale","")).lower().startswith("hi-")]
    except Exception:
        return []

hi_voices = loop.run_until_complete(get_hi_voices()) or ["hi-IN-MadhurNeural", "hi-IN-SwaraNeural"]
print("Hindi voices:", hi_voices)

# --- 3) Resume accounting (seconds already on disk) ---
secs_cap  = TARGET_HOURS * 3600
secs_done = 0.0
for fn in os.listdir(OUT_DIR):
    if fn.endswith(".wav"):
        try:
            d, sr = sf.read(os.path.join(OUT_DIR, fn))
            secs_done += len(d)/sr
        except: pass
print(f"Already on disk: {secs_done/3600:.2f} h")

# --- 4) Metadata prep ---
need_header = not os.path.isfile(FAKE_CSV)
fields = ["utt_id","path","duration","speaker_id","gender","text","label",
          "fake_type","tts_model","voice","rate","seed"]

# --- 5) OPTIMIZED Async synth helpers ---
async def synth_one(text, voice, sem, pbar):
    async with sem:
        key = f"{text}|{voice}|{VOICE_RATE}|{SEED}"
        uid = "tts_edge_" + hashlib.sha1(key.encode("utf-8")).hexdigest()[:16]
        out_wav = os.path.join(OUT_DIR, f"{uid}.wav")

        # Check if already exists
        if os.path.isfile(out_wav):
            try:
                d, sr = sf.read(out_wav)
                dur = len(d)/sr
                pbar.update(dur)
                # OPTIMIZED: Removed unnecessary sleep
                return uid, out_wav, dur, text, voice
            except:
                pass

        tmp_mp3 = os.path.join(tempfile.gettempdir(), f"{uid}.mp3")
        try:
            await edge_tts.Communicate(text=text, voice=voice, rate=VOICE_RATE).save(tmp_mp3)
        except Exception:
            # OPTIMIZED: Only sleep on actual errors
            await asyncio.sleep(0.2)
            return None

        try:
            # OPTIMIZED: Faster ffmpeg with error suppression
            subprocess.run([
                "ffmpeg", "-loglevel", "error", "-y", "-i", tmp_mp3,
                "-ac", "1", "-ar", "16000", "-f", "wav", out_wav
            ], check=True, capture_output=True)
        except Exception:
            try: os.remove(tmp_mp3)
            except: pass
            return None
        finally:
            try: os.remove(tmp_mp3)
            except: pass

        try:
            data, sr = sf.read(out_wav)
            dur = len(data)/sr
            # OPTIMIZED: Slightly looser validation for speed
            if dur < 0.5 or float(np.abs(data).mean()) < 1e-5:
                os.remove(out_wav)
                return None
        except Exception:
            try: os.remove(out_wav)
            except: pass
            return None

        pbar.update(dur)
        # OPTIMIZED: Removed unnecessary sleep
        return uid, out_wav, dur, text, voice

# --- 6) OPTIMIZED batch processing ---
async def run_batch():
    global secs_done
    sem = asyncio.Semaphore(MAX_CONCURRENT)

    def text_stream(items):
        while True:
            for t in items: yield t
            random.shuffle(items)
    ts = text_stream(texts)

    with open(FAKE_CSV, "a", newline="", encoding="utf-8") as fcsv:
        writer = csv.DictWriter(fcsv, fieldnames=fields)
        if need_header: writer.writeheader()

        with tqdm(total=max(0, secs_cap - secs_done), unit="s", desc="Edge-TTS hours") as pbar:
            while secs_done < secs_cap:
                # OPTIMIZED: Process larger batches
                batch_size = min(BATCH_SIZE, MAX_CONCURRENT * 2)
                tasks = [synth_one(next(ts), random.choice(hi_voices), sem, pbar)
                         for _ in range(batch_size)]

                results = await asyncio.gather(*tasks, return_exceptions=True)

                # Process results
                for res in results:
                    if not res or isinstance(res, Exception):
                        continue
                    uid, out_wav, dur, text, voice = res
                    secs_done += dur
                    writer.writerow({
                        "utt_id": uid, "path": out_wav, "duration": round(dur,3),
                        "speaker_id": "edge_tts", "gender": "u", "text": text,
                        "label": "fake", "fake_type": "tts_edge",
                        "tts_model": "edge-tts", "voice": voice, "rate": VOICE_RATE,
                        "seed": SEED
                    })
                    if secs_done >= secs_cap:
                        break

                # OPTIMIZED: Flush CSV more frequently
                fcsv.flush()

                if secs_done >= secs_cap:
                    break

# ---- OPTIMIZED EXECUTION ----
print(f"\n🚀 Starting OPTIMIZED synthesis with {MAX_CONCURRENT} concurrent workers...")
print(f"Target: {TARGET_HOURS}h | Already done: {secs_done/3600:.2f}h | Remaining: {(secs_cap-secs_done)/3600:.2f}h")

loop.run_until_complete(run_batch())

print(f"\n✅ COMPLETED! Total: {secs_done/3600:.2f}h in {OUT_DIR}")
print(f"📊 Metadata: {FAKE_CSV}")
print(f"⚡ Used {MAX_CONCURRENT} concurrent workers for maximum speed!")

Loaded 10022 unique sentences.
Hindi voices: ['hi-IN-MadhurNeural', 'hi-IN-SwaraNeural']


duration check

In [ ]:
import pandas as pd, soundfile as sf, os
from pathlib import Path

def folder_seconds(dirp):
    s = 0.0
    for fn in os.listdir(dirp):
        if not fn.endswith(".wav"): continue
        try:
            d, sr = sf.read(os.path.join(dirp, fn))
            s += len(d)/float(sr)
        except: pass
    return s

OUT_DIR  = "/content/drive/MyDrive/hindi_dfake/raw/fake_tts"
FAKE_CSV = "/content/drive/MyDrive/hindi_dfake/metadata/master_fake.csv"

disk_h = folder_seconds(OUT_DIR)/3600
csv_h  = 0.0
if Path(FAKE_CSV).exists():
    df = pd.read_csv(FAKE_CSV, usecols=["duration"])
    csv_h = df["duration"].fillna(0).sum()/3600

print(f"On-disk hours: {disk_h:.2f} h")
print(f"CSV duration sum: {csv_h:.2f} h (approx; may differ slightly)")

On-disk hours: 22.79 h
CSV duration sum: 76.42 h (approx; may differ slightly)


normalize fake CSV

In [ ]:
# --- ONE-TIME NORMALIZER for master_fake.csv (fix ParserError) ---
from pathlib import Path
import csv, shutil
import soundfile as sf

FAKE_CSV = "/content/drive/MyDrive/hindi_dfake/metadata/master_fake.csv"
BACKUP   = "/content/drive/MyDrive/hindi_dfake/metadata/master_fake.csv.bak"

FULL_HEADER = [
    "utt_id","path","duration","speaker_id","gender","text","label",
    "fake_type","tts_model","voice","rate","seed"
]

def is_audio_ok(p):
    try:
        d, sr = sf.read(str(p))
        return (sr in (16000, 8000)) and (len(d) > 0)
    except Exception:
        return False

if not Path(FAKE_CSV).exists():
    print("No master_fake.csv found — nothing to normalize.")
else:
    # Backup
    shutil.copyfile(FAKE_CSV, BACKUP)
    print(f"Backup written → {BACKUP}")

    total_lines = 0
    kept_rows   = []
    bad_lines   = 0
    missing     = 0
    bad_audio   = 0

    with open(FAKE_CSV, "r", encoding="utf-8", errors="ignore", newline="") as f:
        rdr = csv.reader(f)
        for row in rdr:
            if not row:
                continue
            total_lines += 1

            # Skip any header-like row encountered anywhere
            if row[0].strip().lower() == "utt_id":
                continue

            L = len(row)

            # We expect either old (8 cols) or full (12 cols)
            # If there are more than 12 because of commas in text (not quoted),
            # re-join the middle as text.
            if L >= 12:
                head5 = row[:5]                # utt_id, path, duration, speaker_id, gender
                tail6 = row[-6:]               # label, fake_type, tts_model, voice, rate, seed
                text  = ",".join(row[5:-6]).strip()
                new = head5 + [text] + tail6
            elif L >= 8:
                head5 = row[:5]
                # old tail is only [label, fake_type]
                label, fake_type = row[-2:]
                text = ",".join(row[5:-2]).strip()
                new = head5 + [text, label, fake_type, "", "", "", ""]
            else:
                bad_lines += 1
                continue

            # Basic sanitation
            new = [c.strip() for c in new]
            if len(new) != 12:
                bad_lines += 1
                continue

            p = Path(new[1])
            if not p.exists():
                missing += 1
                continue
            if not is_audio_ok(p):
                bad_audio += 1
                continue

            kept_rows.append(new)

    # Dedup by utt_id (keep first)
    seen = set()
    dedup_rows = []
    for r in kept_rows:
        uid = r[0]
        if uid in seen:
            continue
        seen.add(uid)
        dedup_rows.append(r)

    with open(FAKE_CSV, "w", encoding="utf-8", newline="") as f:
        w = csv.writer(f)
        w.writerow(FULL_HEADER)
        w.writerows(dedup_rows)

    print(f"Lines read:     {total_lines}")
    print(f"Kept rows:      {len(dedup_rows)} (before dedup: {len(kept_rows)})")
    print(f"Dropped lines:  {bad_lines} (malformed)")
    print(f"Missing files:  {missing}")
    print(f"Bad audio:      {bad_audio}")
    print("✅ master_fake.csv normalized and consistent (12 columns).")

Backup written → /content/drive/MyDrive/hindi_dfake/metadata/master_fake.csv.bak
Lines read:     43884
Kept rows:      18589 (before dedup: 43883)
Dropped lines:  0 (malformed)
Missing files:  0
Bad audio:      0
✅ master_fake.csv normalized and consistent (12 columns).


more robust rerun of Edge TTS

In [ ]:
# =================== INSTANT-START, RESUME-SAFE EDGE-TTS ===================
# Safe to run after stopping the previous cell. Won't touch existing WAVs.

# --- CONFIG (tune these) ---
TARGET_HOURS       = 55
ROOT_DIR           = "/content/drive/MyDrive/hindi_dfake"
REAL_CSV           = f"{ROOT_DIR}/metadata/master_real.csv"
OUT_DIR            = f"{ROOT_DIR}/raw/fake_tts"
FAKE_CSV           = f"{ROOT_DIR}/metadata/master_fake.csv"

SEED               = 2025
MIN_CHARS, MAX_CHARS = 6, 220  # allow slightly longer texts to pack better

# Throughput: start with these; adjust if throttled
MAX_CONCURRENT     = 6
BATCH_SIZE         = MAX_CONCURRENT * 2
VOICE_RATE         = "+0%"

# Speed toggles
USE_DIRECT_PCM     = True   # ask for WAV directly
PACK_N             = 3      # pack 3 sentences per request

# Timeouts (seconds)
TTS_TIMEOUT_SEC    = 40     # per request
FFMPEG_TIMEOUT_SEC = 30

# --- SETUP ---
!pip -q install edge-tts tqdm soundfile pandas numpy nest_asyncio > /dev/null
!apt-get -qq install ffmpeg > /dev/null

import os, csv, hashlib, random, tempfile, subprocess, shlex, time
from pathlib import Path
import pandas as pd, numpy as np, soundfile as sf
from tqdm.auto import tqdm
import asyncio, nest_asyncio, edge_tts
nest_asyncio.apply()
loop = asyncio.get_event_loop()
random.seed(SEED); np.random.seed(SEED)
os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(Path(FAKE_CSV).parent, exist_ok=True)

# --- Voices: hard-coded to avoid any network lookup delays ---
hi_voices = ["hi-IN-MadhurNeural", "hi-IN-SwaraNeural"]
print("Hindi voices:", hi_voices)

# --- Load texts ---
def load_texts(csv_path):
    df = pd.read_csv(csv_path)
    col = next((c for c in ["text","transcript","sentence"] if c in df.columns), None)
    assert col, "master_real.csv must have 'text' (or 'transcript'/'sentence') column."
    texts = (df[col].astype(str).str.replace("\n"," ").str.strip()
             .dropna().loc[lambda s: s.str.len().between(MIN_CHARS, MAX_CHARS)]
             .drop_duplicates().tolist())
    random.shuffle(texts)
    return texts

texts = load_texts(REAL_CSV)
print(f"Loaded {len(texts)} unique sentences.")

# --- Light, on-the-fly packer (no big pre-build) ---
JOINER = " । "
def text_stream(items, n=PACK_N, max_len=300):
    i = random.randrange(len(items))
    while True:
        if n <= 1:
            t = items[i % len(items)].strip(); i += 1
            if len(t) >= MIN_CHARS: yield t
            continue
        # build one bundle
        bundle, cur = [], 0
        while len(bundle) < n:
            t = items[i % len(items)].strip(); i += 1
            if len(t) < MIN_CHARS: continue
            if (cur and cur + 1 + len(t) > max_len): break
            bundle.append(t); cur += (len(t) if not cur else 1 + len(t))
        yield JOINER.join(bundle)

# --- Truth accounting from disk ---
def folder_seconds(wav_dir: str) -> float:
    total = 0.0
    for fn in os.listdir(wav_dir):
        if fn.endswith(".wav"):
            p = os.path.join(wav_dir, fn)
            try:
                d, sr = sf.read(p); total += len(d)/float(sr)
            except: pass
    return total

secs_cap  = TARGET_HOURS * 3600
secs_done = folder_seconds(OUT_DIR)
print(f"Already on disk (truth): {secs_done/3600:.2f} h")

# --- Known UIDs from disk + CSV (no dupes) ---
def uid_for(text, voice, rate, seed):
    return "tts_edge_" + hashlib.sha1(f"{text}|{voice}|{rate}|{seed}".encode("utf-8")).hexdigest()[:16]

existing_uids = set()
for fn in os.listdir(OUT_DIR):
    if fn.endswith(".wav") and fn.startswith("tts_edge_"):
        existing_uids.add(Path(fn).stem)
if os.path.isfile(FAKE_CSV):
    try:
        df_existing = pd.read_csv(FAKE_CSV, usecols=["utt_id"])
        existing_uids |= set(df_existing["utt_id"].astype(str).tolist())
    except Exception:
        pass

need_header = not os.path.isfile(FAKE_CSV)
fields = ["utt_id","path","duration","speaker_id","gender","text","label",
          "fake_type","tts_model","voice","rate","seed"]

# --- Safe wrappers with timeouts ---
async def tts_save_with_timeout(comm: edge_tts.Communicate, out_path: str, timeout=TTS_TIMEOUT_SEC):
    return await asyncio.wait_for(comm.save(out_path), timeout=timeout)

def ffmpeg_convert_with_timeout(cmd_list, timeout=FFMPEG_TIMEOUT_SEC):
    p = subprocess.Popen(cmd_list, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    try:
        outs, errs = p.communicate(timeout=timeout)
    except subprocess.TimeoutExpired:
        p.kill()
        try: p.communicate(timeout=3)
        except: pass
        raise TimeoutError("ffmpeg timed out")
    if p.returncode != 0:
        raise RuntimeError(errs.decode("utf-8","ignore"))
    return outs

# --- One-item synth (resume-safe; PCM-first; timeouts; fallback) ---
async def synth_one(text, voice, sem, pbar):
    async with sem:
        uid = uid_for(text, voice, VOICE_RATE, SEED)
        out_wav = os.path.join(OUT_DIR, f"{uid}.wav")

        if uid in existing_uids and os.path.isfile(out_wav):
            return None

        # Try direct PCM first
        if USE_DIRECT_PCM:
            tmp_wav = os.path.join(tempfile.gettempdir(), f"{uid}.wav")
            try:
                await tts_save_with_timeout(
                    edge_tts.Communicate(
                        text=text, voice=voice, rate=VOICE_RATE,
                        output_format="riff-16khz-16bit-mono-pcm"
                    ), tmp_wav
                )
                os.replace(tmp_wav, out_wav)
            except Exception:
                # fallback MP3 -> WAV
                try:
                    if os.path.exists(tmp_wav): os.remove(tmp_wav)
                except: pass
                try:
                    tmp_mp3 = os.path.join(tempfile.gettempdir(), f"{uid}.mp3")
                    await tts_save_with_timeout(edge_tts.Communicate(text=text, voice=voice, rate=VOICE_RATE), tmp_mp3)
                    ffmpeg_convert_with_timeout([
                        "ffmpeg","-loglevel","error","-y","-i",tmp_mp3,
                        "-ac","1","-ar","16000","-f","wav",out_wav
                    ])
                finally:
                    try: os.remove(tmp_mp3)
                    except: pass
        else:
            tmp_mp3 = os.path.join(tempfile.gettempdir(), f"{uid}.mp3")
            try:
                await tts_save_with_timeout(edge_tts.Communicate(text=text, voice=voice, rate=VOICE_RATE), tmp_mp3)
                ffmpeg_convert_with_timeout([
                    "ffmpeg","-loglevel","error","-y","-i",tmp_mp3,
                    "-ac","1","-ar","16000","-f","wav",out_wav
                ])
            finally:
                try: os.remove(tmp_mp3)
                except: pass

        # Validate and account
        try:
            data, sr = sf.read(out_wav)
            dur = len(data)/float(sr)
            if dur < 0.5 or float(np.abs(data).mean()) < 1e-5:
                os.remove(out_wav); return None
        except Exception:
            try: os.remove(out_wav)
            except: pass
            return None

        pbar.update(dur)
        return uid, out_wav, dur, text, voice

# --- Batch runner ---
async def run_batch():
    global secs_done, existing_uids
    sem = asyncio.Semaphore(MAX_CONCURRENT)
    ts = text_stream(texts, n=PACK_N)

    with open(FAKE_CSV, "a", newline="", encoding="utf-8") as fcsv:
        writer = csv.DictWriter(fcsv, fieldnames=fields)
        if need_header: writer.writeheader()

        with tqdm(total=max(0, secs_cap - secs_done), unit="s", desc="Edge-TTS hours") as pbar:
            while secs_done < secs_cap:
                batch_size = min(BATCH_SIZE, MAX_CONCURRENT * 2)
                choices = [next(ts) for _ in range(batch_size)]
                voices  = [random.choice(hi_voices) for _ in range(batch_size)]
                tasks   = [synth_one(t, v, sem, pbar) for t, v in zip(choices, voices)]

                results = await asyncio.gather(*tasks, return_exceptions=True)
                wrote_any = False

                for res in results:
                    if (not res) or isinstance(res, Exception):
                        continue
                    uid, out_wav, dur, text, voice = res
                    if uid in existing_uids:
                        continue
                    secs_done += dur
                    existing_uids.add(uid)
                    writer.writerow({
                        "utt_id": uid, "path": out_wav, "duration": round(dur,3),
                        "speaker_id": "edge_tts", "gender": "u", "text": text,
                        "label": "fake", "fake_type": "tts_edge",
                        "tts_model": "edge-tts", "voice": voice, "rate": VOICE_RATE,
                        "seed": SEED
                    })
                    wrote_any = True
                    if secs_done >= secs_cap:
                        break

                if wrote_any: fcsv.flush()
                if secs_done >= secs_cap: break

print(f"\n🚀 Starting synthesis with {MAX_CONCURRENT} workers")
print(f"Target: {TARGET_HOURS}h | On-disk start: {secs_done/3600:.2f}h | Remaining: {(secs_cap-secs_done)/3600:.2f}h")
loop.run_until_complete(run_batch())

final_secs = folder_seconds(OUT_DIR)
print(f"\n✅ COMPLETED. On-disk total: {final_secs/3600:.2f} h in {OUT_DIR}")
print(f"📊 Metadata: {FAKE_CSV}")

Hindi voices: ['hi-IN-MadhurNeural', 'hi-IN-SwaraNeural']
Loaded 24713 unique sentences.
Already on disk (truth): 49.16 h

🚀 Starting synthesis with 6 workers
Target: 55h | On-disk start: 49.16h | Remaining: 5.84h


Edge-TTS hours:   0%|          | 0/21033.7919999999 [00:00<?, ?s/s]


✅ COMPLETED. On-disk total: 55.01 h in /content/drive/MyDrive/hindi_dfake/raw/fake_tts
📊 Metadata: /content/drive/MyDrive/hindi_dfake/metadata/master_fake.csv


Syncing disc and csv

In [ ]:
# === STRICT SYNC: master_fake.csv ↔ raw/fake_tts ===
from pathlib import Path
import os, pandas as pd, csv, shutil

ROOT = Path("/content/drive/MyDrive/hindi_dfake")
OUT_DIR  = ROOT / "raw/fake_tts"
FAKE_CSV = ROOT / "metadata/master_fake.csv"
BACKUP   = ROOT / "metadata/master_fake.csv.sync.bak"

ADD_MISSING_FILES_TO_CSV = True  # if True, add rows for WAVs that exist but have no CSV row

assert OUT_DIR.exists(), f"{OUT_DIR} not found"
assert FAKE_CSV.exists(), f"{FAKE_CSV} not found"

# Backup
shutil.copyfile(FAKE_CSV, BACKUP)
print("Backup ->", BACKUP)

df = pd.read_csv(FAKE_CSV)
n0 = len(df)

# Normalize columns
cols = ["utt_id","path","duration","speaker_id","gender","text","label",
        "fake_type","tts_model","voice","rate","seed"]
for c in cols:
    if c not in df.columns:
        df[c] = ""  # ensure presence

# Keep only rows pointing to existing files inside OUT_DIR
df = df[df["path"].astype(str).apply(lambda p: p.endswith(".wav") and Path(p).exists() and Path(p).parent == OUT_DIR)].copy()

# Dedup by path (authoritative) then by utt_id
df = df.drop_duplicates(subset=["path"], keep="first")
df = df.drop_duplicates(subset=["utt_id"], keep="first")

# Scan disk for all WAVs
disk_files = {p.name: p for p in OUT_DIR.glob("*.wav")}
csv_files  = set(Path(p).name for p in df["path"].astype(str))

missing_on_csv = [disk_files[n] for n in disk_files.keys() - csv_files]
print("Rows kept:", len(df), "| WAV on disk:", len(disk_files), "| Missing in CSV:", len(missing_on_csv))

if ADD_MISSING_FILES_TO_CSV and missing_on_csv:
    # Add minimal rows for files that exist but are not indexed (we can't reconstruct original text reliably)
    add_rows = []
    for p in missing_on_csv:
        uid = Path(p).stem
        add_rows.append({
            "utt_id": uid,
            "path": str(p),
            "duration": "",    # unknown quickly; can be filled later if needed
            "speaker_id": "edge_tts",
            "gender": "u",
            "text": "",
            "label": "fake",
            "fake_type": "tts_edge",
            "tts_model": "edge-tts",
            "voice": "",
            "rate": "+0%",
            "seed": ""
        })
    df = pd.concat([df, pd.DataFrame(add_rows)], ignore_index=True)

# Save back
df.to_csv(FAKE_CSV, index=False)
print(f"✅ Synced CSV saved: {FAKE_CSV} | rows={len(df)} (was {n0})")

Backup -> /content/drive/MyDrive/hindi_dfake/metadata/master_fake.csv.sync.bak
Rows kept: 23677 | WAV on disk: 23731 | Missing in CSV: 54
✅ Synced CSV saved: /content/drive/MyDrive/hindi_dfake/metadata/master_fake.csv | rows=23731 (was 23997)


pcm audit

In [ ]:
# === PCM AUDIT (and optional normalize) for TTS ===
from pathlib import Path
import soundfile as sf, numpy as np, subprocess, shlex, os
from collections import Counter
from concurrent.futures import ThreadPoolExecutor, as_completed

ROOT = Path("/content/drive/MyDrive/hindi_dfake")
OUT_DIR = ROOT / "raw/fake_tts"

# If FIX=True:
#   - IN_PLACE=False => write fixed copies to processed/fake_tts_pcm16 (keeps originals)
#   - IN_PLACE=True  => overwrite original files in-place (use with care)
FIX = False
IN_PLACE = False

DEST_DIR = ROOT / "processed/fake_tts_pcm16"
if FIX and not IN_PLACE:
    DEST_DIR.mkdir(parents=True, exist_ok=True)

def probe(p: Path):
    try:
        info = sf.info(str(p))
        return (p, info.samplerate, info.channels, info.subtype)
    except Exception:
        return (p, None, None, None)

wav_paths = list(OUT_DIR.glob("*.wav"))
print("Scanning", len(wav_paths), "files...")

sr_ct = Counter(); ch_ct = Counter(); sub_ct = Counter()
bad = []

with ThreadPoolExecutor(max_workers=8) as ex:
    futs = [ex.submit(probe, p) for p in wav_paths]
    for f in as_completed(futs):
        p, sr, ch, sub = f.result()
        if sr is None:
            bad.append(p)
            continue
        sr_ct[sr] += 1
        ch_ct[ch] += 1
        sub_ct[sub] += 1
        ok = (sr == 16000 and ch == 1 and sub == "PCM_16")
        if not ok:
            bad.append((p, sr, ch, sub))

print("\n=== AUDIT SUMMARY ===")
print("Sample rates:", dict(sr_ct))
print("Channels:    ", dict(ch_ct))
print("Subtypes:    ", dict(sub_ct))
print("Non-uniform (not 16k/mono/PCM_16):", len(bad))

if not FIX or len(bad) == 0:
    print("\n(No fixes applied.)")
else:
    print("\nNormalizing to 16k/mono/PCM_16 ...")
    def fix_one(item):
        p, sr, ch, sub = item
        dst = (p if IN_PLACE else DEST_DIR / p.name)
        cmd = [
            "ffmpeg","-loglevel","error","-y","-i",str(p),
            "-ac","1","-ar","16000","-sample_fmt","s16", str(dst)
        ]
        try:
            subprocess.run(cmd, check=True)
            return (p, dst, True)
        except Exception as e:
            return (p, dst, False)

    ok_count = 0
    with ThreadPoolExecutor(max_workers=4) as ex:
        futs = [ex.submit(fix_one, itm) for itm in bad if isinstance(itm, tuple)]
        for f in as_completed(futs):
            src, dst, ok = f.result()
            ok_count += int(ok)
    print(f"Fixed {ok_count}/{len([b for b in bad if isinstance(b, tuple)])} files")
    if not IN_PLACE:
        print("Fixed copies at:", DEST_DIR)

Scanning 23731 files...

=== AUDIT SUMMARY ===
Sample rates: {16000: 23731}
Channels:     {1: 23731}
Subtypes:     {'PCM_16': 23731}
Non-uniform (not 16k/mono/PCM_16): 0

(No fixes applied.)


Edge TTs ping test

In [ ]:
# Version-agnostic Edge-TTS ping: MP3 -> ffmpeg -> WAV
!pip -q install edge-tts soundfile nest_asyncio > /dev/null
!apt-get -qq install ffmpeg > /dev/null

import nest_asyncio, asyncio, edge_tts, tempfile, os, soundfile as sf, subprocess
nest_asyncio.apply()
loop = asyncio.get_event_loop()

async def edge_ping_any():
    tmp_dir = tempfile.gettempdir()
    tmp_mp3 = os.path.join(tmp_dir, "edge_ping.mp3")
    tmp_wav = os.path.join(tmp_dir, "edge_ping.wav")
    try:
        comm = edge_tts.Communicate(text="नमस्ते", voice="hi-IN-MadhurNeural")
        # older edge-tts: no format kw; save MP3
        await asyncio.wait_for(comm.save(tmp_mp3), timeout=30)
        # convert to 16k, mono, PCM_16
        subprocess.run([
            "ffmpeg","-loglevel","error","-y","-i",tmp_mp3,
            "-ac","1","-ar","16000","-sample_fmt","s16", tmp_wav
        ], check=True)
        print("EDGE_PING_OK:", sf.info(tmp_wav))
    except Exception as e:
        print("EDGE_PING_FAIL:", repr(e))
    finally:
        for p in (tmp_mp3, tmp_wav):
            try: os.remove(p)
            except: pass

loop.run_until_complete(edge_ping_any())

EDGE_PING_OK: /tmp/edge_ping.wav
samplerate: 16000 Hz
channels: 1
duration: 1.608 s
format: WAV (Microsoft) [WAV]
subtype: Signed 16 bit PCM [PCM_16]


real vs fake comparision

In [ ]:
# ===== One-Pair Compare (v4): version-agnostic Edge-TTS (MP3->WAV) =====
# Writes: /content/drive/MyDrive/hindi_dfake/processed/compare/<utt_id>/{real.wav, fake_edge.wav}

# --- CONFIG ---
ROOT = "/content/drive/MyDrive/hindi_dfake"
REAL_CSV = f"{ROOT}/metadata/master_real.csv"
COMPARE_DIR = f"{ROOT}/processed/compare"
VOICE_ORDER = ["hi-IN-MadhurNeural", "hi-IN-SwaraNeural"]  # try in this order
VOICE_RATE = "+0%"
TARGET_UTT_ID = None      # set to a specific utt_id or keep None
CHUNK_MAX = 200
REQ_TIMEOUT = 45
RETRIES = 3

# --- SETUP ---
!pip -q install edge-tts soundfile pandas numpy nest_asyncio > /dev/null
!apt-get -qq install ffmpeg > /dev/null

import os, shutil, random, hashlib, tempfile, time, re, subprocess
from pathlib import Path
import soundfile as sf
import pandas as pd, numpy as np
import nest_asyncio, asyncio, edge_tts
from IPython.display import Audio, display

nest_asyncio.apply()
loop = asyncio.get_event_loop()

def clean_text(s: str) -> str:
    s = (s or "").replace("\n", " ").strip()
    s = re.sub(r"\s+", " ", s)
    s = s.replace("’","'").replace("“","\"").replace("”","\"")
    return s

def split_hi(text: str, max_len=CHUNK_MAX):
    parts = re.split(r"([।.!?])", text)
    sents = []
    for i in range(0, len(parts), 2):
        sent = parts[i]
        punc = parts[i+1] if i+1 < len(parts) else ""
        piece = (sent + punc).strip()
        if piece:
            sents.append(piece)
    chunks, buf, n = [], [], 0
    for s in sents:
        if not buf:
            buf, n = [s], len(s); continue
        if n + 1 + len(s) <= max_len:
            buf.append(s); n += 1 + len(s)
        else:
            chunks.append(" ".join(buf)); buf, n = [s], len(s)
    if buf: chunks.append(" ".join(buf))
    if not chunks and len(text) > max_len:
        for i in range(0, len(text), max_len):
            chunks.append(text[i:i+max_len])
    return [c for c in chunks if len(c) >= 4]

def ensure_pcm16_16k(src_path, dst_path):
    # normalize/copy real without altering the source
    try:
        d, sr = sf.read(src_path, always_2d=False)
        if getattr(d, "ndim", 1) == 2: d = d.mean(axis=1)
        sf.write(dst_path, d, 16000, subtype="PCM_16")
    except Exception:
        shutil.copyfile(src_path, dst_path)

async def save_chunk_any_version(txt, voice, out_wav, rate=VOICE_RATE, timeout=REQ_TIMEOUT):
    """
    Save a chunk as WAV(16k, mono, PCM_16) regardless of edge-tts version:
    1) edge_tts -> MP3
    2) ffmpeg -> WAV
    """
    tmp_mp3 = os.path.join(tempfile.gettempdir(), f"cmp_{hashlib.sha1((txt+voice).encode()).hexdigest()[:10]}.mp3")
    try:
        comm = edge_tts.Communicate(text=txt, voice=voice, rate=rate)
        await asyncio.wait_for(comm.save(tmp_mp3), timeout=timeout)  # old API: no format kw
        subprocess.run([
            "ffmpeg","-loglevel","error","-y","-i",tmp_mp3,
            "-ac","1","-ar","16000","-sample_fmt","s16", out_wav
        ], check=True)
    finally:
        try: os.remove(tmp_mp3)
        except: pass

async def synth_concat_edge(text, wav_out):
    text = clean_text(text)
    chunks = split_hi(text, CHUNK_MAX) or [text]
    segs = []
    for idx, chunk in enumerate(chunks, 1):
        success = False
        for v in VOICE_ORDER:
            for attempt in range(1, RETRIES+1):
                tmp_wav = os.path.join(tempfile.gettempdir(), f"cmp_{hashlib.sha1((chunk+v).encode()).hexdigest()[:10]}.wav")
                try:
                    await save_chunk_any_version(chunk, v, tmp_wav)
                    d, sr = sf.read(tmp_wav, always_2d=False)
                    if getattr(d, "ndim", 1) == 2: d = d.mean(axis=1)
                    segs.append(d.astype(np.float32))
                    os.remove(tmp_wav)
                    success = True
                    break
                except Exception:
                    try: os.remove(tmp_wav)
                    except: pass
                    await asyncio.sleep(min(2*attempt, 6))
            if success: break
        if not success:
            raise RuntimeError(f"Edge-TTS failed for chunk {idx}/{len(chunks)}")
    full = np.concatenate(segs, axis=0) if len(segs) > 1 else segs[0]
    sf.write(wav_out, full, 16000, subtype="PCM_16")

# pick a row
df = pd.read_csv(REAL_CSV)
text_col = next((c for c in ["text","transcript","sentence"] if c in df.columns), None)
assert text_col, "master_real.csv needs a text column."

def pick_row():
    if TARGET_UTT_ID:
        return df[df["utt_id"]==TARGET_UTT_ID].iloc[0]
    cand = df[df["path"].astype(str).apply(lambda p: Path(p).exists())]
    cand = cand[cand[text_col].astype(str).str.len().between(6, 400)]
    return cand.sample(1, random_state=777).iloc[0]

row = pick_row()
utt_id = str(row["utt_id"])
text   = clean_text(str(row[text_col]))
real_p = str(row["path"])

pair_dir = Path(COMPARE_DIR) / utt_id
pair_dir.mkdir(parents=True, exist_ok=True)
real_out = pair_dir / "real.wav"
fake_out = pair_dir / "fake_edge.wav"

if not real_out.exists():
    ensure_pcm16_16k(real_p, real_out)

made = None
if not fake_out.exists():
    try:
        loop.run_until_complete(synth_concat_edge(text, str(fake_out)))
        made = "edge"
    except Exception as e:
        print("Edge synth failed:", e)

# report & audio
from IPython.display import Audio, display
print(f"UTT: {utt_id}")
print("Text:", (text[:220] + ("..." if len(text) > 220 else "")))
print("Real:", sf.info(str(real_out)), "|", real_out)
if fake_out.exists():
    print("Fake (edge):", sf.info(str(fake_out)), "|", fake_out)
    display(Audio(filename=str(fake_out)))
else:
    print("Fake: (synthesis failed)")
display(Audio(filename=str(real_out)))

UTT: cb0999e7159064f1
Text: उस दुर्घटना में कई लोगों की मौत हो गयी।
Real: /content/drive/MyDrive/hindi_dfake/processed/compare/cb0999e7159064f1/real.wav
samplerate: 16000 Hz
channels: 1
duration: 3.744 s
format: WAV (Microsoft) [WAV]
subtype: Signed 16 bit PCM [PCM_16] | /content/drive/MyDrive/hindi_dfake/processed/compare/cb0999e7159064f1/real.wav
Fake (edge): /content/drive/MyDrive/hindi_dfake/processed/compare/cb0999e7159064f1/fake_edge.wav
samplerate: 16000 Hz
channels: 1
duration: 3.888 s
format: WAV (Microsoft) [WAV]
subtype: Signed 16 bit PCM [PCM_16] | /content/drive/MyDrive/hindi_dfake/processed/compare/cb0999e7159064f1/fake_edge.wav


sanity checks before attacks

In [ ]:
# ===== Codec/Channel Attacks — DEBUG Selection (fast + verbose) =====
# Real: raw/real_clean/{commonvoice, indictts, ivr}
# Fake: raw/fake_tts

# Paths
ROOT = "/content/drive/MyDrive/hindi_dfake"
META = f"{ROOT}/metadata"
REAL_CSV   = f"{META}/master_real.csv"
FAKE_CSV   = f"{META}/master_fake.csv"
OUT_DIR    = f"{ROOT}/processed/attacks"   # next cells will write here

# Include both classes
INCLUDE_REAL = True
INCLUDE_FAKE = True

# Optional sampling to control runtime/storage
REAL_FRACTION = 1.0   # try 0.10 first if you want to smoke-test
FAKE_FRACTION = 1.0

# Attack list is defined in the next cells; we just select rows here.

# --- DEBUG switches ---
PROBE_AUDIO_HEADERS = False   # set True to verify a small sample with sf.info
PROBE_LIMIT_PER_SRC = 100     # header-probe at most this many files per source

# --- deps ---
!pip -q install pandas soundfile numpy > /dev/null

import os, time, pandas as pd, numpy as np, soundfile as sf
from pathlib import Path
from collections import Counter

start = time.time()
def log(msg):
    print(f"[+{time.time()-start:6.1f}s] {msg}", flush=True)

Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

# 0) File presence
for p in [REAL_CSV, FAKE_CSV]:
    if os.path.exists(p):
        log(f"Found {p} ({os.path.getsize(p)/1e6:.2f} MB)")
    else:
        log(f"Missing: {p}")

# 1) Load minimal columns quickly
USECOLS = ["utt_id","path","duration","speaker_id","gender","text","label"]
log("Reading CSVs...")
dfr = pd.read_csv(REAL_CSV, usecols=[c for c in USECOLS if c != "label"], low_memory=False) if INCLUDE_REAL and os.path.exists(REAL_CSV) else pd.DataFrame(columns=USECOLS)
dff = pd.read_csv(FAKE_CSV, usecols=USECOLS, low_memory=False) if INCLUDE_FAKE and os.path.exists(FAKE_CSV) else pd.DataFrame(columns=USECOLS)
if "label" not in dfr.columns and len(dfr):
    dfr["label"] = "real"
log(f"Loaded rows — real: {len(dfr):,} | fake: {len(dff):,}")

# 2) Filter by folder (no regex groups; no warnings)
def keep_real_folders(df):
    p = df["path"].astype(str)
    mask = (
        p.str.contains("/raw/real_clean/commonvoice", regex=False, na=False) |
        p.str.contains("/raw/real_clean/indictts",   regex=False, na=False) |
        p.str.contains("/raw/real_clean/ivr",        regex=False, na=False)
    )
    return df[mask].copy()

def keep_fake_folder(df):
    p = df["path"].astype(str)
    mask = p.str.contains("/raw/fake_tts", regex=False, na=False)
    return df[mask].copy()

log("Filtering rows to target folders...")
dfr = keep_real_folders(dfr) if len(dfr) else dfr
dff = keep_fake_folder(dff)  if len(dff) else dff
log(f"After folder filter — real: {len(dfr):,} | fake: {len(dff):,}")

# 3) Fast existence check (no audio decode)
def fast_exists(paths):
    out = np.zeros(len(paths), dtype=bool)
    for i, p in enumerate(paths):
        out[i] = Path(p).exists()
        if (i+1) % 10000 == 0:
            log(f"exists() checked: {i+1}/{len(paths)}")
    return out

if len(dfr):
    log("exists() on real...")
    m = fast_exists(dfr["path"].astype(str).tolist())
    dfr = dfr[m]
    log(f"Real kept after exists(): {len(dfr):,}")
if len(dff):
    log("exists() on fake...")
    m = fast_exists(dff["path"].astype(str).tolist())
    dff = dff[m]
    log(f"Fake kept after exists(): {len(dff):,}")

# 4) Optional: header-only probe (sampled) — NOT full decode
if PROBE_AUDIO_HEADERS:
    log("Header-probing a small sample with soundfile.info()...")
    def probe_sample(df, name):
        samp = df.sample(min(len(df), PROBE_LIMIT_PER_SRC), random_state=2025) if len(df) else df
        sr_ct = Counter(); ch_ct = Counter()
        bad = 0
        for p in samp["path"].astype(str):
            try:
                info = sf.info(p)
                sr_ct[info.samplerate] += 1
                ch_ct[info.channels]   += 1
            except Exception:
                bad += 1
        log(f"{name}: sample={len(samp)} | sr={dict(sr_ct)} | ch={dict(ch_ct)} | bad={bad}")
    probe_sample(dfr, "real")
    probe_sample(dff, "fake")

# 5) Optional sampling to control runtime/storage
if 0 < REAL_FRACTION < 1.0 and len(dfr):
    before = len(dfr); dfr = dfr.sample(frac=REAL_FRACTION, random_state=2025); log(f"Sampled real: {before:,} -> {len(dfr):,}")
if 0 < FAKE_FRACTION < 1.0 and len(dff):
    before = len(dff); dff = dff.sample(frac=FAKE_FRACTION, random_state=2025); log(f"Sampled fake: {before:,} -> {len(dff):,}")

# 6) Tag sources for reporting
def src_from_path(p: str) -> str:
    if "/raw/real_clean/commonvoice" in p: return "real_commonvoice"
    if "/raw/real_clean/indictts"   in p: return "real_indictts"
    if "/raw/real_clean/ivr"        in p: return "real_ivr"
    if "/raw/fake_tts"              in p: return "fake_tts"
    return "other"

if len(dfr): dfr["source"] = dfr["path"].astype(str).map(src_from_path)
if len(dff): dff["source"] = dff["path"].astype(str).map(src_from_path)

# 7) Combine and show a tiny preview
df_all = pd.concat([dfr, dff], ignore_index=True)
log(f"Selected inputs total: {len(df_all):,}")
if len(df_all):
    log("Source breakdown:\n" + df_all["source"].value_counts().to_string())
    log("Preview paths:\n" + "\n".join(df_all["path"].astype(str).head(5).tolist()))
else:
    log("No inputs selected — check your paths in master_* CSVs.")

[+   1.0s] Found /content/drive/MyDrive/hindi_dfake/metadata/master_real.csv (10.29 MB)
[+   1.0s] Found /content/drive/MyDrive/hindi_dfake/metadata/master_fake.csv (9.65 MB)
[+   1.0s] Reading CSVs...
[+   2.5s] Loaded rows — real: 25,965 | fake: 23,731
[+   2.5s] Filtering rows to target folders...
[+   2.5s] After folder filter — real: 25,965 | fake: 23,731
[+   2.5s] exists() on real...
[+  40.9s] exists() checked: 10000/25965
[+  43.8s] exists() checked: 20000/25965
[+  45.7s] Real kept after exists(): 25,965
[+  45.7s] exists() on fake...
[+ 123.6s] exists() checked: 10000/23731
[+ 126.4s] exists() checked: 20000/23731
[+ 127.4s] Fake kept after exists(): 23,731
[+ 127.5s] Selected inputs total: 49,696
[+ 127.5s] Source breakdown:
source
fake_tts            23731
real_ivr            11174
real_commonvoice    10088
real_indictts        4703
[+ 127.5s] Preview paths:
/content/drive/MyDrive/hindi_dfake/raw/real_clean/ivr/1a0b61275bc58baf.wav
/content/drive/MyDrive/hindi_dfake/raw/re

Attacks Runner

In [ ]:
# ===== Codec/Channel Attacks — subset + resumable + no-dup + progress =====
import os, time, csv, hashlib, subprocess, tempfile
from pathlib import Path
import pandas as pd, numpy as np, soundfile as sf
from concurrent.futures import ThreadPoolExecutor, as_completed

ROOT = "/content/drive/MyDrive/hindi_dfake"
META = Path(ROOT) / "metadata"
OUT_DIR = Path(ROOT) / "processed" / "attacks"
ATTACK_CSV = META / "attacks.csv"

# --- attacks (you can add more later) ---
ATTACKS = {
    "opus16": {"codec": "libopus",           "bitrate": "16k",  "container": "ogg"},
    "lp8k":   {"filter": "lowpass=f=8000"},
    # "opus24": {"codec": "libopus",           "bitrate": "24k",  "container": "ogg"},
    # "opus32": {"codec": "libopus",           "bitrate": "32k",  "container": "ogg"},
    # "amrnb":  {"codec": "libopencore_amrnb", "bitrate": "12.2k","ar": 8000, "container": "amr"},
}
ENABLED_ATTACKS = list(ATTACKS.keys())

# --- subset knobs (40% like you asked) ---
FRACTION_REAL = 0.40
FRACTION_FAKE = 0.40
HASH_SEED     = 2025    # stable selection across runs (no randomness drift)

MAX_WORKERS = 6
BATCH_WRITE = 1000      # write metadata every N successes
BACKFILL_METADATA = True  # if output file exists but not in CSV, add a row for it

assert 'df_all' in globals() and len(df_all) > 0, "Run the selection DEBUG cell first; it defines df_all."

# ---------- helpers ----------
def ffmpeg_supports(codec_name: str) -> bool:
    try:
        out = subprocess.run(["ffmpeg","-hide_banner","-codecs"],
                             stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
                             text=True, check=True).stdout
        return codec_name in out
    except Exception:
        return False

def run_ffmpeg(cmd_list):
    p = subprocess.run(cmd_list, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if p.returncode != 0:
        raise RuntimeError(p.stderr.decode("utf-8","ignore"))

def out_path_for(in_path: str, attack_key: str) -> Path:
    return OUT_DIR / attack_key / f"{Path(in_path).stem}.{attack_key}.wav"

def ensure_parent(p: Path): p.parent.mkdir(parents=True, exist_ok=True)

def wav_duration(p: Path) -> float:
    try:
        d, sr = sf.read(str(p), dtype="float32", always_2d=False)
        return len(d)/float(sr)
    except: return 0.0

def opus_roundtrip(in_wav: str, out_wav: str, bitrate: str):
    with tempfile.TemporaryDirectory() as td:
        ogg = str(Path(td)/"tmp.ogg")
        run_ffmpeg(["ffmpeg","-loglevel","error","-y","-i",in_wav,
                    "-ac","1","-ar","16000","-c:a","libopus","-b:a",bitrate, ogg])
        run_ffmpeg(["ffmpeg","-loglevel","error","-y","-i",ogg,
                    "-ac","1","-ar","16000","-sample_fmt","s16", out_wav])

def amrnb_roundtrip(in_wav: str, out_wav: str, bitrate: str, ar=8000):
    with tempfile.TemporaryDirectory() as td:
        amr = str(Path(td)/"tmp.amr")
        run_ffmpeg(["ffmpeg","-loglevel","error","-y","-i",in_wav,
                    "-ac","1","-ar",str(ar),"-c:a","libopencore_amrnb","-b:a",bitrate, amr])
        run_ffmpeg(["ffmpeg","-loglevel","error","-y","-i",amr,
                    "-ac","1","-ar","16000","-sample_fmt","s16", out_wav])

def filter_only(in_wav: str, out_wav: str, af: str):
    run_ffmpeg(["ffmpeg","-loglevel","error","-y","-i",in_wav,
                "-ac","1","-ar","16000","-af",af,"-sample_fmt","s16", out_wav])

def stable_frac_mask(series: pd.Series, frac: float, seed: int) -> pd.Series:
    """Deterministic subset by hashing path; same result every run."""
    if frac >= 1.0: return pd.Series([True]*len(series), index=series.index)
    # make 0..1 score from sha1(path + seed)
    s = series.astype(str).apply(lambda p: int(hashlib.sha1((p+str(seed)).encode()).hexdigest()[:8], 16) / 0xFFFFFFFF)
    return s < frac

# ---------- build subset (stratified by label & source, deterministic) ----------
def stratified_subset_hash(df):
    parts = []
    for (lab, src), g in df.groupby(["label","source"], dropna=False):
        frac = FRACTION_REAL if lab == "real" else FRACTION_FAKE
        if frac >= 1.0 or len(g) <= 1:
            parts.append(g)
        else:
            mask = stable_frac_mask(g["path"], frac, HASH_SEED)
            parts.append(g[mask])
    return pd.concat(parts, ignore_index=True) if parts else df

df_input = stratified_subset_hash(df_all.copy())
print("Subset sizes by label/source:")
print(df_input.groupby(["label","source"]).size().to_string())

# ---------- skip work we already did ----------
# paths already listed in attacks.csv
existing_paths_csv = set()
if ATTACK_CSV.exists():
    try:
        existing_paths_csv = set(pd.read_csv(ATTACK_CSV, usecols=["path"])["path"].astype(str))
    except Exception:
        existing_paths_csv = set()

# filter attacks by codec availability
enabled = []
for k in ENABLED_ATTACKS:
    spec = ATTACKS[k]
    if "codec" in spec and not ffmpeg_supports(spec["codec"]):
        print(f"[WARN] Disabling {k}: ffmpeg missing {spec['codec']}")
        continue
    enabled.append(k)
ENABLED_ATTACKS = enabled
print("Attacks enabled:", ENABLED_ATTACKS)

# ensure metadata exists with header for incremental appends
ATTACK_CSV.parent.mkdir(parents=True, exist_ok=True)
if not ATTACK_CSV.exists():
    with open(ATTACK_CSV, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(["parent_utt","path","duration","speaker_id","gender","text","label",
                    "fake_type","attack","codec","bitrate","source"])

def append_rows(rows):
    if not rows: return
    with open(ATTACK_CSV, "a", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=["parent_utt","path","duration","speaker_id","gender","text","label",
                                          "fake_type","attack","codec","bitrate","source"])
        for r in rows:
            w.writerow(r)

# ---------- run per attack with explicit GOAL/DONE/REMAINING ----------
for atk in ENABLED_ATTACKS:
    spec = ATTACKS[atk]

    # planned outputs for our subset
    planned = df_input["path"].astype(str).apply(lambda p: str(out_path_for(p, atk)))
    planned_set = set(planned)

    # what's already on disk (for this attack)
    out_dir = OUT_DIR / atk
    out_dir.mkdir(parents=True, exist_ok=True)
    on_disk = set(str(p) for p in out_dir.glob("*.wav"))

    # already done = on disk (we treat as done even if CSV missed a row; we'll backfill)
    already_done = planned_set & on_disk
    # remaining = planned - on_disk
    remaining_set = planned_set - on_disk

    goal = len(planned_set)
    done = len(already_done)
    todo = len(remaining_set)
    print(f"\n[{atk}] GOAL={goal:,} | ALREADY_DONE(on-disk)={done:,} | TO_DO={todo:,}")

    # (optional) backfill CSV rows for outputs that exist but are missing from CSV
    if BACKFILL_METADATA:
        missing_meta = [p for p in already_done if p not in existing_paths_csv]
        if missing_meta:
            print(f"[{atk}] Backfilling metadata for {len(missing_meta):,} existing files...")
            rows = []
            # build a map from input path -> parent row for quick lookup
            in2row = {str(r["path"]): r for _, r in df_input.iterrows()}
            for out_p in missing_meta:
                # reconstruct parent input path from filename stem (before .atk.wav) by searching
                stem = Path(out_p).name.split(f".{atk}.wav")[0]
                # find any input whose stem matches (fast check)
                # this is approximate but works because we preserve input stem in output name
                cand = [r for r in in2row.values() if Path(str(r["path"])).stem == stem]
                base = cand[0] if cand else None
                dur = wav_duration(Path(out_p))
                rows.append({
                    "parent_utt": "" if base is None else base.get("utt_id",""),
                    "path": out_p,
                    "duration": round(dur,3),
                    "speaker_id": "" if base is None else base.get("speaker_id",""),
                    "gender": "" if base is None else base.get("gender",""),
                    "text": "" if base is None else base.get("text",""),
                    "label": "" if base is None else base.get("label",""),
                    "fake_type": "channel_attack",
                    "attack": atk,
                    "codec": spec.get("codec", spec.get("filter","")),
                    "bitrate": spec.get("bitrate",""),
                    "source": "" if base is None else base.get("source",""),
                })
            append_rows(rows)
            existing_paths_csv.update(missing_meta)
            print(f"[{atk}] Backfill appended.")

    if todo == 0:
        print(f"[{atk}] Nothing to do.")
        continue

    # build rows to process (Series aligned with df_input)
    to_do_mask = planned.isin(list(remaining_set))
    df_todo = df_input[to_do_mask].copy()
    n = len(df_todo)
    print(f"[{atk}] Will process {n:,} new files…")

    created = 0; errs = 0
    buffer_rows = []

    def apply_one(row):
        in_wav = str(row["path"])
        out_wav = str(out_path_for(in_wav, atk))
        outp = Path(out_wav)
        ensure_parent(outp)
        try:
            if spec.get("codec") == "libopus":
                opus_roundtrip(in_wav, out_wav, spec["bitrate"])
            elif spec.get("codec") == "libopencore_amrnb":
                amrnb_roundtrip(in_wav, out_wav, spec["bitrate"], spec.get("ar", 8000))
            elif "filter" in spec:
                filter_only(in_wav, out_wav, spec["filter"])
            else:
                raise ValueError(f"Unknown attack spec: {spec}")
        except Exception as e:
            try:
                if outp.exists(): outp.unlink()
            except: pass
            return {"error": str(e), "path": in_wav}

        dur = wav_duration(outp)
        if dur < 0.2:
            try: outp.unlink()
            except: pass
            return {"error": "too_short", "path": in_wav}

        return {
            "parent_utt": row.get("utt_id",""),
            "path": out_wav,
            "duration": round(dur, 3),
            "speaker_id": row.get("speaker_id",""),
            "gender": row.get("gender",""),
            "text": row.get("text",""),
            "label": row.get("label",""),
            "fake_type": "channel_attack",
            "attack": atk,
            "codec": spec.get("codec", spec.get("filter","")),
            "bitrate": spec.get("bitrate",""),
            "source": row.get("source",""),
        }

    t0 = time.time()
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
        futs = [ex.submit(apply_one, r) for _, r in df_todo.iterrows()]
        done_ct = 0
        for f in as_completed(futs):
            done_ct += 1
            res = f.result()
            if isinstance(res, dict) and "error" in res:
                errs += 1
            else:
                buffer_rows.append(res)
                created += 1

            # periodic progress & metadata flush
            if done_ct % 1000 == 0 or (done_ct == n):
                print(f"[{atk}] {done_ct:,}/{n:,}  (+{created} ok, {errs} err)  | elapsed {time.time()-t0:.1f}s", flush=True)

            if len(buffer_rows) >= BATCH_WRITE or (done_ct == n and buffer_rows):
                append_rows(buffer_rows)
                existing_paths_csv.update([r["path"] for r in buffer_rows])
                buffer_rows.clear()

    print(f"[{atk}] finished: +{created} new, {errs} errors | total elapsed {time.time()-t0:.1f}s")

# quick per-attack summary (files/hours on disk)
def folder_hours(d: Path):
    s = 0.0
    for p in d.glob("*.wav"):
        try:
            x, sr = sf.read(str(p)); s += len(x)/float(sr)
        except: pass
    return s/3600

tot_files = 0; tot_hours = 0.0
for atk in ENABLED_ATTACKS:
    d = OUT_DIR / atk
    if not d.exists(): continue
    n = len(list(d.glob("*.wav")))
    h = folder_hours(d)
    tot_files += n; tot_hours += h
    print(f"[sum] {atk:6s} → files: {n:7d} | hours: {h:7.2f} h")
print(f"[sum] TOTAL  → files: {tot_files:7d} | hours: {tot_hours:7.2f} h")

Subset sizes by label/source:
label     source          
fake      fake_tts            8667
real      real_commonvoice    4022
          real_indictts       1833
          real_ivr            4481
tts_edge  fake_tts             900
Attacks enabled: ['opus16', 'lp8k']

[opus16] GOAL=19,903 | ALREADY_DONE(on-disk)=19,903 | TO_DO=0
[opus16] Nothing to do.

[lp8k] GOAL=19,903 | ALREADY_DONE(on-disk)=13,158 | TO_DO=6,745
[lp8k] Backfilling metadata for 158 existing files...
[lp8k] Backfill appended.
[lp8k] Will process 6,745 new files…
[lp8k] 1,000/6,745  (+1000 ok, 0 err)  | elapsed 146.9s
[lp8k] 2,000/6,745  (+2000 ok, 0 err)  | elapsed 293.6s
[lp8k] 3,000/6,745  (+3000 ok, 0 err)  | elapsed 440.5s
[lp8k] 4,000/6,745  (+4000 ok, 0 err)  | elapsed 587.1s
[lp8k] 5,000/6,745  (+5000 ok, 0 err)  | elapsed 734.2s
[lp8k] 6,000/6,745  (+6000 ok, 0 err)  | elapsed 882.8s
[lp8k] 6,745/6,745  (+6745 ok, 0 err)  | elapsed 996.6s
[lp8k] finished: +6745 new, 0 errors | total elapsed 996.6s
[sum] opus1

In [ ]:
def src_from_path(p: str) -> str:
    p = str(p)
    if "/raw/real_clean/commonvoice" in p: return "real_commonvoice"
    if "/raw/real_clean/indictts"   in p: return "real_indictts"
    if "/raw/real_clean/ivr"        in p: return "real_ivr"
    if "/raw/fake_tts"              in p: return "fake_tts"
    if "/raw/fake_vc"               in p: return "fake_vc"     # <— add this later
    return "other"

Attacks summary

In [ ]:
# === PATCHED: attacks.clean.csv consistency + summaries ===
from pathlib import Path
import pandas as pd, numpy as np

ROOT = "/content/drive/MyDrive/hindi_dfake"
META = Path(ROOT)/"metadata"
CLEAN_CSV = META/"attacks.clean.csv"   # from the previous cell
assert CLEAN_CSV.exists(), f"Missing: {CLEAN_CSV}"

df = pd.read_csv(CLEAN_CSV)

# Make sure the expected columns exist
expected = ["parent_utt","path","duration","speaker_id","gender","text","label",
            "fake_type","attack","codec","bitrate","source"]
for c in expected:
    if c not in df.columns:
        df[c] = "" if c != "duration" else 0.0
df = df[expected]

# Coerce duration to numeric seconds
df["duration"] = pd.to_numeric(df["duration"], errors="coerce").fillna(0.0)

# Helper that works with Series (what groupby.agg passes)
def hours_series(s: pd.Series) -> float:
    return float(pd.to_numeric(s, errors="coerce").fillna(0.0).sum() / 3600.0)

print(f"Loaded {len(df):,} rows from {CLEAN_CSV.name}")
print("\nBy attack:")
print(
    df.groupby("attack", dropna=False)
      .agg(files=("path","count"), hours=("duration", hours_series))
      .reset_index()
      .sort_values("files", ascending=False)
      .to_string(index=False)
)

print("\nBy label (blank = unknown from backfill):")
print(
    df.groupby("label", dropna=False)
      .agg(files=("path","count"), hours=("duration", hours_series))
      .reset_index()
      .sort_values("files", ascending=False)
      .to_string(index=False)
)

if "source" in df.columns:
    print("\nTop (source × attack):")
    print(
        df.groupby(["source","attack"], dropna=False)
          .agg(files=("path","count"), hours=("duration", hours_series))
          .reset_index()
          .sort_values("files", ascending=False)
          .head(12)
          .to_string(index=False)
    )

Loaded 45,093 rows from attacks.clean.csv

By attack:
attack  files     hours
opus16  25190 56.205517
  lp8k  19903 43.541544

By label (blank = unknown from backfill):
   label  files     hours
    real  20672 42.327501
    fake  17334 36.224440
     NaN   5287 12.663974
tts_edge   1800  8.531147

Top (source × attack):
          source attack  files     hours
        fake_tts   lp8k   9567 22.377793
        fake_tts opus16   9567 22.377793
             NaN opus16   5287 12.663974
        real_ivr   lp8k   4481 11.982329
        real_ivr opus16   4481 11.982329
real_commonvoice   lp8k   4022  5.027940
real_commonvoice opus16   4022  5.027940
   real_indictts   lp8k   1833  4.153481
   real_indictts opus16   1833  4.153481


label attacks via masters

In [ ]:
# === Clean + label attacks.clean.csv using master manifests (robust) ===
from pathlib import Path
import pandas as pd
import numpy as np

ROOT = "/content/drive/MyDrive/hindi_dfake"
META = Path(ROOT)/"metadata"

ATT_CLEAN = META/"attacks.clean.csv"
REAL_CSV  = META/"master_real.csv"
FAKE_CSV  = META/"master_fake.csv"

assert ATT_CLEAN.exists(), f"Missing: {ATT_CLEAN}"
assert REAL_CSV.exists() and FAKE_CSV.exists(), "Missing master_real.csv or master_fake.csv"

# ---- helpers ----
def src_from_path(p: str) -> str:
    p = str(p)
    if "/raw/real_clean/commonvoice" in p: return "real_commonvoice"
    if "/raw/real_clean/indictts"   in p: return "real_indictts"
    if "/raw/real_clean/ivr"        in p: return "real_ivr"
    if "/raw/fake_tts"              in p: return "fake_tts"
    if "/raw/fake_vc"               in p: return "fake_vc"
    return "other"

from pathlib import Path as _P
def orig_stem_from_attacked(path: str, attack: str) -> str:
    s = _P(path).stem
    suf = f".{attack}"
    return s[:-len(suf)] if s.endswith(suf) else s

# ---- load and normalize attacks.clean.csv ----
dfA = pd.read_csv(ATT_CLEAN)
# ensure expected cols exist; keep current values in *_cur to avoid suffix mess after merge
for c in ["parent_utt","speaker_id","gender","text","label","source","fake_type","duration","attack","codec","bitrate","path"]:
    if c not in dfA.columns:
        dfA[c] = "" if c != "duration" else 0.0
rename_map = {
    "parent_utt":"parent_utt_cur",
    "speaker_id":"speaker_id_cur",
    "gender":"gender_cur",
    "text":"text_cur",
    "label":"label_cur",
    "source":"source_cur",
}
dfA = dfA.rename(columns=rename_map)
dfA["duration"] = pd.to_numeric(dfA["duration"], errors="coerce").fillna(0.0)
dfA["orig_stem"] = [orig_stem_from_attacked(p, a) for p, a in zip(dfA["path"], dfA["attack"])]

# ---- build stem->meta map from masters ----
def build_map(df_in, parent_label):
    rows = []
    for _, r in df_in.iterrows():
        rp = str(r.get("path",""))
        rows.append({
            "stem": _P(rp).stem,
            "m_label": parent_label,
            "m_source": src_from_path(rp),
            "m_fake_type": str(r.get("fake_type","")) if parent_label=="fake" else "",
            "m_utt": str(r.get("utt_id","")),
            "m_speaker": str(r.get("speaker_id","")),
            "m_gender": str(r.get("gender","")),
            "m_text": str(r.get("text","")),
        })
    return pd.DataFrame(rows)

dfR = pd.read_csv(REAL_CSV)
dfF = pd.read_csv(FAKE_CSV)

dfMap = pd.concat([build_map(dfR, "real"), build_map(dfF, "fake")], ignore_index=True)
dfMap = dfMap.drop_duplicates(subset=["stem"], keep="first")

# ---- merge attacks with map by original stem ----
dfM = dfA.merge(dfMap, how="left", left_on="orig_stem", right_on="stem")

# ---- fix label -> only {'real','fake'} ----
lab_cur = dfM["label_cur"].astype(str).str.lower()
lab_map = dfM["m_label"].astype(str).str.lower()
ft_map  = dfM["m_fake_type"].astype(str)

dfM["label"] = np.where(
    lab_cur.isin({"real","fake"}), lab_cur,
    np.where(lab_map.isin({"real","fake"}), lab_map,
             np.where(ft_map != "", "fake", ""))
)

# ---- fix source (prefer current, then map, then infer from path) ----
src_cur = dfM["source_cur"].astype(str)
src_final = src_cur.where(src_cur != "", dfM["m_source"])
src_final = src_final.where(src_final != "", dfM["path"].astype(str).map(src_from_path))
dfM["source"] = src_final

# ---- force fake_type for these rows ----
dfM["fake_type"] = "channel_attack"

# ---- backfill parent_utt/speaker/gender/text ----
def coalesce(cur, mapped):
    cur = cur.astype(str)
    mapped = mapped.astype(str)
    return np.where(cur != "", cur, mapped)

dfM["parent_utt"] = coalesce(dfM.get("parent_utt_cur", pd.Series([""]*len(dfM))),
                             dfM.get("m_utt", pd.Series([""]*len(dfM))))
dfM["speaker_id"] = coalesce(dfM.get("speaker_id_cur", pd.Series([""]*len(dfM))),
                             dfM.get("m_speaker", pd.Series([""]*len(dfM))))
dfM["gender"]     = coalesce(dfM.get("gender_cur", pd.Series([""]*len(dfM))),
                             dfM.get("m_gender", pd.Series([""]*len(dfM))))
dfM["text"]       = coalesce(dfM.get("text_cur", pd.Series([""]*len(dfM))),
                             dfM.get("m_text", pd.Series([""]*len(dfM))))

# ---- final tidy dataframe ----
out_cols = ["parent_utt","path","duration","speaker_id","gender","text","label",
            "fake_type","attack","codec","bitrate","source"]
df_out = dfM[out_cols].copy()

OUT = META/"attacks.labeled.csv"
df_out.to_csv(OUT, index=False)
print(f"Saved: {OUT} | rows={len(df_out)}")

# ---- quick summaries ----
def hours_series(s: pd.Series) -> float:
    return float(pd.to_numeric(s, errors="coerce").fillna(0.0).sum()/3600.0)

print("\nLabel counts:")
print(df_out["label"].value_counts(dropna=False).to_string())

print("\nBy attack × label:")
print(
    df_out.groupby(["attack","label"], dropna=False)
          .agg(files=("path","count"), hours=("duration", hours_series))
          .reset_index()
          .sort_values(["attack","label"])
          .to_string(index=False)
)

Saved: /content/drive/MyDrive/hindi_dfake/metadata/attacks.labeled.csv | rows=45093

Label counts:
label
real    23658
fake    21435

By attack × label:
attack label  files     hours
  lp8k  fake   9567 22.377793
  lp8k  real  10336 21.163750
opus16  fake  11868 27.002213
opus16  real  13322 29.203304


workspace audit

In [ ]:
# ==== WORKSPACE AUDIT (counts, existence, hours, formats) ====
# Set ROOT for the current account
ROOT = "/content/drive/MyDrive/hindi_dfake"   # <-- adjust if your copy lives elsewhere

from pathlib import Path
import os, time, pandas as pd, numpy as np, soundfile as sf

META = Path(ROOT)/"metadata"
RAW  = Path(ROOT)/"raw"
PROC = Path(ROOT)/"processed"
REAL_CSV = META/"master_real.csv"
FAKE_CSV = META/"master_fake.csv"

start = time.time()
def log(s): print(f"[+{time.time()-start:5.1f}s] {s}", flush=True)

# --- 1) CSV presence + sizes
for p in [REAL_CSV, FAKE_CSV]:
    if p.exists():
        log(f"Found {p} ({p.stat().st_size/1e6:.2f} MB)")
    else:
        log(f"Missing: {p}")

def load_csv(path, label_if_missing=None):
    if not path.exists():
        df = pd.DataFrame(columns=["utt_id","path","duration","speaker_id","gender","text","label"])
    else:
        df = pd.read_csv(path)
    if "label" not in df.columns and label_if_missing is not None:
        df["label"] = label_if_missing
    return df

log("Reading CSVs...")
dfr = load_csv(REAL_CSV, label_if_missing="real")
dff = load_csv(FAKE_CSV)
log(f"Rows — real: {len(dfr):,} | fake: {len(dff):,}")

# --- 2) Filter to your target folders
def src_from_path(p: str) -> str:
    p = str(p)
    if "/raw/real_clean/commonvoice" in p: return "real_commonvoice"
    if "/raw/real_clean/indictts"   in p: return "real_indictts"
    if "/raw/real_clean/ivr"        in p: return "real_ivr"
    if "/raw/fake_tts"              in p: return "fake_tts"
    return "other"

if len(dfr):
    paths_r = dfr["path"].astype(str)
    mask_r = (
        paths_r.str.contains("/raw/real_clean/commonvoice", regex=False, na=False) |
        paths_r.str.contains("/raw/real_clean/indictts",   regex=False, na=False) |
        paths_r.str.contains("/raw/real_clean/ivr",        regex=False, na=False)
    )
    dfr = dfr[mask_r].copy()
    dfr["source"] = dfr["path"].astype(str).map(src_from_path)

if len(dff):
    dff = dff[dff["path"].astype(str).str.contains("/raw/fake_tts", regex=False, na=False)].copy()
    dff["source"] = dff["path"].astype(str).map(src_from_path)

log(f"After folder filter — real: {len(dfr):,} | fake: {len(dff):,}")

# --- 3) Fast existence check
def fast_exists(col):
    pths = col.astype(str).tolist()
    return np.array([Path(p).exists() for p in pths], dtype=bool)

exist_r = fast_exists(dfr["path"]) if len(dfr) else np.array([], dtype=bool)
exist_f = fast_exists(dff["path"]) if len(dff) else np.array([], dtype=bool)

dfr_ok, dfr_miss = dfr[exist_r], dfr[~exist_r]
dff_ok, dff_miss = dff[exist_f], dff[~exist_f]

log(f"Exist — real: {len(dfr_ok):,}/{len(dfr):,} | fake: {len(dff_ok):,}/{len(dff):,}")

# --- 4) Hours on disk (fast-ish)
def hours_from_paths(df):
    s = 0.0
    cnt = 0
    for p in df["path"].astype(str):
        try:
            x, sr = sf.read(p, dtype="float32", always_2d=False)
            s += len(x)/float(sr); cnt += 1
        except: pass
    return s/3600.0, cnt

h_real, cnt_real = hours_from_paths(dfr_ok)
h_fake, cnt_fake = hours_from_paths(dff_ok)
log(f"On-disk hours — real: {h_real:.2f}h (files={cnt_real:,}) | fake: {h_fake:.2f}h (files={cnt_fake:,})")

# --- 5) CSV duration sums (approx)
def csv_hours(df):
    if "duration" in df.columns:
        return float(pd.to_numeric(df["duration"], errors="coerce").fillna(0).sum()/3600.0)
    return 0.0

log(f"CSV hours — real: {csv_hours(dfr):.2f}h | fake: {csv_hours(dff):.2f}h")

# --- 6) What’s missing? Show top missing prefixes
def top_prefixes(series, k=8):
    # take first ~6 path components to show where they point
    parts = series.astype(str).apply(lambda p: "/".join(p.split("/")[:7]))
    return parts.value_counts().head(k)

if len(dfr_miss):
    log("Top missing REAL prefixes:\n" + top_prefixes(dfr_miss["path"]).to_string())
if len(dff_miss):
    log("Top missing FAKE prefixes:\n" + top_prefixes(dff_miss["path"]).to_string())

# sample a few missing paths to eyeball
def sample_paths(df, n=5):
    return "\n".join(df["path"].astype(str).head(n).tolist())

if len(dfr_miss):
    log("Sample missing real paths:\n" + sample_paths(dfr_miss.sample(min(5, len(dfr_miss)), random_state=42)))
if len(dff_miss):
    log("Sample missing fake paths:\n" + sample_paths(dff_miss.sample(min(5, len(dff_miss)), random_state=42)))

# --- 7) Quick format probe on a small sample
def probe_formats(df, n=200):
    from collections import Counter
    sr = Counter(); ch = Counter(); st = Counter(); bad = 0
    for p in df["path"].astype(str).sample(min(n, len(df)), random_state=123):
        try:
            info = sf.info(p)
            sr[info.samplerate]+=1; ch[info.channels]+=1; st[info.subtype]+=1
        except: bad += 1
    return sr, ch, st, bad

if len(dfr_ok):
    sr, ch, st, bad = probe_formats(dfr_ok)
    log(f"REAL probe — sr={dict(sr)} ch={dict(ch)} subtype={dict(st)} bad={bad}")
if len(dff_ok):
    sr, ch, st, bad = probe_formats(dff_ok)
    log(f"FAKE probe — sr={dict(sr)} ch={dict(ch)} subtype={dict(st)} bad={bad}")

log("Audit done.")

[+  0.0s] Found /content/drive/MyDrive/hindi_dfake/metadata/master_real.csv (10.29 MB)
[+  0.0s] Found /content/drive/MyDrive/hindi_dfake/metadata/master_fake.csv (10.78 MB)
[+  0.0s] Reading CSVs...
[+  0.4s] Rows — real: 25,965 | fake: 26,735
[+  0.5s] After folder filter — real: 25,965 | fake: 26,735
[+ 27.2s] Exist — real: 25,965/25,965 | fake: 26,735/26,735


Fake Facebook mms

In [ ]:
# --- MMS-TTS (Meta) Hindi smoke test: 2 clips, Colab-ready, no API keys ---
# Writes WAVs to /content/drive/MyDrive/hindi_dfake/raw/fake_tts_mms
# Appends rows to /content/drive/MyDrive/hindi_dfake/metadata/master_fake.csv

!pip -q install torch torchaudio transformers soundfile librosa pandas numpy tqdm > /dev/null

import os, hashlib, numpy as np, pandas as pd, soundfile as sf, librosa, torch
from pathlib import Path
from tqdm.auto import tqdm
from IPython.display import Audio, display

from transformers import VitsModel, AutoTokenizer

ROOT = "/content/drive/MyDrive/hindi_dfake"
OUT_DIR = f"{ROOT}/raw/fake_tts_mms"        # separate folder for this TTS
FAKE_CSV = f"{ROOT}/metadata/master_fake.csv"
os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(Path(FAKE_CSV).parent, exist_ok=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# Load MMS-TTS Hindi
model_name = "facebook/mms-tts-hin"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = VitsModel.from_pretrained(model_name).to(device).eval()
base_sr = getattr(model.config, "sampling_rate", 16000)  # usually 16000

# Two short Hindi lines to audition
texts = [
    "यह एक त्वरित जाँच है। क्या आवाज़ स्पष्ट है?",
    "दूसरा नमूना तैयार है। यदि ठीक लगे तो हम बड़े बैच पर चलेंगे।"
]

def uid_for(text):
    return "tts_mms_" + hashlib.sha1(text.encode("utf-8")).hexdigest()[:16]

rows, clips = [], []
for text in tqdm(texts, desc="Synthesize"):
    uid = uid_for(text)
    wav_path = os.path.join(OUT_DIR, f"{uid}.wav")

    # Tokenize and generate waveform
    inputs = tokenizer(text, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        out = model(**inputs)
        wav = out.waveform.squeeze().cpu().float().numpy()

    sr = base_sr
    # Ensure mono float32 and resample to 16k
    if wav.ndim > 1:
        wav = wav.mean(axis=0)
    if sr != 16000:
        wav = librosa.resample(wav.astype(np.float32), orig_sr=sr, target_sr=16000)
        sr = 16000

    # Normalize to [-1,1] and write PCM_16
    wav = np.clip(wav, -1.0, 1.0).astype(np.float32)
    sf.write(wav_path, wav, sr, subtype="PCM_16")

    dur = len(wav)/sr
    if dur < 0.5 or float(np.abs(wav).mean()) < 1e-5:
        print(f"Skipping too-short/quiet clip: {uid}")
        continue

    # Append CSV row (your schema)
    rows.append({
        "utt_id": uid,
        "path": wav_path,
        "duration": round(dur, 3),
        "speaker_id": "mms_tts",
        "gender": "u",
        "text": text,
        "label": "fake",
        "fake_type": "tts_mms",
        "tts_model": "facebook/mms-tts-hin",
        "voice": "default",
        "rate": "+0%",
        "seed": ""
    })
    clips.append(wav_path)

# Write/append metadata
if rows:
    df_new = pd.DataFrame(rows)
    if Path(FAKE_CSV).exists():
        df_out = pd.concat([pd.read_csv(FAKE_CSV), df_new], ignore_index=True)
    else:
        df_out = df_new
    df_out.to_csv(FAKE_CSV, index=False)
    print(f"✅ Appended {len(rows)} rows → {FAKE_CSV} (total rows: {len(df_out)})")

# Playback
for p in clips:
    print(Path(p).name)
    display(Audio(p, rate=16000))

Device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/907 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/145M [00:00<?, ?B/s]

Synthesize:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Appended 2 rows → /content/drive/MyDrive/hindi_dfake/metadata/master_fake.csv (total rows: 23735)
tts_mms_9e7cc168f452d2bf.wav


tts_mms_c0298ff1eb16c587.wav


MMS-TTS bulk

In [ ]:
# =================== MMS-TTS (Meta) Hindi — BULK GENERATOR (~3k clips) ===================
# - Free, keyless, Colab-ready
# - Writes WAVs:  /content/drive/MyDrive/hindi_dfake/raw/fake_tts_mms
# - Appends CSV:  /content/drive/MyDrive/hindi_dfake/metadata/master_fake.csv
# - Resume-safe, dedup against disk + CSV, quality-gated, 16 kHz mono PCM_16

!pip -q install torch torchaudio transformers soundfile librosa pandas numpy tqdm > /dev/null

import os, hashlib, random, json, time
from pathlib import Path
from typing import List
import numpy as np
import pandas as pd
import soundfile as sf
import librosa
import torch
from tqdm.auto import tqdm
from transformers import VitsModel, AutoTokenizer

# ------------------ CONFIG ------------------
ROOT        = "/content/drive/MyDrive/hindi_dfake"
REAL_CSV    = f"{ROOT}/metadata/master_real.csv"     # source texts
OUT_DIR     = f"{ROOT}/raw/fake_tts_mms"             # target wav folder (separate from edge_tts)
FAKE_CSV    = f"{ROOT}/metadata/master_fake.csv"     # global metadata you’ve been using

TARGET_CLIPS   = 3000      # ~3k audios
SEED           = 2025
MIN_CHARS      = 6
MAX_CHARS      = 220
PACK_N         = 1         # set to 2 or 3 to concatenate sentences with " । "
MAX_PACK_LEN   = 300       # max total chars when packing
QUALITY_MIN_SEC= 0.6
QUALITY_MIN_MEAN_ABS = 1e-5
CSV_FLUSH_EVERY = 50       # flush metadata to disk every N clips

MODEL_NAME    = "facebook/mms-tts-hin"
VOICE_TAG     = "default"
FAKE_TAG      = "tts_mms"
RATE_TAG      = "+0%"
SPEAKER_ID    = "mms_tts"
GENDER_TAG    = "u"

# ------------------ SETUP ------------------
random.seed(SEED); np.random.seed(SEED)
Path(OUT_DIR).mkdir(parents=True, exist_ok=True)
Path(Path(FAKE_CSV).parent).mkdir(parents=True, exist_ok=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# Load model/tokenizer once
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model     = VitsModel.from_pretrained(MODEL_NAME).to(device).eval()
base_sr   = getattr(model.config, "sampling_rate", 16000)

# ------------------ TEXTS ------------------
def load_texts_from_real(csv_path: str) -> List[str]:
    df = pd.read_csv(csv_path)
    # fallbacks you used before
    for col in ["text", "transcript", "sentence"]:
        if col in df.columns:
            ser = df[col].astype(str)
            break
    else:
        raise ValueError("master_real.csv must have 'text' (or 'transcript'/'sentence') column.")
    texts = (ser.str.replace("\n"," ", regex=False)
                .str.strip()
                .dropna()
                .loc[lambda s: s.str.len().between(MIN_CHARS, MAX_CHARS)]
                .drop_duplicates()
                .tolist())
    random.shuffle(texts)
    return texts

texts_base = load_texts_from_real(REAL_CSV)
print(f"Loaded {len(texts_base)} base sentences")

JOINER = " । "
def text_stream(items: List[str], n=1, max_len=300):
    """Infinite generator of texts; packs n sentences if n>1."""
    i = random.randrange(max(1, len(items)))
    while True:
        if n <= 1:
            t = items[i % len(items)].strip(); i += 1
            if len(t) >= MIN_CHARS: yield t
            continue
        bundle, cur = [], 0
        while len(bundle) < n:
            t = items[i % len(items)].strip(); i += 1
            if len(t) < MIN_CHARS: continue
            add = (len(t) if cur == 0 else 1 + len(t))
            if cur and cur + add > max_len:
                break
            bundle.append(t); cur += add
        if bundle:
            yield JOINER.join(bundle)

ts = text_stream(texts_base, n=PACK_N, max_len=MAX_PACK_LEN)

# ------------------ RESUME/DEDUP ------------------
def uid_for(text: str) -> str:
    # Include model + voice in hash key to avoid collisions across TTS backends
    key = f"{text}|{MODEL_NAME}|{VOICE_TAG}|{SEED}"
    return "tts_mms_" + hashlib.sha1(key.encode("utf-8")).hexdigest()[:16]

existing_uids = set()
for p in Path(OUT_DIR).glob("*.wav"):
    if p.stem.startswith("tts_mms_"):
        existing_uids.add(p.stem)

if Path(FAKE_CSV).exists():
    try:
        df_exist = pd.read_csv(FAKE_CSV, usecols=["utt_id"])
        existing_uids |= set(df_exist["utt_id"].astype(str).tolist())
    except Exception:
        pass

print(f"Resume: {len(existing_uids)} items already indexed (disk and/or CSV)")

# ------------------ SYNTHESIS ------------------
def synth_one(text: str):
    """Return (wav_path, duration_sec) or (None, None) on failure/quality reject."""
    uid = uid_for(text)
    wav_path = os.path.join(OUT_DIR, f"{uid}.wav")
    if uid in existing_uids and os.path.isfile(wav_path):
        return None, None  # already have it

    # tokenize + generate
    inputs = tokenizer(text, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        out = model(**inputs)
        wav = out.waveform.squeeze().cpu().float().numpy()

    sr = base_sr
    if wav.ndim > 1:
        wav = wav.mean(axis=0)
    if sr != 16000:
        wav = librosa.resample(wav.astype(np.float32), orig_sr=sr, target_sr=16000)
        sr = 16000

    wav = np.clip(wav, -1.0, 1.0).astype(np.float32)
    dur = len(wav) / sr

    # quality gates (like your earlier pipelines)
    if dur < QUALITY_MIN_SEC or float(np.abs(wav).mean()) < QUALITY_MIN_MEAN_ABS:
        return None, None

    # write PCM_16
    sf.write(wav_path, wav, sr, subtype="PCM_16")
    return wav_path, dur

# ------------------ MAIN LOOP ------------------
to_make = max(0, TARGET_CLIPS)
made = 0
rows = []
pbar = tqdm(total=to_make, desc="MMS-TTS clips")

while made < to_make:
    text = next(ts)
    uid = uid_for(text)
    if uid in existing_uids:
        continue

    wav_path, dur = synth_one(text)
    if wav_path is None:
        continue

    rows.append({
        "utt_id": uid,
        "path": wav_path,
        "duration": round(float(dur), 3),
        "speaker_id": SPEAKER_ID,
        "gender": GENDER_TAG,
        "text": text,
        "label": "fake",
        "fake_type": FAKE_TAG,
        "tts_model": MODEL_NAME,
        "voice": VOICE_TAG,
        "rate": RATE_TAG,
        "seed": SEED
    })
    existing_uids.add(uid)
    made += 1
    pbar.update(1)

    # periodic flush (resume-safe)
    if (made % CSV_FLUSH_EVERY) == 0:
        if rows:
            if Path(FAKE_CSV).exists():
                df_out = pd.concat([pd.read_csv(FAKE_CSV), pd.DataFrame(rows)], ignore_index=True)
            else:
                df_out = pd.DataFrame(rows)
            df_out.to_csv(FAKE_CSV, index=False)
            rows = []

# final flush
if rows:
    if Path(FAKE_CSV).exists():
        df_out = pd.concat([pd.read_csv(FAKE_CSV), pd.DataFrame(rows)], ignore_index=True)
    else:
        df_out = pd.DataFrame(rows)
    df_out.to_csv(FAKE_CSV, index=False)
    rows = []

pbar.close()

print("\n✅ Done.")
print(f"   Wrote ~{TARGET_CLIPS} clips → {OUT_DIR}")
print(f"   Updated metadata → {FAKE_CSV}")

Device: cuda
Loaded 24713 base sentences
Resume: 23735 items already indexed (disk and/or CSV)


MMS-TTS clips:   0%|          | 0/3000 [00:00<?, ?it/s]


✅ Done.
   Wrote ~3000 clips → /content/drive/MyDrive/hindi_dfake/raw/fake_tts_mms
   Updated metadata → /content/drive/MyDrive/hindi_dfake/metadata/master_fake.csv


MMS-TTS Sanity check

In [ ]:
# =================== MMS TTS: Sanity + CSV Sync ===================
# - Counts WAVs on disk + total hours
# - Backs up master_fake.csv
# - Syncs: drops rows whose files are missing; adds WAVs missing from CSV
# - Deduplicates by path and utt_id
# - Recounts using proper pandas .str accessors

from pathlib import Path
import os, csv, shutil
import pandas as pd
import soundfile as sf
from collections import Counter

# ---------- CONFIG ----------
ROOT     = Path("/content/drive/MyDrive/hindi_dfake")
MMS_DIR  = ROOT / "raw/fake_tts_mms"                     # MMS outputs folder
FAKE_CSV = ROOT / "metadata/master_fake.csv"
BACKUP   = ROOT / "metadata/master_fake.mms_sync.bak"

FAKE_TYPE  = "tts_mms"
TTS_MODEL  = "facebook/mms-tts-hin"                  # tag for metadata
VOICE_TAG  = ""                                      # MMS has no named voices
RATE_TAG   = ""                                      # not applicable
SEED_TAG   = ""                                      # not applicable

# Create metadata dir if needed
FAKE_CSV.parent.mkdir(parents=True, exist_ok=True)

# ---------- Helpers ----------
def folder_seconds(dirp: Path) -> float:
    s = 0.0
    for p in dirp.glob("*.wav"):
        try:
            d, sr = sf.read(str(p))
            s += len(d) / float(sr)
        except:
            pass
    return s

def read_duration_sec(p: Path) -> float:
    try:
        d, sr = sf.read(str(p))
        return len(d) / float(sr)
    except:
        return 0.0

# ---------- 1) Folder sanity ----------
assert MMS_DIR.exists(), f"MMS_DIR not found: {MMS_DIR}"
wav_paths = sorted(MMS_DIR.glob("*.wav"))
hours = folder_seconds(MMS_DIR)/3600.0
print(f"[MMS] wav files on disk: {len(wav_paths)} | hours ≈ {hours:.2f}")

# Show a few example files
for p in wav_paths[:3]:
    print("▶", p.name)

# ---------- 2) Load / initialize CSV ----------
if not FAKE_CSV.exists():
    # Initialize with header if not present
    with open(FAKE_CSV, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(["utt_id","path","duration","speaker_id","gender","text",
                    "label","fake_type","tts_model","voice","rate","seed"])
    print(f"[init] Created new CSV with header → {FAKE_CSV}")

# Backup
shutil.copyfile(FAKE_CSV, BACKUP)
print(f"[backup] → {BACKUP}")

# Read CSV
df = pd.read_csv(FAKE_CSV)

# Ensure required columns exist (fill blanks if missing)
required_cols = ["utt_id","path","duration","speaker_id","gender","text",
                 "label","fake_type","tts_model","voice","rate","seed"]
for c in required_cols:
    if c not in df.columns:
        df[c] = ""

# ---------- 3) Basic validation: keep rows that point to real files ----------
# We'll only drop rows that reference files that don't exist *and* are inside MMS_DIR.
# (We won't touch rows for other datasets.)
def is_inside_mms_dir(p_str: str) -> bool:
    try:
        return str(p_str).startswith(str(MMS_DIR))
    except:
        return False

exists_mask = df["path"].astype(str).apply(lambda p: Path(p).exists())
inside_mms_mask = df["path"].astype(str).apply(is_inside_mms_dir)
missing_on_disk_mask = inside_mms_mask & (~exists_mask)
missing_on_disk = df[missing_on_disk_mask]

print(f"[sync] missing_on_disk rows: {len(missing_on_disk)}")

# Drop those missing-on-disk MMS rows
if len(missing_on_disk):
    df = df.loc[~missing_on_disk_mask].copy()

# ---------- 4) Find WAVs on disk that aren't in the CSV ----------
csv_paths = set(Path(p).name for p in df["path"].astype(str) if p.endswith(".wav"))
disk_paths = {p.name: p for p in wav_paths}
missing_on_csv_names = list(sorted(disk_paths.keys() - csv_paths))
print(f"[sync] missing_on_csv files: {len(missing_on_csv_names)}")

# ---------- 5) Add any missing WAVs to CSV with minimal metadata ----------
add_rows = []
for name in missing_on_csv_names:
    p = disk_paths[name]
    uid = p.stem
    dur = read_duration_sec(p)
    # Minimal row; text unknown (unless you saved it elsewhere)
    add_rows.append({
        "utt_id": uid,
        "path": str(p),
        "duration": round(dur, 3),
        "speaker_id": "mms_tts",
        "gender": "u",
        "text": "",
        "label": "fake",
        "fake_type": FAKE_TYPE,
        "tts_model": TTS_MODEL,
        "voice": VOICE_TAG,
        "rate": RATE_TAG,
        "seed": SEED_TAG
    })

if add_rows:
    df = pd.concat([df, pd.DataFrame(add_rows)], ignore_index=True)

# ---------- 6) Deduplicate (path first, then utt_id) ----------
before = len(df)
df = df.drop_duplicates(subset=["path"], keep="first")
df = df.drop_duplicates(subset=["utt_id"], keep="first")
dedup_removed = before - len(df)
print(f"[sync] dedup removed {dedup_removed} rows (global)")

# ---------- 7) Save CSV ----------
df.to_csv(FAKE_CSV, index=False)
print(f"[sync] ✅ saved → {FAKE_CSV}")

# ---------- 8) Post recount (FIXED: use .str.startswith) ----------
df_all = pd.read_csv(FAKE_CSV)
fake_type_series = df_all.get("fake_type", pd.Series([""] * len(df_all)))
mask_mms = (
    df_all["path"].astype(str).str.startswith(str(MMS_DIR))
    | fake_type_series.astype(str).eq(FAKE_TYPE)
)
print(f"[post] MMS rows in CSV: {int(mask_mms.sum())} | total rows: {len(df_all)}")

# ---------- 9) (Optional) Quick audit of audio format ----------
# Comment out if not needed — this can take a bit on large sets.
AUDIT = False
if AUDIT:
    print("\n[audit] probing MMS WAV formats ...")
    sr_ct = Counter(); ch_ct = Counter(); sub_ct = Counter()
    bad = 0
    for p in wav_paths[:1000]:  # cap audit for speed; remove slice for full
        try:
            info = sf.info(str(p))
            sr_ct[info.samplerate] += 1
            ch_ct[info.channels] += 1
            sub_ct[info.subtype] += 1
        except:
            bad += 1
    print(" sample rates:", dict(sr_ct))
    print(" channels:    ", dict(ch_ct))
    print(" subtype:     ", dict(sub_ct))
    print(" unreadable:  ", bad)

[MMS] wav files on disk: 3002 | hours ≈ 5.50
▶ tts_mms_002ac71fc558fd80.wav
▶ tts_mms_0031889fd22560a0.wav
▶ tts_mms_004c969ada2f33f4.wav
[backup] → /content/drive/MyDrive/hindi_dfake/metadata/master_fake.mms_sync.bak
[sync] missing_on_disk rows: 0
[sync] missing_on_csv files: 0
[sync] dedup removed 0 rows (global)
[sync] ✅ saved → /content/drive/MyDrive/hindi_dfake/metadata/master_fake.csv
[post] MMS rows in CSV: 3002 | total rows: 26735


# **PRE - PROCESSING**

In [ ]:
import pandas as pd, glob, os
root = "/content/drive/MyDrive/hindi_dfake/metadata"
for f in sorted(glob.glob(f"{root}/*.csv")):
    try:
        df = pd.read_csv(f, nrows=5)  # peek header fast
        if "label" in df.columns:
            df = pd.read_csv(f, usecols=["label"])
            print(f, "\n", df["label"].value_counts(dropna=False), "\n")
        else:
            print(f, "(no 'label' column) {os.path.basename(f)} — skipped\n")
    except Exception as e:
        print(f, "(read issue) {os.path.basename(f)} — {e}\n")

/content/drive/MyDrive/hindi_dfake/metadata/attacks.clean.csv 
 label
real        20672
fake        17334
NaN          5287
tts_edge     1800
Name: count, dtype: int64 

/content/drive/MyDrive/hindi_dfake/metadata/attacks.csv 
 label
real        20672
fake        17334
tts_edge     1800
Name: count, dtype: int64 

/content/drive/MyDrive/hindi_dfake/metadata/attacks.labeled.csv 
 label
real    23658
fake    21435
Name: count, dtype: int64 

/content/drive/MyDrive/hindi_dfake/metadata/master_fake.csv 
 label
fake    23731
Name: count, dtype: int64 

/content/drive/MyDrive/hindi_dfake/metadata/master_real.csv 
 label
real    25965
Name: count, dtype: int64 

/content/drive/MyDrive/hindi_dfake/metadata/proc_manifest.csv (no 'label' column) {os.path.basename(f)} — skipped



ffmpeg filter chain

In [ ]:
# --- PROBE ONE FILE to see ffmpeg/filter errors clearly ---
import pandas as pd
from pathlib import Path
import subprocess, tempfile, soundfile as sf, numpy as np

ROOT = Path("/content/drive/MyDrive/hindi_dfake")
META = ROOT/"metadata"
REAL = pd.read_csv(META/"master_real.csv")
FAKE = pd.read_csv(META/"master_fake.csv")
df = pd.concat([REAL, FAKE], ignore_index=True)
df = df[df["path"].apply(lambda p: Path(p).exists())]
sample = df.iloc[0]["path"]
print("Sample file:", sample)

TRIM_THR_DB = -45
TRIM_DUR_S  = 0.20
GAIN_DB     = 0.0  # just to check filter wiring
PROFILE     = "base"

filt_parts = []
filt_parts.append("highpass=f=20")
# no EQ for base
# per-file gain
filt_parts.append(f"volume={GAIN_DB}dB")
# NOTE: use start_duration/stop_duration (not start_silence/stop_silence)
filt_parts.append(f"silenceremove=start_periods=1:start_duration={TRIM_DUR_S}:start_threshold={TRIM_THR_DB}dB:"
                  f"stop_periods=1:stop_duration={TRIM_DUR_S}:stop_threshold={TRIM_THR_DB}dB")
FILTER = ",".join(filt_parts)

tmp = Path(tempfile.gettempdir())/ "probe_out.wav"
cmd = [
    "ffmpeg","-nostdin","-hide_banner","-loglevel","error","-y",
    "-i", sample,
    "-ac","1","-ar","16000","-af", FILTER, "-sample_fmt","s16",
    str(tmp)
]
print("Filter:", FILTER)
p = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
print("Return code:", p.returncode)
if p.returncode != 0:
    print("----- STDERR -----\n", p.stderr[:1200])
else:
    try:
        import soundfile as sf, numpy as np
        x, sr = sf.read(str(tmp), dtype="float32")
        print("OK. Out shape:", x.shape, "sr:", sr, "dur(s):", len(x)/sr)
    except Exception as e:
        print("Wrote file but could not read back:", e)

Sample file: /content/drive/MyDrive/hindi_dfake/raw/real_clean/ivr/1a0b61275bc58baf.wav
Filter: highpass=f=20,volume=0.0dB,silenceremove=start_periods=1:start_duration=0.2:start_threshold=-45dB:stop_periods=1:stop_duration=0.2:stop_threshold=-45dB
Return code: 0
OK. Out shape: (129127,) sr: 16000 dur(s): 8.0704375


In [ ]:
from pathlib import Path
import soundfile as sf, numpy as np, pandas as pd

ROOT = Path("/content/drive/MyDrive/hindi_dfake")
PROFILE = "base"
OUT_DIR = ROOT / "processed" / "wav" / PROFILE

def hours_here(d: Path):
    s=0.0
    for p in d.rglob("*.wav"):
        try:
            x, sr = sf.read(str(p), dtype="float32", always_2d=False)
            s += len(x)/float(sr)
        except: pass
    return s/3600

h = hours_here(OUT_DIR)
print(f"{PROFILE} files:", len(list(OUT_DIR.rglob('*.wav'))), "| hours:", f"{h:.2f}h")

# compare to manifest
mf = pd.read_csv(ROOT/"metadata"/"proc_manifest.csv")
mfp = mf[mf["profile"]==PROFILE]
print("Manifest rows:", len(mfp), "| manifest hours:", f"{mfp['duration'].fillna(0).sum()/3600:.2f}h")

base files: 0 | hours: 0.00h
Manifest rows: 0 | manifest hours: 0.00h


In [ ]:
import random, numpy as np, soundfile as sf
from pathlib import Path

ROOT = Path("/content/drive/MyDrive/hindi_dfake")
PROFILE = "base"
paths = list((ROOT/"processed"/"wav"/PROFILE).rglob("*.wav"))
random.shuffle(paths)
paths = paths[:2000]  # sample

rms_db = []
clip_pct = []
for p in paths:
    try:
        x, sr = sf.read(str(p), dtype="float32", always_2d=False)
        if x.ndim>1: x = x.mean(axis=1)
        if len(x)==0: continue
        rms  = float(np.sqrt(np.mean(np.square(x))))
        rms_db.append(-120. if rms<=1e-9 else 20*np.log10(max(rms,1e-9)))
        clip_pct.append(100.0*np.mean(np.abs(x)>=0.999))
    except: pass

if rms_db:
    print(f"RMS dB ~ mean: {np.mean(rms_db):.1f}, median: {np.median(rms_db):.1f}, 5–95%: {np.percentile(rms_db,5):.1f}/{np.percentile(rms_db,95):.1f}")
    print(f"Clipping % ~ mean: {np.mean(clip_pct):.3f}%  (should be ~0)")
else:
    print("No samples read.")


No samples read.


mini-test base profile

In [ ]:
# ---- Mini-test preprocessing: 20 real + 20 fake (same-FS temp; manifest-aware) ----
import os, csv, uuid, subprocess, hashlib, time
from pathlib import Path
import pandas as pd, numpy as np, soundfile as sf

# === Config (match your full cell) ===
ROOT = Path("/content/drive/MyDrive/hindi_dfake")
META = ROOT / "metadata"
OUT_ROOT = ROOT / "processed" / "wav"
PROFILE = "base"                 # change if you're testing another profile
INCLUDE_ATTACKS = True           # include attacks.labeled.csv in the pool
TARGET_DB   = -26.0
TRIM_THR_DB = -45
TRIM_DUR_S  = 0.20
MANIFEST = META / "proc_manifest.csv"
N_PER_LABEL = 20                 # 20 real + 20 fake = 40 files

REAL_CSV  = META / "master_real.csv"
FAKE_CSV  = META / "master_fake.csv"
ATT_CSV   = META / "attacks.labeled.csv"

def read_csv_safe(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path)
    for c in ["utt_id","path","duration","label","fake_type","source"]:
        if c not in df.columns: df[c] = "" if c != "duration" else 0.0
    return df

def normalize_label(lab, fake_type, src, path):
    l = str(lab).strip().lower()
    if l in {"real","fake"}: return l
    ft = str(fake_type).strip().lower()
    s  = str(src).strip().lower()
    p  = str(path)
    if "/raw/real_clean" in p or s.startswith("real_"): return "real"
    if ft in {"tts_edge","channel_attack","vc"} or "/raw/fake_" in p or s.startswith("fake_"): return "fake"
    return "fake"

def out_path_for(in_path: str) -> Path:
    p = Path(in_path)
    try: rel = p.relative_to(ROOT)
    except: rel = Path("_external") / p.name
    return OUT_ROOT / PROFILE / rel

def ensure_parent(p: Path): p.parent.mkdir(parents=True, exist_ok=True)

def rms_dbfs(p: Path) -> float:
    try:
        x, sr = sf.read(str(p), dtype="float32", always_2d=False)
        if x.ndim > 1: x = x.mean(axis=1)
        if len(x) == 0: return -120.0
        rms = float(np.sqrt(np.mean(np.square(x))))
        if rms <= 1e-9: return -120.0
        return float(20.0*np.log10(np.clip(rms, 1e-9, 1.0)))
    except Exception:
        return -120.0

def build_filter_chain(profile: str, gain_db: float) -> str:
    parts = ["highpass=f=20"]
    if profile in ("research_eq","strong"):
        parts += [
            "equalizer=f=3000:t=q:w=1.0:g=2.5",
            "equalizer=f=4800:t=q:w=0.9:g=2.0",
            "treble=g=1.0:f=6000:t=h:w=0.7"
        ]
    parts.append(f"volume={gain_db}dB")
    parts.append(
        f"silenceremove=start_periods=1:start_duration={TRIM_DUR_S}:start_threshold={TRIM_THR_DB}dB:"
        f"stop_periods=1:stop_duration={TRIM_DUR_S}:stop_threshold={TRIM_THR_DB}dB"
    )
    return ",".join(parts)

# === Build pool (real + fake + attacks if enabled) ===
dfs = [read_csv_safe(REAL_CSV), read_csv_safe(FAKE_CSV)]
if INCLUDE_ATTACKS and ATT_CSV.exists():
    dfs.append(read_csv_safe(ATT_CSV))
df_all = pd.concat(dfs, ignore_index=True)
df_all = df_all[df_all["path"].astype(str).apply(lambda p: Path(p).exists())].copy()
df_all["label"] = [normalize_label(l, ft, s, p) for l, ft, s, p in zip(
    df_all["label"], df_all["fake_type"], df_all["source"], df_all["path"]
)]

print("Pool counts:", df_all["label"].value_counts().to_dict())

# === Skip items already processed for this profile ===
done_set = set()
if MANIFEST.exists():
    mf = pd.read_csv(MANIFEST, usecols=["profile","path_out"])
    done_set = set(mf.loc[mf["profile"]==PROFILE, "path_out"].astype(str))

def is_done(pin: str) -> bool:
    return (out_path_for(pin).exists()) or (str(out_path_for(pin)) in done_set)

df_all = df_all[~df_all["path"].astype(str).apply(is_done)].copy()

# === Balanced sample: N_PER_LABEL from each label ===
sample_rows = []
for lab in ["real","fake"]:
    g = df_all[df_all["label"]==lab]
    sample_rows += g.sample(min(N_PER_LABEL, len(g)), random_state=123).to_dict("records")
print(f"Will process: real={min(N_PER_LABEL, len(df_all[df_all['label']=='real']))}, "
      f"fake={min(N_PER_LABEL, len(df_all[df_all['label']=='fake']))}")

# === Process sequentially (clear stderr on any failure) ===
created, errs = 0, 0
err_examples = []
MANIFEST.parent.mkdir(parents=True, exist_ok=True)
if not MANIFEST.exists():
    with open(MANIFEST, "w", newline="", encoding="utf-8") as f:
        csv.writer(f).writerow(["utt_id","path_in","path_out","profile","duration","rms_db","peak_db"])

for r in sample_rows:
    pin  = str(r["path"])
    pout = out_path_for(pin)
    ensure_parent(pout)

    gain_db = float(np.clip(TARGET_DB - rms_dbfs(Path(pin)), -20.0, 20.0))
    FILTER  = build_filter_chain(PROFILE, gain_db)

    tmp = pout.parent / f".{pout.stem}.tmp-{uuid.uuid4().hex}.wav"  # same-FS temp
    cmd = [
        "ffmpeg","-nostdin","-hide_banner","-loglevel","error","-y",
        "-i", pin, "-ac","1","-ar","16000","-af", FILTER, "-sample_fmt","s16", str(tmp)
    ]
    p = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if p.returncode != 0:
        errs += 1
        try: tmp.unlink(missing_ok=True)
        except: pass
        if len(err_examples) < 3:
            err_examples.append((pin, p.stderr[:600]))
        print("[ERR] ffmpeg failed for:", pin)
        continue

    os.replace(str(tmp), str(pout))

    # write one manifest row
    try:
        x, sr = sf.read(str(pout), dtype="float32", always_2d=False)
        if x.ndim>1: x = x.mean(axis=1)
        dur = len(x)/float(sr) if sr else 0.0
        peak = float(np.max(np.abs(x))) if len(x) else 0.0
        rms  = float(np.sqrt(np.mean(np.square(x)))) if len(x) else 0.0
        rms_db = -120.0 if rms<=1e-9 else float(20*np.log10(np.clip(rms,1e-9,1.0)))
        peak_db = -120.0 if peak<=1e-9 else float(20*np.log10(np.clip(peak,1e-9,1.0)))
    except Exception:
        dur, rms_db, peak_db = 0.0, -120.0, -120.0

    with open(MANIFEST, "a", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=["utt_id","path_in","path_out","profile","duration","rms_db","peak_db"])
        w.writerow({
            "utt_id": r.get("utt_id",""),
            "path_in": pin,
            "path_out": str(pout),
            "profile": PROFILE,
            "duration": round(dur,3),
            "rms_db": round(rms_db,2),
            "peak_db": round(peak_db,2),
        })
    created += 1
    print("[ok]", pout)

print(f"\nMini-test done. created={created}, errors={errs}")
if err_examples:
    print("\n--- First errors (trimmed) ---")
    for pin, msg in err_examples:
        print(pin, "\n", msg, "\n")

# quick check: count files + manifest rows for this profile
from itertools import islice
outs = list((OUT_ROOT/PROFILE).rglob("*.wav"))
print(f"Now in processed/wav/{PROFILE}: files={len(outs)} (sample shown below)")
for p in outs[:5]:
    print("  ", p)
m = pd.read_csv(MANIFEST)
m = m[m["profile"]==PROFILE]
print(f"Manifest rows for '{PROFILE}':", len(m))
print("Manifest hours:", round(pd.to_numeric(m["duration"], errors="coerce").fillna(0).sum()/3600, 2), "h")

Pool counts: {'real': 49623, 'fake': 45166}
Will process: real=20, fake=20
[ok] /content/drive/MyDrive/hindi_dfake/processed/wav/base/processed/attacks/lp8k/2f1ca2233eb6b6a9.lp8k.wav
[ok] /content/drive/MyDrive/hindi_dfake/processed/wav/base/raw/real_clean/indictts/5969b01dbf6cbabd.wav
[ok] /content/drive/MyDrive/hindi_dfake/processed/wav/base/processed/attacks/opus16/9f55e16acb8415da.opus16.wav
[ok] /content/drive/MyDrive/hindi_dfake/processed/wav/base/raw/real_clean/commonvoice/85eefbbedb900d1b.wav
[ok] /content/drive/MyDrive/hindi_dfake/processed/wav/base/raw/real_clean/ivr/ae46b36408345e59.wav
[ok] /content/drive/MyDrive/hindi_dfake/processed/wav/base/raw/real_clean/ivr/53c8b5ddc1482727.wav
[ok] /content/drive/MyDrive/hindi_dfake/processed/wav/base/processed/attacks/opus16/7fb2160ad56aac3c.opus16.wav
[ok] /content/drive/MyDrive/hindi_dfake/processed/wav/base/processed/attacks/opus16/83737a0972c7e11c.opus16.wav
[ok] /content/drive/MyDrive/hindi_dfake/processed/wav/base/raw/real_clea

Base profile

In [ ]:
# ========== PREPROCESSING (full run; resumable; same-FS temp) ==========
# Pipeline: DC high-pass -> (optional artifact EQ) -> per-file volume -> light trim
# Writes: /content/drive/MyDrive/hindi_dfake/processed/wav/<profile>/...
# Logs  : /content/drive/MyDrive/hindi_dfake/metadata/proc_manifest.csv

import os, csv, time, uuid, subprocess, hashlib
from pathlib import Path
import pandas as pd, numpy as np, soundfile as sf
from concurrent.futures import ThreadPoolExecutor, as_completed

# ---------------- CONFIG ----------------
ROOT = Path("/content/drive/MyDrive/hindi_dfake")
META = ROOT / "metadata"
OUT_ROOT = ROOT / "processed" / "wav"

PROFILE = "base"           # "base", "research_eq", "research_rb", "strong"
INCLUDE_ATTACKS = True     # include attacks.labeled.csv
FRACTION_REAL = 1.00       # process EVERYTHING
FRACTION_FAKE = 1.00
HASH_SEED = 2025

TARGET_DB   = -26.0        # target RMS (dBFS-ish)
TRIM_THR_DB = -45          # silence threshold for trim
TRIM_DUR_S  = 0.20         # trim head/tail if >= this length

MAX_WORKERS = 6            # bump if stable
MANIFEST = META / "proc_manifest.csv"
BATCH_WRITE = 1000

REAL_CSV  = META / "master_real.csv"
FAKE_CSV  = META / "master_fake.csv"
ATT_CSV   = META / "attacks.labeled.csv"   # cleaned labels
assert REAL_CSV.exists() and FAKE_CSV.exists(), "Missing master_real.csv/master_fake.csv"

def log(s): print(s, flush=True)

def read_csv_safe(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path)
    for c in ["utt_id","path","duration","label","fake_type","source"]:
        if c not in df.columns:
            df[c] = "" if c != "duration" else 0.0
    return df

def stable_frac_mask(series: pd.Series, frac: float, seed: int) -> pd.Series:
    if frac >= 1.0: return pd.Series([True]*len(series), index=series.index)
    s = series.astype(str).apply(lambda p: int(hashlib.sha1((p+str(seed)).encode()).hexdigest()[:8], 16) / 0xFFFFFFFF)
    return s < frac

def out_path_for(in_path: str) -> Path:
    p = Path(in_path)
    try:    rel = p.relative_to(ROOT)
    except: rel = Path("_external") / p.name
    return OUT_ROOT / PROFILE / rel

def ensure_parent(p: Path): p.parent.mkdir(parents=True, exist_ok=True)

def rms_dbfs(p: Path) -> float:
    try:
        x, sr = sf.read(str(p), dtype="float32", always_2d=False)
        if x.ndim > 1: x = x.mean(axis=1)
        if len(x) == 0: return -120.0
        rms = float(np.sqrt(np.mean(np.square(x))))
        if rms <= 1e-9: return -120.0
        return 20.0*np.log10(np.clip(rms, 1e-9, 1.0))
    except Exception:
        return -120.0

def build_filter_chain(profile: str, gain_db: float) -> str:
    parts = ["highpass=f=20"]
    if profile in ("research_eq","strong"):
        parts += [
            "equalizer=f=3000:t=q:w=1.0:g=2.5",
            "equalizer=f=4800:t=q:w=0.9:g=2.0",
            "treble=g=1.0:f=6000:t=h:w=0.7"
        ]
    parts.append(f"volume={gain_db}dB")
    # portable args: start_duration/stop_duration
    parts.append(
        f"silenceremove=start_periods=1:start_duration={TRIM_DUR_S}:start_threshold={TRIM_THR_DB}dB:"
        f"stop_periods=1:stop_duration={TRIM_DUR_S}:stop_threshold={TRIM_THR_DB}dB"
    )
    return ",".join(parts)

def duration_s(p: Path) -> float:
    try:
        x, sr = sf.read(str(p), dtype="float32", always_2d=False)
        return len(x)/float(sr) if sr else 0.0
    except Exception:
        return 0.0

# -------- Load inputs --------
dfR = read_csv_safe(REAL_CSV)
dfF = read_csv_safe(FAKE_CSV)
dfs = [dfR, dfF]
if INCLUDE_ATTACKS and ATT_CSV.exists(): dfs.append(read_csv_safe(ATT_CSV))
df_all = pd.concat(dfs, ignore_index=True)

# keep only files that exist
df_all = df_all[df_all["path"].astype(str).apply(lambda p: Path(p).exists())].copy()

# normalize labels -> only {'real','fake'}
def normalize_label(lab, fake_type, src, path):
    l = str(lab).strip().lower()
    if l in {"real","fake"}: return l
    ft = str(fake_type).strip().lower()
    s  = str(src).strip().lower()
    p  = str(path)
    if "/raw/real_clean" in p or s.startswith("real_"): return "real"
    if ft in {"tts_edge","channel_attack","vc"} or "/raw/fake_" in p or s.startswith("fake_"): return "fake"
    return "fake"
df_all["label"] = [normalize_label(l, ft, s, p) for l, ft, s, p in
                   zip(df_all["label"], df_all["fake_type"], df_all["source"], df_all["path"])]

print("Input counts (normalized):")
print(df_all.groupby(df_all["label"].astype(str).str.lower()).size().to_string())

# deterministic per-label subset (here: everything)
parts = []
for lab, g in df_all.groupby(df_all["label"].astype(str).str.lower()):
    frac = FRACTION_REAL if lab == "real" else FRACTION_FAKE
    parts.append(g if frac>=1.0 else g[stable_frac_mask(g["path"], frac, HASH_SEED)])
df_in = pd.concat(parts, ignore_index=True) if parts else df_all.copy()

# -------- Manifest (resume) --------
MANIFEST.parent.mkdir(parents=True, exist_ok=True)
if MANIFEST.exists():
    mf = pd.read_csv(MANIFEST)
    done_set = set(mf.loc[mf["profile"]==PROFILE, "path_out"].astype(str))
else:
    with open(MANIFEST, "w", newline="", encoding="utf-8") as f:
        csv.writer(f).writerow(["utt_id","path_in","path_out","profile","duration","rms_db","peak_db"])
    done_set = set()

def already_done(path_in: str) -> bool:
    outp = out_path_for(path_in)
    return (str(outp) in done_set) or outp.exists()

# -------- Worker --------
def preprocess_one(row):
    pin = str(row["path"])
    pout = out_path_for(pin)
    if str(pout) in done_set or pout.exists():
        return None

    ensure_parent(pout)

    cur_db  = rms_dbfs(Path(pin))
    gain_db = float(np.clip(TARGET_DB - cur_db, -20.0, 20.0))
    FILTER  = build_filter_chain(PROFILE, gain_db)

    # temp in SAME dir to avoid cross-device link
    tmp = pout.parent / f".{pout.stem}.tmp-{uuid.uuid4().hex}.wav"
    try:
        cmd = [
            "ffmpeg","-nostdin","-hide_banner","-loglevel","error","-y",
            "-i", pin, "-ac","1","-ar","16000","-af", FILTER, "-sample_fmt","s16",
            str(tmp)
        ]
        p = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        if p.returncode != 0:
            try: tmp.unlink(missing_ok=True)
            except: pass
            return {"error": p.stderr.decode("utf-8","ignore")[:300], "path_in": pin}

        os.replace(str(tmp), str(pout))  # atomic within same FS

        # quick stats
        try:
            x, sr = sf.read(str(pout), dtype="float32", always_2d=False)
            if x.ndim > 1: x = x.mean(axis=1)
            dur = len(x)/float(sr) if sr else 0.0
            peak = float(np.max(np.abs(x))) if len(x) else 0.0
            rms  = float(np.sqrt(np.mean(np.square(x)))) if len(x) else 0.0
            rms_db = -120.0 if rms<=1e-9 else 20*np.log10(np.clip(rms, 1e-9, 1.0))
            peak_db = -120.0 if peak<=1e-9 else 20*np.log10(np.clip(peak, 1e-9, 1.0))
        except Exception:
            dur, rms_db, peak_db = 0.0, -120.0, -120.0

        return {"utt_id": row.get("utt_id",""), "path_in": pin, "path_out": str(pout),
                "profile": PROFILE, "duration": round(dur,3),
                "rms_db": round(rms_db,2), "peak_db": round(peak_db,2)}
    except Exception as e:
        try: tmp.unlink(missing_ok=True)
        except: pass
        return {"error": str(e), "path_in": pin}

# -------- Run --------
planned = len(df_in)
already = sum(1 for _ , r in df_in.iterrows() if already_done(str(r["path"])))
todo = planned - already
log(f"\n[preproc:{PROFILE}] PLANNED={planned:,} | ALREADY_DONE={already:,} | TO_DO={todo:,}")

created = 0; errs = 0
buf, errbuf = [], []
t0 = time.time()

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
    futs = [ex.submit(preprocess_one, r) for _, r in df_in.iterrows() if not already_done(str(r["path"]))]
    done_ct = 0
    for f in as_completed(futs):
        done_ct += 1
        res = f.result()
        if res is None:
            pass
        elif isinstance(res, dict) and "error" in res:
            errs += 1
            if len(errbuf) < 5: errbuf.append(res)  # capture first few errors for inspection
        else:
            buf.append(res)
            created += 1

        if done_ct % 500 == 0 or done_ct == todo:
            print(f"[{PROFILE}] {done_ct:,}/{todo:,}  (+{created} ok, {errs} err)  | elapsed {time.time()-t0:.1f}s", flush=True)

        if len(buf) >= BATCH_WRITE or (done_ct == todo and buf):
            with open(MANIFEST, "a", newline="", encoding="utf-8") as fcsv:
                w = csv.DictWriter(fcsv, fieldnames=["utt_id","path_in","path_out","profile","duration","rms_db","peak_db"])
                for row in buf: w.writerow(row)
            buf.clear()

print(f"[{PROFILE}] FINISHED: +{created} files, {errs} errors | total elapsed {time.time()-t0:.1f}s")
if errbuf:
    print("\n--- First few errors (trimmed) ---")
    for e in errbuf:
        print(e["path_in"], "->", e["error"])

# -------- Summary --------
def hours_from_manifest(profile: str):
    try:
        m = pd.read_csv(MANIFEST)
        m = m[m["profile"]==profile]
        return float(pd.to_numeric(m["duration"], errors="coerce").fillna(0).sum()/3600.0), len(m)
    except Exception:
        return 0.0, 0

h, n = hours_from_manifest(PROFILE)
print(f"[{PROFILE}] Manifest rows: {n:,} | hours: {h:.2f}h")

Input counts (normalized):
label
fake    45166
real    49623

[preproc:base] PLANNED=94,789 | ALREADY_DONE=20,774 | TO_DO=74,015
[base] 500/74,015  (+500 ok, 0 err)  | elapsed 116.6s
[base] 1,000/74,015  (+1000 ok, 0 err)  | elapsed 158.5s
[base] 1,500/74,015  (+1500 ok, 0 err)  | elapsed 222.7s
[base] 2,000/74,015  (+2000 ok, 0 err)  | elapsed 290.0s
[base] 2,500/74,015  (+2500 ok, 0 err)  | elapsed 358.5s
[base] 3,000/74,015  (+3000 ok, 0 err)  | elapsed 425.7s
[base] 3,500/74,015  (+3500 ok, 0 err)  | elapsed 489.8s
[base] 4,000/74,015  (+4000 ok, 0 err)  | elapsed 555.2s
[base] 4,500/74,015  (+4500 ok, 0 err)  | elapsed 618.7s
[base] 5,000/74,015  (+5000 ok, 0 err)  | elapsed 682.8s
[base] 5,500/74,015  (+5500 ok, 0 err)  | elapsed 746.5s
[base] 6,000/74,015  (+6000 ok, 0 err)  | elapsed 811.0s
[base] 6,500/74,015  (+6500 ok, 0 err)  | elapsed 875.9s
[base] 7,000/74,015  (+7000 ok, 0 err)  | elapsed 939.7s
[base] 7,500/74,015  (+7500 ok, 0 err)  | elapsed 1003.9s
[base] 8,000/74,01

audit and repair for base

In [ ]:
# ================= FAST PREPROC AUDIT + REPAIR (no full disk-hours scan) =================
from pathlib import Path
import pandas as pd, csv
import soundfile as sf

ROOT      = Path("/content/drive/MyDrive/hindi_dfake")
META      = ROOT / "metadata"
OUT_ROOT  = ROOT / "processed" / "wav"
MANIFEST  = META / "proc_manifest.csv"
PROFILE   = "base"   # change/add more profiles later if needed

def read_csv_safe(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path)
    for c in ["utt_id","path","duration","label","fake_type","source"]:
        if c not in df.columns:
            df[c] = "" if c != "duration" else 0.0
    return df

def out_path_for(profile: str, in_path: str) -> Path:
    p = Path(in_path)
    try:    rel = p.relative_to(ROOT)
    except: rel = Path("_external") / p.name
    return OUT_ROOT / profile / rel

def in_path_from_out(profile: str, out_path: Path) -> Path:
    base = OUT_ROOT / profile
    try:
        rel = out_path.relative_to(base)
        return ROOT / rel
    except Exception:
        return ROOT / "_unknown_source" / out_path.name

def fast_duration(path: Path) -> float:
    try:
        info = sf.info(str(path))
        if info.samplerate and info.frames:
            return float(info.frames) / float(info.samplerate)
    except Exception:
        pass
    return 0.0

# Universe of inputs that actually exist
dfs = [read_csv_safe(META/"master_real.csv"), read_csv_safe(META/"master_fake.csv")]
att = META/"attacks.labeled.csv"
if att.exists():
    dfs.append(read_csv_safe(att))
df_all = pd.concat(dfs, ignore_index=True)
df_all = df_all[df_all["path"].astype(str).apply(lambda p: Path(p).exists())].copy()

# Build lookup from input path -> row (for backfill metadata)
in_index = df_all.set_index("path").to_dict("index")

# Manifest slice
if MANIFEST.exists():
    mf = pd.read_csv(MANIFEST)
else:
    MANIFEST.parent.mkdir(parents=True, exist_ok=True)
    with open(MANIFEST, "w", newline="", encoding="utf-8") as f:
        csv.writer(f).writerow(["utt_id","path_in","path_out","profile","duration","rms_db","peak_db"])
    mf = pd.read_csv(MANIFEST)

mf_p = mf[mf["profile"] == PROFILE].copy()
set_mani = set(mf_p["path_out"].astype(str))

# On-disk outputs for this profile
disk_paths = list((OUT_ROOT / PROFILE).rglob("*.wav"))
set_disk = set(str(p) for p in disk_paths)

print(f"[counts] profile={PROFILE} | outputs_on_disk={len(set_disk):,} | manifest_rows={len(mf_p):,}")

missing_in_manifest = sorted(set_disk - set_mani)
print(f"[diff] on-disk but not in manifest: {len(missing_in_manifest):,}")

# ----- Backfill only the missing ones (fast) -----
if missing_in_manifest:
    rows = []
    for out_str in missing_in_manifest:
        outp = Path(out_str)
        in_p = in_path_from_out(PROFILE, outp)
        base = in_index.get(str(in_p), {})  # may be {}
        dur = fast_duration(outp)

        rows.append({
            "utt_id": base.get("utt_id",""),
            "path_in": str(in_p),
            "path_out": out_str,
            "profile": PROFILE,
            "duration": round(dur,3),
            "rms_db": "",   # unknown here (optional to compute later)
            "peak_db": "",  # unknown here
        })

    with open(MANIFEST, "a", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=["utt_id","path_in","path_out","profile","duration","rms_db","peak_db"])
        for r in rows: w.writerow(r)
    print(f"[fix] backfilled {len(rows):,} manifest rows.")

    # reload
    mf = pd.read_csv(MANIFEST)
    mf_p = mf[mf["profile"] == PROFILE].copy()

# ----- Dedup manifest by (profile, path_out) -----
before = len(mf)
mf = mf.drop_duplicates(subset=["profile","path_out"], keep="first").reset_index(drop=True)
after = len(mf)
if after != before:
    mf.to_csv(MANIFEST, index=False)
    print(f"[fix] dedup manifest: -{before-after} rows")

# Final counts
mf_p = mf[mf["profile"] == PROFILE].copy()
print(f"[post] manifest_rows={len(mf_p):,}")
print("[ok] FAST audit/repair done.")

[counts] profile=base | outputs_on_disk=94,789 | manifest_rows=94,789
[diff] on-disk but not in manifest: 0
[post] manifest_rows=94,789
[ok] FAST audit/repair done.


mini test for Strong

In [ ]:
# ===================== STRONG mini-test (EQ + RawBoost v3) =====================
# Processes a small, deterministic sample first (default: 20 real + 20 fake)

import os, csv, uuid, time, hashlib, subprocess
from pathlib import Path
import pandas as pd, numpy as np, soundfile as sf
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.auto import tqdm

# -------- CONFIG --------
ROOT = Path("/content/drive/MyDrive/hindi_dfake")
META = ROOT / "metadata"
OUT_ROOT = ROOT / "processed" / "wav"
PROFILE = "strong"                   # strong = base + artifact EQ + RawBoost v3
INCLUDE_ATTACKS = True
SAMPLE_REAL = 20                     # how many REAL to process in this test
SAMPLE_FAKE = 20                     # how many FAKE to process in this test
MAX_WORKERS = 4
BATCH_WRITE = 100
TARGET_DB   = -26.0
TRIM_THR_DB = -45
TRIM_DUR_S  = 0.20
MANIFEST = META / "proc_manifest.csv"

# RawBoost v3–style knobs
RB_ENABLE           = True
RB_BANDPASS_HZ      = (2500, 6000)
RB_SNR_DB           = 25.0
RB_NOISE_PROB       = 1.0
RB_IMPULSE_PROB     = 0.5
RB_IMPULSES_PER_SEC = 1.0
RB_IMPULSE_GAIN     = 0.08
RB_REVERB_PROB      = 0.6
RB_REVERB_T_SEC     = 0.03
RB_REVERB_DECAY     = 0.35
RB_REVERB_WET       = 0.18

# -------- helpers --------
def read_csv_safe(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path)
    for c in ["utt_id","path","duration","label","fake_type","source"]:
        if c not in df.columns:
            df[c] = "" if c != "duration" else 0.0
    return df

def normalize_label(lab, fake_type, src, path):
    l = str(lab).strip().lower()
    if l in {"real","fake"}: return l
    ft = str(fake_type).strip().lower()
    s  = str(src).strip().lower()
    p  = str(path)
    if "/raw/real_clean" in p or s.startswith("real_"): return "real"
    if ft in {"tts_edge","channel_attack","vc"} or "/raw/fake_" in p or s.startswith("fake_"): return "fake"
    return "fake"

def out_path_for(in_path: str) -> Path:
    p = Path(in_path)
    try:    rel = p.relative_to(ROOT)
    except: rel = Path("_external") / p.name
    return OUT_ROOT / PROFILE / rel

def ensure_parent(p: Path): p.parent.mkdir(parents=True, exist_ok=True)

def rms_dbfs_arr(x: np.ndarray) -> float:
    if x.ndim > 1: x = x.mean(axis=1)
    if len(x) == 0: return -120.0
    rms = float(np.sqrt(np.mean(np.square(x))))
    if rms <= 1e-9: return -120.0
    return 20.0*np.log10(np.clip(rms, 1e-9, 1.0))

# RawBoost v3–style (deterministic)
def _rng_from_key(key: str):
    seed = int(hashlib.sha1(key.encode("utf-8")).hexdigest()[:8], 16)
    return np.random.default_rng(seed)

def _add_colored_noise(x, sr, rng, band=(2500,6000), snr_db=25.0):
    n = len(x)
    if n == 0: return x
    wn = rng.standard_normal(n).astype(np.float32)
    X = np.fft.rfft(wn)
    freqs = np.fft.rfftfreq(n, d=1.0/sr)
    mask = (freqs >= band[0]) & (freqs <= band[1])
    X[~mask] = 0.0
    noise = np.fft.irfft(X, n=n).astype(np.float32)
    sig_rms = np.sqrt(np.mean(np.square(x))) + 1e-9
    target_noise_rms = sig_rms / (10.0**(snr_db/20.0))
    cur_rms = np.sqrt(np.mean(np.square(noise))) + 1e-12
    noise *= (target_noise_rms / cur_rms)
    return np.clip(x + noise, -1.0, 1.0)

def _add_impulses(x, sr, rng, per_sec=1.0, gain=0.08):
    n = len(x)
    if n == 0: return x
    dur = n/float(sr)
    k = max(1, int(per_sec * dur))
    idx = rng.integers(0, n, size=k)
    amp = gain * (np.sqrt(np.mean(np.square(x))) + 1e-9)
    y = x.copy()
    y[idx] = np.clip(y[idx] + amp * rng.choice([-1.0, 1.0], size=k), -1.0, 1.0)
    return y

def _add_small_reverb(x, sr, rng, t_sec=0.03, decay=0.35, wet=0.18):
    n = len(x)
    if n == 0: return x
    ir_len = max(8, int(t_sec * sr))
    t = np.arange(ir_len, dtype=np.float32)
    ir = np.exp(-decay * t / ir_len).astype(np.float32)
    for _ in range(3):
        pos = int(rng.integers(0, ir_len))
        ir[pos] += float(rng.uniform(0.1, 0.3))
    ir /= (np.sum(np.abs(ir)) + 1e-9)
    y = np.convolve(x, ir, mode="full")[:n].astype(np.float32)
    return np.clip((1.0 - wet) * x + wet * y, -1.0, 1.0)

def rawboost_v3(x: np.ndarray, sr: int, key: str) -> np.ndarray:
    if x.ndim > 1: x = x.mean(axis=1)
    x = np.clip(x, -1.0, 1.0).astype(np.float32)
    if not RB_ENABLE: return x
    rng = _rng_from_key(key)
    if rng.uniform() < RB_NOISE_PROB:
        x = _add_colored_noise(x, sr, rng, RB_BANDPASS_HZ, RB_SNR_DB)
    if rng.uniform() < RB_IMPULSE_PROB:
        x = _add_impulses(x, sr, rng, RB_IMPULSES_PER_SEC, RB_IMPULSE_GAIN)
    if rng.uniform() < RB_REVERB_PROB:
        x = _add_small_reverb(x, sr, rng, RB_REVERB_T_SEC, RB_REVERB_DECAY, RB_REVERB_WET)
    return x

def build_filter_chain_strong(gain_db: float) -> str:
    parts = [
        "highpass=f=20",
        "equalizer=f=3000:t=q:w=1.0:g=2.5",
        "equalizer=f=4800:t=q:w=0.9:g=2.0",
        "treble=g=1.0:f=6000:t=h:w=0.7",
        f"volume={gain_db}dB",
        f"silenceremove=start_periods=1:start_duration={TRIM_DUR_S}:start_threshold={TRIM_THR_DB}dB:stop_periods=1:stop_duration={TRIM_DUR_S}:stop_threshold={TRIM_THR_DB}dB"
    ]
    return ",".join(parts)

# -------- inputs: build small deterministic sample --------
dfR = read_csv_safe(META/"master_real.csv")
dfF = read_csv_safe(META/"master_fake.csv")
dfs = [dfR, dfF]
if INCLUDE_ATTACKS and (META/"attacks.labeled.csv").exists():
    dfs.append(read_csv_safe(META/"attacks.labeled.csv"))
df_all = pd.concat(dfs, ignore_index=True)
df_all = df_all[df_all["path"].astype(str).apply(lambda p: Path(p).exists())].copy()
df_all["label"] = [normalize_label(l, ft, s, p) for l, ft, s, p in
                   zip(df_all["label"], df_all["fake_type"], df_all["source"], df_all["path"])]

# stable selection via sha1 on path
def pick_k(df, k):
    if k <= 0: return df.iloc[0:0]
    tmp = df.assign(_key=df["path"].astype(str).apply(lambda p: hashlib.sha1(p.encode()).hexdigest()))
    return tmp.sort_values("_key").head(k).drop(columns="_key")

df_real = pick_k(df_all[df_all["label"]=="real"], SAMPLE_REAL)
df_fake = pick_k(df_all[df_all["label"]=="fake"], SAMPLE_FAKE)
df_in   = pd.concat([df_real, df_fake], ignore_index=True)

print(f"Mini-test selection: real={len(df_real)} | fake={len(df_fake)} | total={len(df_in)}")

# -------- manifest init & skip logic --------
MANIFEST.parent.mkdir(parents=True, exist_ok=True)
if MANIFEST.exists():
    mf = pd.read_csv(MANIFEST, usecols=["profile","path_out"])
    done_set = set(mf.loc[mf["profile"]==PROFILE, "path_out"].astype(str))
else:
    with open(MANIFEST, "w", newline="", encoding="utf-8") as f:
        csv.writer(f).writerow(["utt_id","path_in","path_out","profile","duration","rms_db","peak_db"])
    done_set = set()

def already_done(pin: str) -> bool:
    pout = out_path_for(pin)
    return pout.exists() or (str(pout) in done_set)

todo_rows = [r for _, r in df_in.iterrows() if not already_done(str(r["path"]))]

# -------- worker --------
def preprocess_one(row):
    pin = str(row["path"])
    pout = out_path_for(pin)
    ensure_parent(pout)

    # read / resample if needed
    try:
        x, sr = sf.read(pin, dtype="float32", always_2d=False)
    except Exception as e:
        return {"error": f"read_fail: {e}", "path_in": pin}
    if x.ndim > 1: x = x.mean(axis=1)
    if sr != 16000:
        tmp = pout.parent / f".{pout.stem}.in-{uuid.uuid4().hex}.wav"
        cmd = ["ffmpeg","-nostdin","-hide_banner","-loglevel","error","-y",
               "-i", pin, "-ac","1","-ar","16000","-sample_fmt","s16", str(tmp)]
        p = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        if p.returncode != 0:
            try: tmp.unlink(missing_ok=True)
            except: pass
            return {"error": "resample_fail", "path_in": pin}
        x, sr = sf.read(str(tmp), dtype="float32", always_2d=False)
        try: tmp.unlink(missing_ok=True)
        except: pass

    x = np.clip(x, -1.0, 1.0).astype(np.float32)

    # RawBoost v3–style
    key = f"{pin}|{PROFILE}"
    try:
        y = rawboost_v3(x, sr, key)
    except Exception:
        y = x

    # write RB temp near target (avoid cross-device link)
    tmp_rb = pout.parent / f".{pout.stem}.rb-{uuid.uuid4().hex}.wav"
    sf.write(str(tmp_rb), y, 16000, subtype="PCM_16")

    # per-file gain then ffmpeg EQ + trim
    gain_db = float(np.clip(TARGET_DB - rms_dbfs_arr(y), -20.0, 20.0))
    FILTER  = build_filter_chain_strong(gain_db)

    tmp_out = pout.parent / f".{pout.stem}.tmp-{uuid.uuid4().hex}.wav"
    cmd = ["ffmpeg","-nostdin","-hide_banner","-loglevel","error","-y",
           "-i", str(tmp_rb), "-ac","1","-ar","16000",
           "-af", FILTER, "-sample_fmt","s16", str(tmp_out)]
    p = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    try: tmp_rb.unlink(missing_ok=True)
    except: pass
    if p.returncode != 0:
        try: tmp_out.unlink(missing_ok=True)
        except: pass
        return {"error": p.stderr.decode("utf-8","ignore")[:300], "path_in": pin}

    os.replace(str(tmp_out), str(pout))

    # stats
    try:
        z, sr2 = sf.read(str(pout), dtype="float32", always_2d=False)
        if z.ndim > 1: z = z.mean(axis=1)
        dur = len(z)/float(sr2) if sr2 else 0.0
        peak = float(np.max(np.abs(z))) if len(z) else 0.0
        rms  = float(np.sqrt(np.mean(np.square(z)))) if len(z) else 0.0
        rms_db = -120.0 if rms<=1e-9 else 20*np.log10(np.clip(rms,1e-9,1.0))
        peak_db = -120.0 if peak<=1e-9 else 20*np.log10(np.clip(peak,1e-9,1.0))
    except Exception:
        dur, rms_db, peak_db = 0.0, -120.0, -120.0

    return {"utt_id": row.get("utt_id",""), "path_in": pin, "path_out": str(pout),
            "profile": PROFILE, "duration": round(dur,3),
            "rms_db": round(rms_db,2), "peak_db": round(peak_db,2)}

# -------- run --------
print(f"Processing {len(todo_rows)} files for profile='{PROFILE}' …")
created, errs = 0, 0
buf = []
pbar = tqdm(total=len(todo_rows), unit="file", desc=f"strong-mini")

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
    futs = [ex.submit(preprocess_one, r) for r in todo_rows]
    for f in as_completed(futs):
        res = f.result()
        pbar.update(1)
        if isinstance(res, dict) and "error" in res:
            errs += 1
        elif res:
            buf.append(res)
            created += 1
            if len(buf) >= BATCH_WRITE:
                with open(MANIFEST, "a", newline="", encoding="utf-8") as fcsv:
                    w = csv.DictWriter(fcsv, fieldnames=["utt_id","path_in","path_out","profile","duration","rms_db","peak_db"])
                    for r in buf: w.writerow(r)
                buf.clear()

pbar.close()
# final flush
if buf:
    with open(MANIFEST, "a", newline="", encoding="utf-8") as fcsv:
        w = csv.DictWriter(fcsv, fieldnames=["utt_id","path_in","path_out","profile","duration","rms_db","peak_db"])
        for r in buf: w.writerow(r)

print(f"Mini-test done. created={created}, errors={errs}")

# show a few outputs
outs = sorted([str(p) for p in (OUT_ROOT/PROFILE).rglob("*.wav")])[:10]
print("Sample outputs:")
for p in outs[:10]:
    print("  ", p)

# quick manifest slice check
if MANIFEST.exists():
    m = pd.read_csv(MANIFEST)
    mp = m[m["profile"]==PROFILE]
    print(f"Manifest rows for '{PROFILE}':", len(mp))

Mini-test selection: real=20 | fake=20 | total=40
Processing 40 files for profile='strong' …


strong-mini:   0%|          | 0/40 [00:00<?, ?file/s]

Mini-test done. created=40, errors=0
Sample outputs:
   /content/drive/MyDrive/hindi_dfake/processed/wav/strong/processed/attacks/lp8k/2e019fe4f6183cea.lp8k.wav
   /content/drive/MyDrive/hindi_dfake/processed/wav/strong/processed/attacks/lp8k/40bd5448d145bb5d.lp8k.wav
   /content/drive/MyDrive/hindi_dfake/processed/wav/strong/processed/attacks/lp8k/4d542941e4246394.lp8k.wav
   /content/drive/MyDrive/hindi_dfake/processed/wav/strong/processed/attacks/lp8k/69d1d4bfcef314a9.lp8k.wav
   /content/drive/MyDrive/hindi_dfake/processed/wav/strong/processed/attacks/lp8k/6a2bc759d7d01e67.lp8k.wav
   /content/drive/MyDrive/hindi_dfake/processed/wav/strong/processed/attacks/lp8k/928373bb82b00d5b.lp8k.wav
   /content/drive/MyDrive/hindi_dfake/processed/wav/strong/processed/attacks/lp8k/tts_edge_17d4982821626b81.lp8k.wav
   /content/drive/MyDrive/hindi_dfake/processed/wav/strong/processed/attacks/lp8k/tts_edge_4e0b8d175798551f.lp8k.wav
   /content/drive/MyDrive/hindi_dfake/processed/wav/strong/proces

Strong

In [ ]:
# ================= PREPROCESSING — STRONG (EQ + RawBoost v3) =================
# Writes: /content/drive/MyDrive/hindi_dfake/processed/wav/strong/...
# Logs to: /content/drive/MyDrive/hindi_dfake/metadata/proc_manifest.csv (profile=strong)

import os, csv, uuid, time, hashlib, subprocess
from pathlib import Path
import pandas as pd, numpy as np, soundfile as sf
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.auto import tqdm

# ------------ CONFIG ------------
ROOT = Path("/content/drive/MyDrive/hindi_dfake")
META = ROOT / "metadata"
OUT_ROOT = ROOT / "processed" / "wav"

PROFILE = "strong"          # strong = base + artifact EQ + RawBoost v3
INCLUDE_ATTACKS = True
FRACTION_REAL = 1.00        # process all
FRACTION_FAKE = 1.00
HASH_SEED = 2025

# Loudness & trim
TARGET_DB   = -26.0
TRIM_THR_DB = -45
TRIM_DUR_S  = 0.20

# Concurrency / batching
MAX_WORKERS   = 6           # 6–8 is reasonable on Colab CPU
INFLIGHT_MULT = 3
BATCH_WRITE   = 500

# RawBoost v3–style knobs
RB_ENABLE           = True
RB_BANDPASS_HZ      = (2500, 6000)
RB_SNR_DB           = 25.0
RB_NOISE_PROB       = 1.0
RB_IMPULSE_PROB     = 0.5
RB_IMPULSES_PER_SEC = 1.0
RB_IMPULSE_GAIN     = 0.08
RB_REVERB_PROB      = 0.6
RB_REVERB_T_SEC     = 0.03
RB_REVERB_DECAY     = 0.35
RB_REVERB_WET       = 0.18

REAL_CSV  = META / "master_real.csv"
FAKE_CSV  = META / "master_fake.csv"
ATT_CSV   = META / "attacks.labeled.csv"
MANIFEST  = META / "proc_manifest.csv"
assert REAL_CSV.exists() and FAKE_CSV.exists()

# ------------ helpers ------------
def read_csv_safe(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path)
    for c in ["utt_id","path","duration","label","fake_type","source"]:
        if c not in df.columns:
            df[c] = "" if c != "duration" else 0.0
    return df

def normalize_label(lab, fake_type, src, path):
    l = str(lab).strip().lower()
    if l in {"real","fake"}: return l
    ft = str(fake_type).strip().lower()
    s  = str(src).strip().lower()
    p  = str(path)
    if "/raw/real_clean" in p or s.startswith("real_"): return "real"
    if ft in {"tts_edge","channel_attack","vc"} or "/raw/fake_" in p or s.startswith("fake_"): return "fake"
    return "fake"

def stable_frac_mask(series: pd.Series, frac: float, seed: int) -> pd.Series:
    if frac >= 1.0: return pd.Series([True]*len(series), index=series.index)
    s = series.astype(str).apply(lambda p: int(hashlib.sha1((p+str(seed)).encode()).hexdigest()[:8], 16) / 0xFFFFFFFF)
    return s < frac

def out_path_for(in_path: str) -> Path:
    p = Path(in_path)
    try:    rel = p.relative_to(ROOT)
    except: rel = Path("_external") / p.name
    return OUT_ROOT / PROFILE / rel

def ensure_parent(p: Path): p.parent.mkdir(parents=True, exist_ok=True)

def rms_dbfs_arr(x: np.ndarray) -> float:
    if x.ndim > 1: x = x.mean(axis=1)
    if len(x) == 0: return -120.0
    rms = float(np.sqrt(np.mean(np.square(x))))
    if rms <= 1e-9: return -120.0
    return 20.0*np.log10(np.clip(rms, 1e-9, 1.0))

# ---- RawBoost v3–style (deterministic per file) ----
def _rng_from_key(key: str):
    seed = int(hashlib.sha1(key.encode("utf-8")).hexdigest()[:8], 16)
    return np.random.default_rng(seed)

def _add_colored_noise(x, sr, rng, band=(2500,6000), snr_db=25.0):
    n = len(x)
    if n == 0: return x
    wn = rng.standard_normal(n).astype(np.float32)
    X = np.fft.rfft(wn); freqs = np.fft.rfftfreq(n, d=1.0/sr)
    mask = (freqs >= band[0]) & (freqs <= band[1]); X[~mask] = 0.0
    noise = np.fft.irfft(X, n=n).astype(np.float32)
    sig_rms = np.sqrt(np.mean(np.square(x))) + 1e-9
    target_noise_rms = sig_rms / (10.0**(snr_db/20.0))
    cur_rms = np.sqrt(np.mean(np.square(noise))) + 1e-12
    noise *= (target_noise_rms / cur_rms)
    return np.clip(x + noise, -1.0, 1.0)

def _add_impulses(x, sr, rng, per_sec=1.0, gain=0.08):
    n = len(x)
    if n == 0: return x
    dur = n/float(sr); k = max(1, int(per_sec * dur))
    idx = rng.integers(0, n, size=k)
    amp = gain * (np.sqrt(np.mean(np.square(x))) + 1e-9)
    y = x.copy()
    y[idx] = np.clip(y[idx] + amp * rng.choice([-1.0, 1.0], size=k), -1.0, 1.0)
    return y

def _add_small_reverb(x, sr, rng, t_sec=0.03, decay=0.35, wet=0.18):
    n = len(x)
    if n == 0: return x
    ir_len = max(8, int(t_sec * sr))
    t = np.arange(ir_len, dtype=np.float32)
    ir = np.exp(-decay * t / ir_len).astype(np.float32)
    for _ in range(3):
        pos = int(rng.integers(0, ir_len))
        ir[pos] += float(rng.uniform(0.1, 0.3))
    ir /= (np.sum(np.abs(ir)) + 1e-9)
    y = np.convolve(x, ir, mode="full")[:n].astype(np.float32)
    return np.clip((1.0 - wet) * x + wet * y, -1.0, 1.0)

def rawboost_v3(x: np.ndarray, sr: int, key: str) -> np.ndarray:
    if x.ndim > 1: x = x.mean(axis=1)
    x = np.clip(x, -1.0, 1.0).astype(np.float32)
    if not RB_ENABLE: return x
    rng = _rng_from_key(key)
    if rng.uniform() < RB_NOISE_PROB:
        x = _add_colored_noise(x, sr, rng, RB_BANDPASS_HZ, RB_SNR_DB)
    if rng.uniform() < RB_IMPULSE_PROB:
        x = _add_impulses(x, sr, rng, RB_IMPULSES_PER_SEC, RB_IMPULSE_GAIN)
    if rng.uniform() < RB_REVERB_PROB:
        x = _add_small_reverb(x, sr, rng, RB_REVERB_T_SEC, RB_REVERB_DECAY, RB_REVERB_WET)
    return x

def build_filter_chain_strong(gain_db: float) -> str:
    return ",".join([
        "highpass=f=20",
        "equalizer=f=3000:t=q:w=1.0:g=2.5",
        "equalizer=f=4800:t=q:w=0.9:g=2.0",
        "treble=g=1.0:f=6000:t=h:w=0.7",
        f"volume={gain_db}dB",
        f"silenceremove=start_periods=1:start_duration={TRIM_DUR_S}:start_threshold={TRIM_THR_DB}dB:stop_periods=1:stop_duration={TRIM_DUR_S}:stop_threshold={TRIM_THR_DB}dB"
    ])

# ------------ load inputs ------------
dfR = read_csv_safe(REAL_CSV)
dfF = read_csv_safe(FAKE_CSV)
dfs = [dfR, dfF]
if INCLUDE_ATTACKS and ATT_CSV.exists():
    dfs.append(read_csv_safe(ATT_CSV))
df_all = pd.concat(dfs, ignore_index=True)
df_all = df_all[df_all["path"].astype(str).apply(lambda p: Path(p).exists())].copy()
df_all["label"] = [normalize_label(l, ft, s, p) for l, ft, s, p in
                   zip(df_all["label"], df_all["fake_type"], df_all["source"], df_all["path"])]

# stratified deterministic subset (here: all)
parts = []
for lab, g in df_all.groupby(df_all["label"].astype(str).str.lower()):
    frac = FRACTION_REAL if lab == "real" else FRACTION_FAKE
    parts.append(g if frac>=1.0 else g[stable_frac_mask(g["path"], frac, HASH_SEED)])
df_in = pd.concat(parts, ignore_index=True) if parts else df_all.copy()

# ------------ manifest skip logic ------------
MANIFEST.parent.mkdir(parents=True, exist_ok=True)
if MANIFEST.exists():
    mf = pd.read_csv(MANIFEST, usecols=["profile","path_out"])
    done_set = set(mf.loc[mf["profile"]==PROFILE, "path_out"].astype(str))
else:
    with open(MANIFEST, "w", newline="", encoding="utf-8") as f:
        csv.writer(f).writerow(["utt_id","path_in","path_out","profile","duration","rms_db","peak_db"])
    done_set = set()

def already_done(pin: str) -> bool:
    pout = out_path_for(pin)
    return pout.exists() or (str(pout) in done_set)

todo_rows = [r for _, r in df_in.iterrows() if not already_done(str(r["path"]))]
planned, todo = len(df_in), len(todo_rows)
print(f"Input counts (normalized):")
print(df_all.groupby(df_all['label']).size().to_string())
print(f"\n[preproc:{PROFILE}] PLANNED={planned:,} | ALREADY_DONE={planned-todo:,} | TO_DO={todo:,}")

# ------------ worker ------------
def preprocess_one(row):
    pin = str(row["path"])
    pout = out_path_for(pin)
    ensure_parent(pout)

    # read (and resample if needed)
    try:
        x, sr = sf.read(pin, dtype="float32", always_2d=False)
    except Exception as e:
        return {"error": f"read_fail: {e}", "path_in": pin}
    if x.ndim > 1: x = x.mean(axis=1)
    if sr != 16000:
        tmp = pout.parent / f".{pout.stem}.in-{uuid.uuid4().hex}.wav"
        cmd = ["ffmpeg","-nostdin","-hide_banner","-loglevel","error","-y",
               "-i", pin, "-ac","1","-ar","16000","-sample_fmt","s16", str(tmp)]
        p = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        if p.returncode != 0:
            try: tmp.unlink(missing_ok=True)
            except: pass
            return {"error": "resample_fail", "path_in": pin}
        x, sr = sf.read(str(tmp), dtype="float32", always_2d=False)
        try: tmp.unlink(missing_ok=True)
        except: pass

    x = np.clip(x, -1.0, 1.0).astype(np.float32)

    # RawBoost
    key = f"{pin}|{PROFILE}"
    try:    y = rawboost_v3(x, sr, key)
    except: y = x

    # write RB temp near target
    tmp_rb = pout.parent / f".{pout.stem}.rb-{uuid.uuid4().hex}.wav"
    sf.write(str(tmp_rb), y, 16000, subtype="PCM_16")

    # EQ + loudness + trim via ffmpeg
    gain_db = float(np.clip(TARGET_DB - rms_dbfs_arr(y), -20.0, 20.0))
    FILTER  = build_filter_chain_strong(gain_db)

    tmp_out = pout.parent / f".{pout.stem}.tmp-{uuid.uuid4().hex}.wav"
    cmd = ["ffmpeg","-nostdin","-hide_banner","-loglevel","error","-y",
           "-i", str(tmp_rb), "-ac","1","-ar","16000",
           "-af", FILTER, "-sample_fmt","s16", str(tmp_out)]
    p = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    try: tmp_rb.unlink(missing_ok=True)
    except: pass
    if p.returncode != 0:
        try: tmp_out.unlink(missing_ok=True)
        except: pass
        return {"error": p.stderr.decode("utf-8","ignore")[:300], "path_in": pin}

    os.replace(str(tmp_out), str(pout))

    # stats
    try:
        z, sr2 = sf.read(str(pout), dtype="float32", always_2d=False)
        if z.ndim > 1: z = z.mean(axis=1)
        dur = len(z)/float(sr2) if sr2 else 0.0
        peak = float(np.max(np.abs(z))) if len(z) else 0.0
        rms  = float(np.sqrt(np.mean(np.square(z)))) if len(z) else 0.0
        rms_db = -120.0 if rms<=1e-9 else 20*np.log10(np.clip(rms,1e-9,1.0))
        peak_db = -120.0 if peak<=1e-9 else 20*np.log10(np.clip(peak,1e-9,1.0))
    except Exception:
        dur, rms_db, peak_db = 0.0, -120.0, -120.0

    return {"utt_id": row.get("utt_id",""), "path_in": pin, "path_out": str(pout),
            "profile": PROFILE, "duration": round(dur,3),
            "rms_db": round(rms_db,2), "peak_db": round(peak_db,2)}

# ------------ streaming scheduler with progress ------------
created = 0; errs = 0; buf = []
start = time.time()
inflight_cap = max(MAX_WORKERS * INFLIGHT_MULT, MAX_WORKERS)
pbar = tqdm(total=len(todo_rows), desc=f"preproc:{PROFILE}", unit="file")

def flush_manifest(rows):
    if not rows: return
    with open(MANIFEST, "a", newline="", encoding="utf-8") as fcsv:
        w = csv.DictWriter(fcsv, fieldnames=["utt_id","path_in","path_out","profile","duration","rms_db","peak_db"])
        for r in rows: w.writerow(r)

it = iter(todo_rows)
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
    inflight = set()
    for _ in range(min(inflight_cap, len(todo_rows))):
        try: inflight.add(ex.submit(preprocess_one, next(it)))
        except StopIteration: break
    while inflight:
        done = next(as_completed(inflight))
        inflight.remove(done)
        res = done.result()
        pbar.update(1)
        if isinstance(res, dict) and "error" in res:
            errs += 1
        elif res:
            buf.append(res); created += 1
            if len(buf) >= BATCH_WRITE:
                flush_manifest(buf); buf.clear()
        try: inflight.add(ex.submit(preprocess_one, next(it)))
        except StopIteration: pass

flush_manifest(buf)
pbar.close()
print(f"\n[{PROFILE}] FINISHED: +{created} files, {errs} errors | elapsed {(time.time()-start)/60:.1f} min")

# quick manifest slice
if MANIFEST.exists():
    m = pd.read_csv(MANIFEST)
    mp = m[m["profile"]==PROFILE]
    hrs = float(pd.to_numeric(mp["duration"], errors="coerce").fillna(0).sum()/3600.0)
    print(f"[{PROFILE}] Manifest rows: {len(mp):,} | hours: {hrs:.2f} h")

Input counts (normalized):
label
fake    45166
real    49623

[preproc:strong] PLANNED=94,789 | ALREADY_DONE=40 | TO_DO=94,749


preproc:strong:   0%|          | 0/94749 [00:00<?, ?file/s]


[strong] FINISHED: +94749 files, 0 errors | elapsed 270.4 min
[strong] Manifest rows: 94,789 | hours: 181.31 h


Strong audit


In [ ]:
# ===================== STRONG: one-shot sanity audit =====================
from pathlib import Path
import pandas as pd, csv, random
import soundfile as sf

ROOT      = Path("/content/drive/MyDrive/hindi_dfake")
META      = ROOT / "metadata"
OUT_ROOT  = ROOT / "processed" / "wav"
PROFILE   = "strong"

REAL_CSV  = META / "master_real.csv"
FAKE_CSV  = META / "master_fake.csv"
ATT_CSV   = META / "attacks.labeled.csv"
MANIFEST  = META / "proc_manifest.csv"

def read_csv_safe(p):
    df = pd.read_csv(p)
    for c in ["utt_id","path","duration","label","fake_type","source"]:
        if c not in df.columns:
            df[c] = "" if c != "duration" else 0.0
    return df

def normalize_label(lab, fake_type, src, path):
    l = str(lab).strip().lower()
    if l in {"real","fake"}: return l
    ft = str(fake_type).strip().lower()
    s  = str(src).strip().lower()
    p  = str(path)
    if "/raw/real_clean" in p or s.startswith("real_"): return "real"
    if ft in {"tts_edge","channel_attack","vc"} or "/raw/fake_" in p or s.startswith("fake_"): return "fake"
    return "fake"

def out_path_for(in_path: str) -> Path:
    p = Path(in_path)
    try:
        rel = p.relative_to(ROOT)
    except Exception:
        rel = Path("_external") / p.name
    return OUT_ROOT / PROFILE / rel

# ---------- load inputs (only those that exist) ----------
dfs = [read_csv_safe(REAL_CSV), read_csv_safe(FAKE_CSV)]
if ATT_CSV.exists():
    dfs.append(read_csv_safe(ATT_CSV))
df_all = pd.concat(dfs, ignore_index=True)
df_all = df_all[df_all["path"].astype(str).apply(lambda p: Path(p).exists())].copy()
df_all["label"] = [normalize_label(l, ft, s, p) for l, ft, s, p in
                   zip(df_all["label"], df_all["fake_type"], df_all["source"], df_all["path"])]

expected = set(str(out_path_for(p)) for p in df_all["path"].astype(str))

# ---------- what’s on disk & in manifest ----------
on_disk  = set(str(p) for p in (OUT_ROOT/PROFILE).rglob("*.wav"))
if MANIFEST.exists():
    mf = pd.read_csv(MANIFEST)
else:
    MANIFEST.parent.mkdir(parents=True, exist_ok=True)
    with open(MANIFEST, "w", newline="", encoding="utf-8") as f:
        csv.writer(f).writerow(["utt_id","path_in","path_out","profile","duration","rms_db","peak_db"])
    mf = pd.read_csv(MANIFEST)

mf_p     = mf[mf["profile"]==PROFILE]
in_mani  = set(mf_p["path_out"].astype(str))

print("[Universe] inputs that exist:", len(df_all))
print(f"[counts:{PROFILE}] expected={len(expected):,} | on_disk={len(on_disk):,} | manifest_rows={len(in_mani):,}")

miss_disk   = expected - on_disk
extra_disk  = on_disk - expected
miss_mani   = on_disk  - in_mani
extra_mani  = in_mani  - on_disk

print(f"[diff] expected but missing on disk: {len(miss_disk):,}")
print(f"[diff] on disk but missing in manifest: {len(miss_mani):,}")
print(f"[diff] manifest rows without file: {len(extra_mani):,}")

# ---------- backfill manifest if needed ----------
def fast_duration(p):
    try:
        info = sf.info(p)
        return round(info.frames / info.samplerate, 3)
    except: return 0.0

if len(miss_mani) > 0:
    print(f"[fix] backfilling {len(miss_mani):,} manifest rows …")
    rows = []
    # rough reconstruction of input path by mirroring
    for out_str in list(miss_mani):
        outp = Path(out_str)
        try:
            rel = outp.relative_to(OUT_ROOT/PROFILE)
            in_p = ROOT / rel
        except Exception:
            in_p = ROOT / "_unknown_source" / outp.name
        rows.append({
            "utt_id": "", "path_in": str(in_p), "path_out": out_str,
            "profile": PROFILE, "duration": fast_duration(str(outp)),
            "rms_db": "", "peak_db": ""
        })
    with open(MANIFEST, "a", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=["utt_id","path_in","path_out","profile","duration","rms_db","peak_db"])
        for r in rows: w.writerow(r)
    print("[fix] backfill done.")

# ---------- dedup manifest on (profile, path_out) ----------
mf2 = pd.read_csv(MANIFEST)
before = len(mf2)
mf2 = mf2.drop_duplicates(subset=["profile","path_out"], keep="first").reset_index(drop=True)
if len(mf2) != before:
    mf2.to_csv(MANIFEST, index=False)
    print(f"[fix] dedup manifest: -{before - len(mf2)} rows")

# ---------- spot-check audio format on a random sample ----------
sample_n = min(300, len(on_disk))
sample_paths = random.sample(list(on_disk), sample_n) if sample_n > 0 else []
sr_counts, ch_counts, subtype_counts, bad = {}, {}, {}, 0
for p in sample_paths:
    try:
        info = sf.info(p)
        sr_counts[info.samplerate] = sr_counts.get(info.samplerate, 0) + 1
        ch_counts[info.channels]   = ch_counts.get(info.channels,   0) + 1
        subtype_counts[info.subtype] = subtype_counts.get(info.subtype, 0) + 1
    except Exception:
        bad += 1

print(f"[probe] sample={sample_n} | sr={sr_counts} | ch={ch_counts} | subtype={subtype_counts} | bad={bad}")

# ---------- hours from manifest (profile=strong) ----------
mf = pd.read_csv(MANIFEST)
ms = mf[mf["profile"]==PROFILE]
hours = float(pd.to_numeric(ms["duration"], errors="coerce").fillna(0).sum()/3600.0)
print(f"[hours:{PROFILE}] rows={len(ms):,} | hours={hours:.2f}h")

print("\n[OK] Audit complete.")

[Universe] inputs that exist: 94789
[counts:strong] expected=94,789 | on_disk=94,789 | manifest_rows=94,789
[diff] expected but missing on disk: 0
[diff] on disk but missing in manifest: 0
[diff] manifest rows without file: 0
[probe] sample=300 | sr={16000: 300} | ch={1: 300} | subtype={'PCM_16': 300} | bad=0
[hours:strong] rows=94,789 | hours=181.31h

[OK] Audit complete.


In [ ]:
# ========== PREPROCESSING (BASE profile; resumable; same style as before) ==========
# Pipeline: DC high-pass -> per-file gain to TARGET_DB -> light trim
# Writes: /content/drive/MyDrive/hindi_dfake/processed/wav/base/...
# Logs  : /content/drive/MyDrive/hindi_dfake/metadata/proc_manifest.csv (profile=base)

import os, csv, time, uuid, hashlib, subprocess
from pathlib import Path
import pandas as pd, numpy as np, soundfile as sf
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.auto import tqdm

# ---------------- CONFIG ----------------
ROOT = Path("/content/drive/MyDrive/hindi_dfake")
META = ROOT / "metadata"
OUT_ROOT = ROOT / "processed" / "wav"

PROFILE = "base"          # <- BASE profile
INCLUDE_ATTACKS = True    # include attacks.labeled.csv if present
FRACTION_REAL = 1.00      # process all (resumable)
FRACTION_FAKE = 1.00
HASH_SEED = 2025

# Loudness & trim (same as your earlier cells)
TARGET_DB   = -26.0       # target RMS (dBFS-ish)
TRIM_THR_DB = -45
TRIM_DUR_S  = 0.20

# Concurrency / batching
MAX_WORKERS   = 6
INFLIGHT_MULT = 3
BATCH_WRITE   = 500

REAL_CSV  = META / "master_real.csv"
FAKE_CSV  = META / "master_fake.csv"
ATT_CSV   = META / "attacks.labeled.csv"
MANIFEST  = META / "proc_manifest.csv"
assert REAL_CSV.exists() and FAKE_CSV.exists(), "Missing master_real.csv/master_fake.csv"

# ---------------- helpers ----------------
def read_csv_safe(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path)
    for c in ["utt_id","path","duration","label","fake_type","source"]:
        if c not in df.columns:
            df[c] = "" if c != "duration" else 0.0
    return df

def normalize_label(lab, fake_type, src, path):
    l = str(lab).strip().lower()
    if l in {"real","fake"}: return l
    ft = str(fake_type).strip().lower()
    s  = str(src).strip().lower()
    p  = str(path)
    if "/raw/real_clean" in p or s.startswith("real_"): return "real"
    if ft in {"tts_edge","channel_attack","vc"} or "/raw/fake_" in p or s.startswith("fake_"): return "fake"
    return "fake"

def stable_frac_mask(series: pd.Series, frac: float, seed: int) -> pd.Series:
    if frac >= 1.0: return pd.Series([True]*len(series), index=series.index)
    s = series.astype(str).apply(lambda p: int(hashlib.sha1((p+str(seed)).encode()).hexdigest()[:8], 16) / 0xFFFFFFFF)
    return s < frac

def out_path_for(in_path: str) -> Path:
    p = Path(in_path)
    try:    rel = p.relative_to(ROOT)
    except: rel = Path("_external") / p.name
    return OUT_ROOT / PROFILE / rel

def ensure_parent(p: Path): p.parent.mkdir(parents=True, exist_ok=True)

def rms_dbfs(p: Path) -> float:
    try:
        x, sr = sf.read(str(p), dtype="float32", always_2d=False)
        if x.ndim > 1: x = x.mean(axis=1)
        if len(x) == 0: return -120.0
        rms = float(np.sqrt(np.mean(np.square(x))))
        if rms <= 1e-9: return -120.0
        return 20.0*np.log10(np.clip(rms, 1e-9, 1.0))
    except Exception:
        return -120.0

def build_filter_chain_base(gain_db: float) -> str:
    return ",".join([
        "highpass=f=20",
        f"volume={gain_db}dB",
        f"silenceremove=start_periods=1:start_duration={TRIM_DUR_S}:start_threshold={TRIM_THR_DB}dB:"
        f"stop_periods=1:stop_duration={TRIM_DUR_S}:stop_threshold={TRIM_THR_DB}dB"
    ])

# ---------------- load inputs ----------------
dfR = read_csv_safe(REAL_CSV)
dfF = read_csv_safe(FAKE_CSV)
dfs = [dfR, dfF]
if INCLUDE_ATTACKS and ATT_CSV.exists(): dfs.append(read_csv_safe(ATT_CSV))

df_all = pd.concat(dfs, ignore_index=True)
df_all = df_all[df_all["path"].astype(str).apply(lambda p: Path(p).exists())].copy()
df_all["label"] = [normalize_label(l, ft, s, p) for l, ft, s, p in
                   zip(df_all["label"], df_all["fake_type"], df_all["source"], df_all["path"])]

print("Input counts (normalized):")
print(df_all.groupby(df_all["label"].astype(str).str.lower()).size().to_string())

# deterministic per-label subset (here: everything; still resumable)
parts = []
for lab, g in df_all.groupby(df_all["label"].astype(str).str.lower()):
    frac = FRACTION_REAL if lab == "real" else FRACTION_FAKE
    parts.append(g if frac>=1.0 else g[stable_frac_mask(g["path"], frac, HASH_SEED)])
df_in = pd.concat(parts, ignore_index=True) if parts else df_all.copy()

# ---------------- manifest / resume ----------------
MANIFEST.parent.mkdir(parents=True, exist_ok=True)
if MANIFEST.exists():
    mf = pd.read_csv(MANIFEST, usecols=["profile","path_out"])
    done_set = set(mf.loc[mf["profile"]==PROFILE, "path_out"].astype(str))
else:
    with open(MANIFEST, "w", newline="", encoding="utf-8") as f:
        csv.writer(f).writerow(["utt_id","path_in","path_out","profile","duration","rms_db","peak_db"])
    done_set = set()

def already_done(pin: str) -> bool:
    pout = out_path_for(pin)
    return pout.exists() or (str(pout) in done_set)

todo_rows = [r for _, r in df_in.iterrows() if not already_done(str(r["path"]))]
planned, todo = len(df_in), len(todo_rows)
print(f"\n[preproc:{PROFILE}] PLANNED={planned:,} | ALREADY_DONE={planned-todo:,} | TO_DO={todo:,}")

# ---------------- worker (BASE chain) ----------------
def preprocess_one(row):
    pin = str(row["path"])
    pout = out_path_for(pin)
    ensure_parent(pout)

    # If input SR != 16k, resample first (same FS temp)
    try:
        x, sr = sf.read(pin, dtype="float32", always_2d=False)
    except Exception as e:
        return {"error": f"read_fail: {e}", "path_in": pin}
    if sr != 16000 or (x.ndim > 1):
        tmp_in = pout.parent / f".{pout.stem}.in-{uuid.uuid4().hex}.wav"
        cmd = ["ffmpeg","-nostdin","-hide_banner","-loglevel","error","-y",
               "-i", pin, "-ac","1","-ar","16000","-sample_fmt","s16", str(tmp_in)]
        p = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        if p.returncode != 0:
            try: tmp_in.unlink(missing_ok=True)
            except: pass
            return {"error": "resample_fail", "path_in": pin}
        pin_for_chain = str(tmp_in)
    else:
        pin_for_chain = pin

    gain_db = float(np.clip(TARGET_DB - rms_dbfs(Path(pin_for_chain)), -20.0, 20.0))
    FILTER  = build_filter_chain_base(gain_db)

    tmp_out = pout.parent / f".{pout.stem}.tmp-{uuid.uuid4().hex}.wav"
    cmd = ["ffmpeg","-nostdin","-hide_banner","-loglevel","error","-y",
           "-i", pin_for_chain, "-ac","1","-ar","16000",
           "-af", FILTER, "-sample_fmt","s16", str(tmp_out)]
    p = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # clean temp (if created)
    if pin_for_chain != pin:
        try: Path(pin_for_chain).unlink(missing_ok=True)
        except: pass

    if p.returncode != 0:
        try: tmp_out.unlink(missing_ok=True)
        except: pass
        return {"error": p.stderr.decode("utf-8","ignore")[:300], "path_in": pin}

    os.replace(str(tmp_out), str(pout))

    # quick stats
    try:
        z, sr2 = sf.read(str(pout), dtype="float32", always_2d=False)
        if z.ndim > 1: z = z.mean(axis=1)
        dur = len(z)/float(sr2) if sr2 else 0.0
        peak = float(np.max(np.abs(z))) if len(z) else 0.0
        rms  = float(np.sqrt(np.mean(np.square(z)))) if len(z) else 0.0
        rms_db = -120.0 if rms<=1e-9 else 20*np.log10(np.clip(rms,1e-9,1.0))
        peak_db = -120.0 if peak<=1e-9 else 20*np.log10(np.clip(peak,1e-9,1.0))
    except Exception:
        dur, rms_db, peak_db = 0.0, -120.0, -120.0

    return {"utt_id": row.get("utt_id",""), "path_in": pin, "path_out": str(pout),
            "profile": PROFILE, "duration": round(dur,3),
            "rms_db": round(rms_db,2), "peak_db": round(peak_db,2)}

# ---------------- streaming scheduler + progress ----------------
created = 0; errs = 0; buf = []
start = time.time()
inflight_cap = max(MAX_WORKERS * INFLIGHT_MULT, MAX_WORKERS)
pbar = tqdm(total=len(todo_rows), desc=f"preproc:{PROFILE}", unit="file")

def flush_manifest(rows):
    if not rows: return
    with open(MANIFEST, "a", newline="", encoding="utf-8") as fcsv:
        w = csv.DictWriter(fcsv, fieldnames=["utt_id","path_in","path_out","profile","duration","rms_db","peak_db"])
        for r in rows: w.writerow(r)

it = iter(todo_rows)
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
    inflight = set()
    for _ in range(min(inflight_cap, len(todo_rows))):
        try: inflight.add(ex.submit(preprocess_one, next(it)))
        except StopIteration: break
    while inflight:
        done = next(as_completed(inflight))
        inflight.remove(done)
        res = done.result()
        pbar.update(1)
        if isinstance(res, dict) and "error" in res:
            errs += 1
        elif res:
            buf.append(res); created += 1
            if len(buf) >= BATCH_WRITE:
                flush_manifest(buf); buf.clear()
        try: inflight.add(ex.submit(preprocess_one, next(it)))
        except StopIteration: pass

flush_manifest(buf)
pbar.close()
print(f"\n[{PROFILE}] FINISHED: +{created} files, {errs} errors | elapsed {(time.time()-start)/60:.1f} min")

# ---------------- quick audit / summary ----------------
def hours_from_manifest(profile: str):
    try:
        m = pd.read_csv(MANIFEST)
        m = m[m["profile"]==profile]
        return float(pd.to_numeric(m["duration"], errors="coerce").fillna(0).sum()/3600.0), len(m)
    except Exception:
        return 0.0, 0

outs = list((OUT_ROOT/PROFILE).rglob("*.wav"))
print(f"[{PROFILE}] outputs on disk: {len(outs):,} (showing a few)")
for p in outs[:8]:
    print("  ", p)

hrs, nrows = hours_from_manifest(PROFILE)
print(f"[{PROFILE}] manifest rows: {nrows:,} | hours: {hrs:.2f} h")

# keep manifest dedupbed (safety)
if MANIFEST.exists():
    mf = pd.read_csv(MANIFEST)
    before = len(mf)
    mf = mf.drop_duplicates(subset=["profile","path_out"], keep="first").reset_index(drop=True)
    if len(mf) != before:
        mf.to_csv(MANIFEST, index=False)
        print(f"[fix] dedup manifest: -{before-len(mf)} rows")
print("[OK] BASE preprocessing done.")


Input counts (normalized):
label
fake    48170
real    49623

[preproc:base] PLANNED=97,793 | ALREADY_DONE=94,789 | TO_DO=3,004


preproc:base:   0%|          | 0/3004 [00:00<?, ?file/s]


[base] FINISHED: +3004 files, 0 errors | elapsed 9.0 min
[base] outputs on disk: 97,793 (showing a few)
   /content/drive/MyDrive/hindi_dfake/processed/wav/base/raw/fake_tts/tts_edge_6859d4f039aea68f.wav
   /content/drive/MyDrive/hindi_dfake/processed/wav/base/raw/fake_tts/tts_edge_19e1c8d7cad439c0.wav
   /content/drive/MyDrive/hindi_dfake/processed/wav/base/raw/fake_tts/tts_edge_a5ea2af5277b57c4.wav
   /content/drive/MyDrive/hindi_dfake/processed/wav/base/raw/fake_tts/tts_edge_7aee2f81107fe9de.wav
   /content/drive/MyDrive/hindi_dfake/processed/wav/base/raw/fake_tts/tts_edge_c103d799a208b3c8.wav
   /content/drive/MyDrive/hindi_dfake/processed/wav/base/raw/fake_tts/tts_edge_8883f29f2fb7bfdf.wav
   /content/drive/MyDrive/hindi_dfake/processed/wav/base/raw/fake_tts/tts_edge_e43afa0cec72f0aa.wav
   /content/drive/MyDrive/hindi_dfake/processed/wav/base/raw/fake_tts/tts_edge_47ea9317998de3aa.wav
[base] manifest rows: 97,793 | hours: 78.72 h
[OK] BASE preprocessing done.


In [ ]:
# ================= PREPROCESSING — STRONG (EQ + RawBoost v3) =================
# Writes: /content/drive/MyDrive/hindi_dfake/processed/wav/strong/...
# Logs to: /content/drive/MyDrive/hindi_dfake/metadata/proc_manifest.csv (profile=strong)

import os, csv, uuid, time, hashlib, subprocess
from pathlib import Path
import pandas as pd, numpy as np, soundfile as sf
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.auto import tqdm

# ------------ CONFIG ------------
ROOT = Path("/content/drive/MyDrive/hindi_dfake")
META = ROOT / "metadata"
OUT_ROOT = ROOT / "processed" / "wav"

PROFILE = "strong"          # strong = base + artifact EQ + RawBoost v3
INCLUDE_ATTACKS = True
FRACTION_REAL = 1.00        # process all
FRACTION_FAKE = 1.00
HASH_SEED = 2025

# Loudness & trim
TARGET_DB   = -26.0
TRIM_THR_DB = -45
TRIM_DUR_S  = 0.20

# Concurrency / batching (safe defaults for Colab)
MAX_WORKERS   = 6
INFLIGHT_MULT = 3
BATCH_WRITE   = 500

# RawBoost v3–style knobs (same as we used earlier)
RB_ENABLE           = True
RB_BANDPASS_HZ      = (2500, 6000)
RB_SNR_DB           = 25.0
RB_NOISE_PROB       = 1.0
RB_IMPULSE_PROB     = 0.5
RB_IMPULSES_PER_SEC = 1.0
RB_IMPULSE_GAIN     = 0.08
RB_REVERB_PROB      = 0.6
RB_REVERB_T_SEC     = 0.03
RB_REVERB_DECAY     = 0.35
RB_REVERB_WET       = 0.18

REAL_CSV  = META / "master_real.csv"
FAKE_CSV  = META / "master_fake.csv"
ATT_CSV   = META / "attacks.labeled.csv"
MANIFEST  = META / "proc_manifest.csv"
assert REAL_CSV.exists() and FAKE_CSV.exists()

# ------------ helpers ------------
def read_csv_safe(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path)
    for c in ["utt_id","path","duration","label","fake_type","source"]:
        if c not in df.columns:
            df[c] = "" if c != "duration" else 0.0
    return df

def normalize_label(lab, fake_type, src, path):
    l = str(lab).strip().lower()
    if l in {"real","fake"}: return l
    ft = str(fake_type).strip().lower()
    s  = str(src).strip().lower()
    p  = str(path)
    if "/raw/real_clean" in p or s.startswith("real_"): return "real"
    if ft in {"tts_edge","channel_attack","vc"} or "/raw/fake_" in p or s.startswith("fake_"): return "fake"
    return "fake"

def stable_frac_mask(series: pd.Series, frac: float, seed: int) -> pd.Series:
    if frac >= 1.0: return pd.Series([True]*len(series), index=series.index)
    s = series.astype(str).apply(lambda p: int(hashlib.sha1((p+str(seed)).encode()).hexdigest()[:8], 16) / 0xFFFFFFFF)
    return s < frac

def out_path_for(in_path: str) -> Path:
    p = Path(in_path)
    try:    rel = p.relative_to(ROOT)
    except: rel = Path("_external") / p.name
    return OUT_ROOT / PROFILE / rel

def ensure_parent(p: Path): p.parent.mkdir(parents=True, exist_ok=True)

def rms_dbfs_arr(x: np.ndarray) -> float:
    if x.ndim > 1: x = x.mean(axis=1)
    if len(x) == 0: return -120.0
    rms = float(np.sqrt(np.mean(np.square(x))))
    if rms <= 1e-9: return -120.0
    return 20.0*np.log10(np.clip(rms, 1e-9, 1.0))

# ---- RawBoost v3–style (deterministic per file) ----
def _rng_from_key(key: str):
    seed = int(hashlib.sha1(key.encode("utf-8")).hexdigest()[:8], 16)
    return np.random.default_rng(seed)

def _add_colored_noise(x, sr, rng, band=(2500,6000), snr_db=25.0):
    n = len(x)
    if n == 0: return x
    wn = rng.standard_normal(n).astype(np.float32)
    X = np.fft.rfft(wn); freqs = np.fft.rfftfreq(n, d=1.0/sr)
    X[~((freqs >= band[0]) & (freqs <= band[1]))] = 0.0
    noise = np.fft.irfft(X, n=n).astype(np.float32)
    sig_rms = np.sqrt(np.mean(np.square(x))) + 1e-9
    target_noise_rms = sig_rms / (10.0**(snr_db/20.0))
    cur_rms = np.sqrt(np.mean(np.square(noise))) + 1e-12
    noise *= (target_noise_rms / cur_rms)
    return np.clip(x + noise, -1.0, 1.0)

def _add_impulses(x, sr, rng, per_sec=1.0, gain=0.08):
    n = len(x)
    if n == 0: return x
    dur = n/float(sr); k = max(1, int(per_sec * dur))
    idx = rng.integers(0, n, size=k)
    amp = gain * (np.sqrt(np.mean(np.square(x))) + 1e-9)
    y = x.copy()
    y[idx] = np.clip(y[idx] + amp * rng.choice([-1.0, 1.0], size=k), -1.0, 1.0)
    return y

def _add_small_reverb(x, sr, rng, t_sec=0.03, decay=0.35, wet=0.18):
    n = len(x)
    if n == 0: return x
    ir_len = max(8, int(t_sec * sr))
    t = np.arange(ir_len, dtype=np.float32)
    ir = np.exp(-decay * t / ir_len).astype(np.float32)
    for _ in range(3):
        pos = int(rng.integers(0, ir_len))
        ir[pos] += float(rng.uniform(0.1, 0.3))
    ir /= (np.sum(np.abs(ir)) + 1e-9)
    y = np.convolve(x, ir, mode="full")[:n].astype(np.float32)
    return np.clip((1.0 - wet) * x + wet * y, -1.0, 1.0)

def rawboost_v3(x: np.ndarray, sr: int, key: str) -> np.ndarray:
    if x.ndim > 1: x = x.mean(axis=1)
    x = np.clip(x, -1.0, 1.0).astype(np.float32)
    if not RB_ENABLE: return x
    rng = _rng_from_key(key)
    if rng.uniform() < RB_NOISE_PROB:
        x = _add_colored_noise(x, sr, rng, RB_BANDPASS_HZ, RB_SNR_DB)
    if rng.uniform() < RB_IMPULSE_PROB:
        x = _add_impulses(x, sr, rng, RB_IMPULSES_PER_SEC, RB_IMPULSE_GAIN)
    if rng.uniform() < RB_REVERB_PROB:
        x = _add_small_reverb(x, sr, rng, RB_REVERB_T_SEC, RB_REVERB_DECAY, RB_REVERB_WET)
    return x

def build_filter_chain_strong(gain_db: float) -> str:
    return ",".join([
        "highpass=f=20",
        "equalizer=f=3000:t=q:w=1.0:g=2.5",
        "equalizer=f=4800:t=q:w=0.9:g=2.0",
        "treble=g=1.0:f=6000:t=h:w=0.7",
        f"volume={gain_db}dB",
        f"silenceremove=start_periods=1:start_duration={TRIM_DUR_S}:start_threshold={TRIM_THR_DB}dB:stop_periods=1:stop_duration={TRIM_DUR_S}:stop_threshold={TRIM_THR_DB}dB"
    ])

# ------------ load inputs ------------
dfR = read_csv_safe(REAL_CSV)
dfF = read_csv_safe(FAKE_CSV)
dfs = [dfR, dfF]
if INCLUDE_ATTACKS and ATT_CSV.exists():
    dfs.append(read_csv_safe(ATT_CSV))
df_all = pd.concat(dfs, ignore_index=True)
df_all = df_all[df_all["path"].astype(str).apply(lambda p: Path(p).exists())].copy()
df_all["label"] = [normalize_label(l, ft, s, p) for l, ft, s, p in
                   zip(df_all["label"], df_all["fake_type"], df_all["source"], df_all["path"])]

# deterministic per-label subset (here: everything, but we still keep the hook)
parts = []
for lab, g in df_all.groupby(df_all["label"].astype(str).str.lower()):
    frac = FRACTION_REAL if lab == "real" else FRACTION_FAKE
    parts.append(g if frac>=1.0 else g[stable_frac_mask(g["path"], frac, HASH_SEED)])
df_in = pd.concat(parts, ignore_index=True) if parts else df_all.copy()

# ------------ manifest skip logic ------------
MANIFEST.parent.mkdir(parents=True, exist_ok=True)
if MANIFEST.exists():
    mf = pd.read_csv(MANIFEST, usecols=["profile","path_out"])
    done_set = set(mf.loc[mf["profile"]==PROFILE, "path_out"].astype(str))
else:
    with open(MANIFEST, "w", newline="", encoding="utf-8") as f:
        csv.writer(f).writerow(["utt_id","path_in","path_out","profile","duration","rms_db","peak_db"])
    done_set = set()

def already_done(pin: str) -> bool:
    pout = out_path_for(pin)
    return pout.exists() or (str(pout) in done_set)

# Only process files that don't have a STRONG output yet
todo_rows = [r for _, r in df_in.iterrows() if not already_done(str(r["path"]))]

planned, todo = len(df_in), len(todo_rows)
print("Input counts (normalized):")
print(df_all.groupby(df_all['label']).size().to_string())
print(f"\n[preproc:{PROFILE}] PLANNED={planned:,} | ALREADY_DONE={planned-todo:,} | TO_DO={todo:,}")

# ------------ worker ------------
def preprocess_one(row):
    pin = str(row["path"])
    pout = out_path_for(pin)
    ensure_parent(pout)

    # read (and resample if needed)
    try:
        x, sr = sf.read(pin, dtype="float32", always_2d=False)
    except Exception as e:
        return {"error": f"read_fail: {e}", "path_in": pin}
    if x.ndim > 1: x = x.mean(axis=1)
    if sr != 16000:
        tmp = pout.parent / f".{pout.stem}.in-{uuid.uuid4().hex}.wav"
        cmd = ["ffmpeg","-nostdin","-hide_banner","-loglevel","error","-y",
               "-i", pin, "-ac","1","-ar","16000","-sample_fmt","s16", str(tmp)]
        p = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        if p.returncode != 0:
            try: tmp.unlink(missing_ok=True)
            except: pass
            return {"error": "resample_fail", "path_in": pin}
        x, sr = sf.read(str(tmp), dtype="float32", always_2d=False)
        try: tmp.unlink(missing_ok=True)
        except: pass

    x = np.clip(x, -1.0, 1.0).astype(np.float32)

    # RawBoost v3–style
    key = f"{pin}|{PROFILE}"
    try:    y = rawboost_v3(x, sr, key)
    except: y = x

    # write RB temp near target (avoid cross-device link)
    tmp_rb = pout.parent / f".{pout.stem}.rb-{uuid.uuid4().hex}.wav"
    sf.write(str(tmp_rb), y, 16000, subtype="PCM_16")

    # EQ + loudness + trim via ffmpeg
    gain_db = float(np.clip(TARGET_DB - rms_dbfs_arr(y), -20.0, 20.0))
    FILTER  = build_filter_chain_strong(gain_db)

    tmp_out = pout.parent / f".{pout.stem}.tmp-{uuid.uuid4().hex}.wav"
    cmd = ["ffmpeg","-nostdin","-hide_banner","-loglevel","error","-y",
           "-i", str(tmp_rb), "-ac","1","-ar","16000",
           "-af", FILTER, "-sample_fmt","s16", str(tmp_out)]
    p = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    try: tmp_rb.unlink(missing_ok=True)
    except: pass
    if p.returncode != 0:
        try: tmp_out.unlink(missing_ok=True)
        except: pass
        return {"error": p.stderr.decode("utf-8","ignore")[:300], "path_in": pin}

    os.replace(str(tmp_out), str(pout))

    # stats
    try:
        z, sr2 = sf.read(str(pout), dtype="float32", always_2d=False)
        if z.ndim > 1: z = z.mean(axis=1)
        dur = len(z)/float(sr2) if sr2 else 0.0
        peak = float(np.max(np.abs(z))) if len(z) else 0.0
        rms  = float(np.sqrt(np.mean(np.square(z)))) if len(z) else 0.0
        rms_db = -120.0 if rms<=1e-9 else 20*np.log10(np.clip(rms,1e-9,1.0))
        peak_db = -120.0 if peak<=1e-9 else 20*np.log10(np.clip(peak,1e-9,1.0))
    except Exception:
        dur, rms_db, peak_db = 0.0, -120.0, -120.0

    return {"utt_id": row.get("utt_id",""), "path_in": pin, "path_out": str(pout),
            "profile": PROFILE, "duration": round(dur,3),
            "rms_db": round(rms_db,2), "peak_db": round(peak_db,2)}

# ------------ streaming scheduler with progress ------------
created = 0; errs = 0; buf = []
start = time.time()
inflight_cap = max(MAX_WORKERS * INFLIGHT_MULT, MAX_WORKERS)
pbar = tqdm(total=len(todo_rows), desc=f"preproc:{PROFILE}", unit="file")

def flush_manifest(rows):
    if not rows: return
    with open(MANIFEST, "a", newline="", encoding="utf-8") as fcsv:
        w = csv.DictWriter(fcsv, fieldnames=["utt_id","path_in","path_out","profile","duration","rms_db","peak_db"])
        for r in rows: w.writerow(r)

it = iter(todo_rows)
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
    inflight = set()
    for _ in range(min(inflight_cap, len(todo_rows))):
        try: inflight.add(ex.submit(preprocess_one, next(it)))
        except StopIteration: break
    while inflight:
        done = next(as_completed(inflight))
        inflight.remove(done)
        res = done.result()
        pbar.update(1)
        if isinstance(res, dict) and "error" in res:
            errs += 1
        elif res:
            buf.append(res); created += 1
            if len(buf) >= BATCH_WRITE:
                flush_manifest(buf); buf.clear()
        try: inflight.add(ex.submit(preprocess_one, next(it)))
        except StopIteration: pass

flush_manifest(buf)
pbar.close()
print(f"\n[{PROFILE}] FINISHED: +{created} files, {errs} errors | elapsed {(time.time()-start)/60:.1f} min")

# quick manifest slice
if MANIFEST.exists():
    m = pd.read_csv(MANIFEST)
    mp = m[m["profile"]==PROFILE]
    hrs = float(pd.to_numeric(mp["duration"], errors="coerce").fillna(0).sum()/3600.0)
    print(f"[{PROFILE}] Manifest rows: {len(mp):,} | hours: {hrs:.2f} h")

print("[OK] STRONG preprocessing done.")

Input counts (normalized):
label
fake    48170
real    49623

[preproc:strong] PLANNED=97,793 | ALREADY_DONE=94,789 | TO_DO=3,004


preproc:strong:   0%|          | 0/3004 [00:00<?, ?file/s]


[strong] FINISHED: +3004 files, 0 errors | elapsed 10.1 min
[strong] Manifest rows: 97,793 | hours: 185.21 h
[OK] STRONG preprocessing done.


In [ ]:
# ========== PREPROCESS FLEURS (BASE profile; exact same chain as MMS-TTS) ==========
# Pipeline: DC high-pass -> per-file gain to TARGET_DB -> light trim
# Writes: /content/drive/MyDrive/hindi_dfake/processed/wav/base/...
# Logs  : /content/drive/MyDrive/hindi_dfake/metadata/proc_manifest.csv (profile=base)

import os, csv, time, uuid, hashlib, subprocess
from pathlib import Path
import pandas as pd, numpy as np, soundfile as sf
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.auto import tqdm

# ---------------- CONFIG ----------------
ROOT = Path("/content/drive/MyDrive/hindi_dfake")
META = ROOT / "metadata"
OUT_ROOT = ROOT / "processed" / "wav"

PROFILE = "base"          # <- BASE profile (unchanged)
HASH_SEED = 2025

# Loudness & trim (identical to your MMS-TTS run)
TARGET_DB   = -26.0
TRIM_THR_DB = -45
TRIM_DUR_S  = 0.20

# Concurrency / batching
MAX_WORKERS   = 6
INFLIGHT_MULT = 3
BATCH_WRITE   = 500

FLEURS_CSV = META / "thirdparty_real_test.fleurs.csv"   # <-- input list (FLEURS only)
MANIFEST   = META / "proc_manifest.csv"

assert FLEURS_CSV.exists(), f"Missing {FLEURS_CSV}"

# ---------------- helpers (unchanged) ----------------
def read_csv_safe(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path)
    for c in ["utt_id","path","duration","label","fake_type","source"]:
        if c not in df.columns:
            df[c] = "" if c != "duration" else 0.0
    return df

def normalize_label(lab, fake_type, src, path):
    l = str(lab).strip().lower()
    if l in {"real","fake"}: return l
    ft = str(fake_type).strip().lower()
    s  = str(src).strip().lower()
    p  = str(path)
    if "/raw/real_clean" in p or s.startswith("real_"): return "real"
    if ft in {"tts_edge","channel_attack","vc"} or "/raw/fake_" in p or s.startswith("fake_"): return "fake"
    return "fake"

def out_path_for(in_path: str) -> Path:
    p = Path(in_path)
    try:    rel = p.relative_to(ROOT)
    except: rel = Path("_external") / p.name
    return OUT_ROOT / PROFILE / rel

def ensure_parent(p: Path): p.parent.mkdir(parents=True, exist_ok=True)

def rms_dbfs(p: Path) -> float:
    try:
        x, sr = sf.read(str(p), dtype="float32", always_2d=False)
        if hasattr(x, "ndim") and x.ndim > 1: x = x.mean(axis=1)
        if len(x) == 0: return -120.0
        rms = float(np.sqrt(np.mean(np.square(x))))
        if rms <= 1e-9: return -120.0
        return 20.0*np.log10(np.clip(rms, 1e-9, 1.0))
    except Exception:
        return -120.0

def build_filter_chain_base(gain_db: float) -> str:
    return ",".join([
        "highpass=f=20",
        f"volume={gain_db}dB",
        f"silenceremove=start_periods=1:start_duration={TRIM_DUR_S}:start_threshold={TRIM_THR_DB}dB:"
        f"stop_periods=1:stop_duration={TRIM_DUR_S}:stop_threshold={TRIM_THR_DB}dB"
    ])

# ---------------- load inputs (FLEURS ONLY) ----------------
df_fl = read_csv_safe(FLEURS_CSV)
# keep only files that exist on disk
df_fl = df_fl[df_fl["path"].astype(str).apply(lambda p: Path(p).exists())].copy()
# force label to real (FLEURS is real speech)
df_fl["label"] = [normalize_label("real", ft, s, p) for ft, s, p in
                  zip(df_fl["fake_type"], df_fl["source"], df_fl["path"])]

print("Input counts (FLEURS only, normalized):")
print(df_fl.groupby(df_fl["label"].astype(str).str.lower()).size().to_string())

df_in = df_fl.copy()  # all fleurs; resume logic below will skip already-done

# ---------------- manifest / resume ----------------
MANIFEST.parent.mkdir(parents=True, exist_ok=True)
if MANIFEST.exists():
    mf = pd.read_csv(MANIFEST, usecols=["profile","path_out"])
    done_set = set(mf.loc[mf["profile"]==PROFILE, "path_out"].astype(str))
else:
    with open(MANIFEST, "w", newline="", encoding="utf-8") as f:
        csv.writer(f).writerow(["utt_id","path_in","path_out","profile","duration","rms_db","peak_db"])
    done_set = set()

def already_done(pin: str) -> bool:
    pout = out_path_for(pin)
    return pout.exists() or (str(pout) in done_set)

todo_rows = [r for _, r in df_in.iterrows() if not already_done(str(r["path"]))]
planned, todo = len(df_in), len(todo_rows)
print(f"\n[preproc:{PROFILE} | FLEURS] PLANNED={planned:,} | ALREADY_DONE={planned-todo:,} | TO_DO={todo:,}")

# ---------------- worker (BASE chain; identical) ----------------
def preprocess_one(row):
    pin = str(row["path"])
    pout = out_path_for(pin)
    ensure_parent(pout)

    # If input SR != 16k or multichannel, resample first
    try:
        x, sr = sf.read(pin, dtype="float32", always_2d=False)
    except Exception as e:
        return {"error": f"read_fail: {e}", "path_in": pin}
    if (sr != 16000) or (hasattr(x, "ndim") and x.ndim > 1):
        tmp_in = pout.parent / f".{pout.stem}.in-{uuid.uuid4().hex}.wav"
        cmd = ["ffmpeg","-nostdin","-hide_banner","-loglevel","error","-y",
               "-i", pin, "-ac","1","-ar","16000","-sample_fmt","s16", str(tmp_in)]
        p = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        if p.returncode != 0:
            try: tmp_in.unlink(missing_ok=True)
            except: pass
            return {"error": "resample_fail", "path_in": pin}
        pin_for_chain = str(tmp_in)
    else:
        pin_for_chain = pin

    gain_db = float(np.clip(TARGET_DB - rms_dbfs(Path(pin_for_chain)), -20.0, 20.0))
    FILTER  = build_filter_chain_base(gain_db)

    tmp_out = pout.parent / f".{pout.stem}.tmp-{uuid.uuid4().hex}.wav"
    cmd = ["ffmpeg","-nostdin","-hide_banner","-loglevel","error","-y",
           "-i", pin_for_chain, "-ac","1","-ar","16000",
           "-af", FILTER, "-sample_fmt","s16", str(tmp_out)]
    p = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # clean temp (if created)
    if pin_for_chain != pin:
        try: Path(pin_for_chain).unlink(missing_ok=True)
        except: pass

    if p.returncode != 0:
        try: tmp_out.unlink(missing_ok=True)
        except: pass
        return {"error": p.stderr.decode("utf-8","ignore")[:300], "path_in": pin}

    os.replace(str(tmp_out), str(pout))

    # quick stats
    try:
        z, sr2 = sf.read(str(pout), dtype="float32", always_2d=False)
        if hasattr(z, "ndim") and z.ndim > 1: z = z.mean(axis=1)
        dur = len(z)/float(sr2) if sr2 else 0.0
        peak = float(np.max(np.abs(z))) if len(z) else 0.0
        rms  = float(np.sqrt(np.mean(np.square(z)))) if len(z) else 0.0
        rms_db = -120.0 if rms<=1e-9 else 20*np.log10(np.clip(rms,1e-9,1.0))
        peak_db = -120.0 if peak<=1e-9 else 20*np.log10(np.clip(peak,1e-9,1.0))
    except Exception:
        dur, rms_db, peak_db = 0.0, -120.0, -120.0

    return {"utt_id": row.get("utt_id",""), "path_in": pin, "path_out": str(pout),
            "profile": PROFILE, "duration": round(dur,3),
            "rms_db": round(rms_db,2), "peak_db": round(peak_db,2)}

# ---------------- scheduler + progress (unchanged) ----------------
created = 0; errs = 0; buf = []
start = time.time()
inflight_cap = max(MAX_WORKERS * INFLIGHT_MULT, MAX_WORKERS)
pbar = tqdm(total=len(todo_rows), desc=f"preproc:{PROFILE}:FLEURS", unit="file")

def flush_manifest(rows):
    if not rows: return
    with open(MANIFEST, "a", newline="", encoding="utf-8") as fcsv:
        w = csv.DictWriter(fcsv, fieldnames=["utt_id","path_in","path_out","profile","duration","rms_db","peak_db"])
        for r in rows: w.writerow(r)

it = iter(todo_rows)
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
    inflight = set()
    for _ in range(min(inflight_cap, len(todo_rows))):
        try: inflight.add(ex.submit(preprocess_one, next(it)))
        except StopIteration: break
    while inflight:
        done = next(as_completed(inflight))
        inflight.remove(done)
        res = done.result()
        pbar.update(1)
        if isinstance(res, dict) and "error" in res:
            errs += 1
        elif res:
            buf.append(res); created += 1
            if len(buf) >= BATCH_WRITE:
                flush_manifest(buf); buf.clear()
        try: inflight.add(ex.submit(preprocess_one, next(it)))
        except StopIteration: pass

flush_manifest(buf)
pbar.close()
print(f"\n[{PROFILE}:FLEURS] FINISHED: +{created} files, {errs} errors | elapsed {(time.time()-start)/60:.1f} min")

# ---------------- quick audit / summary (unchanged) ----------------
def hours_from_manifest(profile: str):
    try:
        m = pd.read_csv(MANIFEST)
        m = m[m["profile"]==profile]
        return float(pd.to_numeric(m["duration"], errors="coerce").fillna(0).sum()/3600.0), len(m)
    except Exception:
        return 0.0, 0

outs = list((OUT_ROOT/PROFILE).rglob("*.wav"))
print(f"[{PROFILE}] outputs on disk: {len(outs):,} (showing a few)")
for p in outs[:8]:
    print("  ", p)

hrs, nrows = hours_from_manifest(PROFILE)
print(f"[{PROFILE}] manifest rows: {nrows:,} | hours: {hrs:.2f} h")

# keep manifest dedupbed (safety)
if MANIFEST.exists():
    mf = pd.read_csv(MANIFEST)
    before = len(mf)
    mf = mf.drop_duplicates(subset=["profile","path_out"], keep="first").reset_index(drop=True)
    if len(mf) != before:
        mf.to_csv(MANIFEST, index=False)
        print(f"[fix] dedup manifest: -{before-len(mf)} rows")
print("[OK] BASE preprocessing for FLEURS done.")

Input counts (FLEURS only, normalized):
label
real    604

[preproc:base | FLEURS] PLANNED=604 | ALREADY_DONE=0 | TO_DO=604


preproc:base:FLEURS:   0%|          | 0/604 [00:00<?, ?file/s]


[base:FLEURS] FINISHED: +604 files, 0 errors | elapsed 2.2 min
[base] outputs on disk: 98,397 (showing a few)
   /content/drive/MyDrive/hindi_dfake/processed/wav/base/raw/fake_tts/tts_edge_6859d4f039aea68f.wav
   /content/drive/MyDrive/hindi_dfake/processed/wav/base/raw/fake_tts/tts_edge_19e1c8d7cad439c0.wav
   /content/drive/MyDrive/hindi_dfake/processed/wav/base/raw/fake_tts/tts_edge_a5ea2af5277b57c4.wav
   /content/drive/MyDrive/hindi_dfake/processed/wav/base/raw/fake_tts/tts_edge_7aee2f81107fe9de.wav
   /content/drive/MyDrive/hindi_dfake/processed/wav/base/raw/fake_tts/tts_edge_c103d799a208b3c8.wav
   /content/drive/MyDrive/hindi_dfake/processed/wav/base/raw/fake_tts/tts_edge_8883f29f2fb7bfdf.wav
   /content/drive/MyDrive/hindi_dfake/processed/wav/base/raw/fake_tts/tts_edge_e43afa0cec72f0aa.wav
   /content/drive/MyDrive/hindi_dfake/processed/wav/base/raw/fake_tts/tts_edge_47ea9317998de3aa.wav
[base] manifest rows: 98,397 | hours: 79.59 h
[OK] BASE preprocessing for FLEURS done.


In [ ]:
# ================= PREPROCESS FLEURS — STRONG (EQ + RawBoost v3) =================
# Writes: /content/drive/MyDrive/hindi_dfake/processed/wav/strong/...
# Logs :  /content/drive/MyDrive/hindi_dfake/metadata/proc_manifest.csv (profile=strong)
# Scope:  ONLY rows from metadata/thirdparty_real_test.fleurs.csv (resume-safe)

import os, csv, uuid, time, hashlib, subprocess
from pathlib import Path
import pandas as pd, numpy as np, soundfile as sf
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.auto import tqdm

# ------------ CONFIG ------------
ROOT = Path("/content/drive/MyDrive/hindi_dfake")
META = ROOT / "metadata"
OUT_ROOT = ROOT / "processed" / "wav"

PROFILE = "strong"          # strong = base + artifact EQ + RawBoost v3 (same as MMS-TTS)
HASH_SEED = 2025

# Loudness & trim (identical to your MMS-TTS STRONG run)
TARGET_DB   = -26.0
TRIM_THR_DB = -45
TRIM_DUR_S  = 0.20

# Concurrency / batching
MAX_WORKERS   = 6
INFLIGHT_MULT = 3
BATCH_WRITE   = 500

FLEURS_CSV = META / "thirdparty_real_test.fleurs.csv"   # <-- input list (FLEURS only)
MANIFEST   = META / "proc_manifest.csv"
assert FLEURS_CSV.exists(), f"Missing {FLEURS_CSV}"

# ------------ helpers (unchanged logic from your STRONG job) ------------
def read_csv_safe(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path)
    for c in ["utt_id","path","duration","label","fake_type","source"]:
        if c not in df.columns:
            df[c] = "" if c != "duration" else 0.0
    return df

def out_path_for(in_path: str) -> Path:
    p = Path(in_path)
    try:    rel = p.relative_to(ROOT)
    except: rel = Path("_external") / p.name
    return OUT_ROOT / PROFILE / rel

def ensure_parent(p: Path): p.parent.mkdir(parents=True, exist_ok=True)

def rms_dbfs_arr(x: np.ndarray) -> float:
    if x.ndim > 1: x = x.mean(axis=1)
    if len(x) == 0: return -120.0
    rms = float(np.sqrt(np.mean(np.square(x))))
    if rms <= 1e-9: return -120.0
    return 20.0*np.log10(np.clip(rms, 1.0e-9, 1.0))

# ---- RawBoost v3–style (deterministic per file) ----
def _rng_from_key(key: str):
    seed = int(hashlib.sha1(key.encode("utf-8")).hexdigest()[:8], 16)
    return np.random.default_rng(seed)

def _add_colored_noise(x, sr, rng, band=(2500,6000), snr_db=25.0):
    n = len(x)
    if n == 0: return x
    wn = rng.standard_normal(n).astype(np.float32)
    X = np.fft.rfft(wn); freqs = np.fft.rfftfreq(n, d=1.0/sr)
    X[~((freqs >= band[0]) & (freqs <= band[1]))] = 0.0
    noise = np.fft.irfft(X, n=n).astype(np.float32)
    sig_rms = np.sqrt(np.mean(np.square(x))) + 1e-9
    target_noise_rms = sig_rms / (10.0**(snr_db/20.0))
    cur_rms = np.sqrt(np.mean(np.square(noise))) + 1e-12
    noise *= (target_noise_rms / cur_rms)
    return np.clip(x + noise, -1.0, 1.0)

def _add_impulses(x, sr, rng, per_sec=1.0, gain=0.08):
    n = len(x)
    if n == 0: return x
    dur = n/float(sr); k = max(1, int(per_sec * dur))
    idx = rng.integers(0, n, size=k)
    amp = gain * (np.sqrt(np.mean(np.square(x))) + 1e-9)
    y = x.copy()
    y[idx] = np.clip(y[idx] + amp * rng.choice([-1.0, 1.0], size=k), -1.0, 1.0)
    return y

def _add_small_reverb(x, sr, rng, t_sec=0.03, decay=0.35, wet=0.18):
    n = len(x)
    if n == 0: return x
    ir_len = max(8, int(t_sec * sr))
    t = np.arange(ir_len, dtype=np.float32)
    ir = np.exp(-decay * t / ir_len).astype(np.float32)
    for _ in range(3):
        pos = int(rng.integers(0, ir_len))
        ir[pos] += float(rng.uniform(0.1, 0.3))
    ir /= (np.sum(np.abs(ir)) + 1e-9)
    y = np.convolve(x, ir, mode="full")[:n].astype(np.float32)
    return np.clip((1.0 - wet) * x + wet * y, -1.0, 1.0)

# knobs copied from your MMS-TTS config
RB_ENABLE           = True
RB_BANDPASS_HZ      = (2500, 6000)
RB_SNR_DB           = 25.0
RB_NOISE_PROB       = 1.0
RB_IMPULSE_PROB     = 0.5
RB_IMPULSES_PER_SEC = 1.0
RB_IMPULSE_GAIN     = 0.08
RB_REVERB_PROB      = 0.6
RB_REVERB_T_SEC     = 0.03
RB_REVERB_DECAY     = 0.35
RB_REVERB_WET       = 0.18

def rawboost_v3(x: np.ndarray, sr: int, key: str) -> np.ndarray:
    if x.ndim > 1: x = x.mean(axis=1)
    x = np.clip(x, -1.0, 1.0).astype(np.float32)
    if not RB_ENABLE: return x
    rng = _rng_from_key(key)
    if rng.uniform() < RB_NOISE_PROB:
        x = _add_colored_noise(x, sr, rng, RB_BANDPASS_HZ, RB_SNR_DB)
    if rng.uniform() < RB_IMPULSE_PROB:
        x = _add_impulses(x, sr, rng, RB_IMPULSES_PER_SEC, RB_IMPULSE_GAIN)
    if rng.uniform() < RB_REVERB_PROB:
        x = _add_small_reverb(x, sr, rng, RB_REVERB_T_SEC, RB_REVERB_DECAY, RB_REVERB_WET)
    return x

def build_filter_chain_strong(gain_db: float) -> str:
    return ",".join([
        "highpass=f=20",
        "equalizer=f=3000:t=q:w=1.0:g=2.5",
        "equalizer=f=4800:t=q:w=0.9:g=2.0",
        "treble=g=1.0:f=6000:t=h:w=0.7",
        f"volume={gain_db}dB",
        f"silenceremove=start_periods=1:start_duration={TRIM_DUR_S}:start_threshold={TRIM_THR_DB}dB:"
        f"stop_periods=1:stop_duration={TRIM_DUR_S}:stop_threshold={TRIM_THR_DB}dB"
    ])

# ------------ load inputs (FLEURS ONLY) ------------
df_fl = read_csv_safe(FLEURS_CSV)
df_fl = df_fl[df_fl["path"].astype(str).apply(lambda p: Path(p).exists())].copy()
df_in = df_fl.copy()

# ------------ manifest / resume ------------
MANIFEST.parent.mkdir(parents=True, exist_ok=True)
if MANIFEST.exists():
    mf = pd.read_csv(MANIFEST, usecols=["profile","path_out"])
    done_set = set(mf.loc[mf["profile"]==PROFILE, "path_out"].astype(str))
else:
    with open(MANIFEST, "w", newline="", encoding="utf-8") as f:
        csv.writer(f).writerow(["utt_id","path_in","path_out","profile","duration","rms_db","peak_db"])
    done_set = set()

def already_done(pin: str) -> bool:
    pout = out_path_for(pin)
    return pout.exists() or (str(pout) in done_set)

todo_rows = [r for _, r in df_in.iterrows() if not already_done(str(r["path"]))]
planned, todo = len(df_in), len(todo_rows)
print(f"[preproc:{PROFILE}:FLEURS] PLANNED={planned:,} | ALREADY_DONE={planned-todo:,} | TO_DO={todo:,}")

# ------------ worker ------------
def preprocess_one(row):
    pin = str(row["path"])
    pout = out_path_for(pin)
    ensure_parent(pout)

    # read + resample if needed
    try:
        x, sr = sf.read(pin, dtype="float32", always_2d=False)
    except Exception as e:
        return {"error": f"read_fail: {e}", "path_in": pin}
    if hasattr(x, "ndim") and x.ndim > 1: x = x.mean(axis=1)
    if sr != 16000:
        tmp = pout.parent / f".{pout.stem}.in-{uuid.uuid4().hex}.wav"
        cmd = ["ffmpeg","-nostdin","-hide_banner","-loglevel","error","-y",
               "-i", pin, "-ac","1","-ar","16000","-sample_fmt","s16", str(tmp)]
        p = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        if p.returncode != 0:
            try: tmp.unlink(missing_ok=True)
            except: pass
            return {"error": "resample_fail", "path_in": pin}
        x, sr = sf.read(str(tmp), dtype="float32", always_2d=False)
        try: tmp.unlink(missing_ok=True)
        except: pass

    x = np.clip(x, -1.0, 1.0).astype(np.float32)

    # RawBoost v3–style
    key = f"{pin}|{PROFILE}"
    try:    y = rawboost_v3(x, sr, key)
    except: y = x

    # write RB temp near target (avoid cross-device link)
    tmp_rb = pout.parent / f".{pout.stem}.rb-{uuid.uuid4().hex}.wav"
    sf.write(str(tmp_rb), y, 16000, subtype="PCM_16")

    # EQ + loudness + trim via ffmpeg
    gain_db = float(np.clip(TARGET_DB - rms_dbfs_arr(y), -20.0, 20.0))
    FILTER  = build_filter_chain_strong(gain_db)

    tmp_out = pout.parent / f".{pout.stem}.tmp-{uuid.uuid4().hex}.wav"
    cmd = ["ffmpeg","-nostdin","-hide_banner","-loglevel","error","-y",
           "-i", str(tmp_rb), "-ac","1","-ar","16000",
           "-af", FILTER, "-sample_fmt","s16", str(tmp_out)]
    p = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    try: tmp_rb.unlink(missing_ok=True)
    except: pass
    if p.returncode != 0:
        try: tmp_out.unlink(missing_ok=True)
        except: pass
        return {"error": p.stderr.decode("utf-8","ignore")[:300], "path_in": pin}

    os.replace(str(tmp_out), str(pout))

    # stats
    try:
        z, sr2 = sf.read(str(pout), dtype="float32", always_2d=False)
        if hasattr(z, "ndim") and z.ndim > 1: z = z.mean(axis=1)
        dur = len(z)/float(sr2) if sr2 else 0.0
        peak = float(np.max(np.abs(z))) if len(z) else 0.0
        rms  = float(np.sqrt(np.mean(np.square(z)))) if len(z) else 0.0
        rms_db = -120.0 if rms<=1e-9 else 20*np.log10(np.clip(rms,1e-9,1.0))
        peak_db = -120.0 if peak<=1e-9 else 20*np.log10(np.clip(peak,1e-9,1.0))
    except Exception:
        dur, rms_db, peak_db = 0.0, -120.0, -120.0

    return {"utt_id": row.get("utt_id",""), "path_in": pin, "path_out": str(pout),
            "profile": PROFILE, "duration": round(dur,3),
            "rms_db": round(rms_db,2), "peak_db": round(peak_db,2)}

# ------------ scheduler + progress ------------
created = 0; errs = 0; buf = []
start = time.time()
inflight_cap = max(MAX_WORKERS * INFLIGHT_MULT, MAX_WORKERS)
pbar = tqdm(total=len(todo_rows), desc=f"preproc:{PROFILE}:FLEURS", unit="file")

def flush_manifest(rows):
    if not rows: return
    with open(MANIFEST, "a", newline="", encoding="utf-8") as fcsv:
        w = csv.DictWriter(fcsv, fieldnames=["utt_id","path_in","path_out","profile","duration","rms_db","peak_db"])
        for r in rows: w.writerow(r)

it = iter(todo_rows)
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
    inflight = set()
    for _ in range(min(inflight_cap, len(todo_rows))):
        try: inflight.add(ex.submit(preprocess_one, next(it)))
        except StopIteration: break
    while inflight:
        done = next(as_completed(inflight))
        inflight.remove(done)
        res = done.result()
        pbar.update(1)
        if isinstance(res, dict) and "error" in res:
            errs += 1
        elif res:
            buf.append(res); created += 1
            if len(buf) >= BATCH_WRITE:
                flush_manifest(buf); buf.clear()
        try: inflight.add(ex.submit(preprocess_one, next(it)))
        except StopIteration: pass

flush_manifest(buf)
pbar.close()
print(f"\n[{PROFILE}:FLEURS] FINISHED: +{created} files, {errs} errors | elapsed {(time.time()-start)/60:.1f} min")

# quick manifest slice
if MANIFEST.exists():
    m = pd.read_csv(MANIFEST)
    mp = m[m["profile"]==PROFILE]
    hrs = float(pd.to_numeric(mp["duration"], errors="coerce").fillna(0).sum()/3600.0)
    print(f"[{PROFILE}] Manifest rows: {len(mp):,} | hours: {hrs:.2f} h")

print("[OK] STRONG preprocessing for FLEURS done.")

[preproc:strong:FLEURS] PLANNED=604 | ALREADY_DONE=0 | TO_DO=604


preproc:strong:FLEURS:   0%|          | 0/604 [00:00<?, ?file/s]


[strong:FLEURS] FINISHED: +604 files, 0 errors | elapsed 2.3 min
[strong] Manifest rows: 98,397 | hours: 186.23 h
[OK] STRONG preprocessing for FLEURS done.


In [ ]:
# === RAW AUDIT (Colab, read-only) ===
# Reports, for RAW only:
#   • master_real.csv  -> rows, exists_on_disk, hours
#   • master_fake.csv  -> rows, exists_on_disk, hours
#   • attacks.labeled.csv -> rows/exists/hours, plus split by label: real/fake
#
# Hours logic:
#   - If the CSV has a 'duration' column, sum it (seconds -> hours).
#   - Otherwise, sample up to 1000 existing files and estimate hours from headers.

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!pip -q install pandas soundfile

from pathlib import Path
import pandas as pd
import numpy as np
import soundfile as sf
import random

# ---------- CONFIG ----------
ROOT = Path("/content/drive/MyDrive/hindi_dfake")
META = ROOT / "metadata"
REAL_CSV = META / "master_real.csv"
FAKE_CSV = META / "master_fake.csv"
ATT_CSV  = META / "attacks.labeled.csv"

SAMPLE_N_FOR_ESTIMATE = 1000  # cap for fast hour estimation if no 'duration' column

# ---------- helpers ----------
def safe_read_csv(p: Path) -> pd.DataFrame:
    if not p.exists():
        return pd.DataFrame(columns=["path"])
    df = pd.read_csv(p)
    if "path" not in df.columns:
        # try common alt name
        for alt in ["filepath", "audio_path", "wav", "file"]:
            if alt in df.columns:
                df = df.rename(columns={alt: "path"})
                break
    return df

def count_exists(paths):
    return int(sum(Path(p).exists() for p in paths))

def hours_from_csv_duration(df: pd.DataFrame):
    if "duration" in df.columns:
        dur_sec = pd.to_numeric(df["duration"], errors="coerce").fillna(0.0).sum()
        return float(dur_sec / 3600.0)
    return None

def estimate_hours_by_probe(paths, sample_n=SAMPLE_N_FOR_ESTIMATE):
    # Only probe files that actually exist
    ex_paths = [p for p in paths if Path(p).exists()]
    if not ex_paths:
        return 0.0, 0
    if len(ex_paths) > sample_n:
        random.seed(123)
        ex_paths = random.sample(ex_paths, sample_n)
    total_sec = 0.0
    ok = 0
    for p in ex_paths:
        try:
            info = sf.info(p)
            if info.samplerate and info.frames:
                total_sec += info.frames / float(info.samplerate)
                ok += 1
        except:
            pass
    # If we probed a subset, scale up by exists_count / ok
    exists_count = count_exists(paths)
    if ok == 0:
        return 0.0, 0
    est_total_sec = total_sec * (exists_count / ok)
    return float(est_total_sec / 3600.0), ok

def report_block(name, df: pd.DataFrame):
    rows = len(df)
    paths = df["path"].astype(str).tolist() if "path" in df.columns else []
    exists_ct = count_exists(paths)

    hrs = hours_from_csv_duration(df)
    used = "csv_duration"
    probed_n = None
    if hrs is None:
        hrs, probed_n = estimate_hours_by_probe(paths)
        used = f"probe_estimate (sampled {probed_n})"

    print(f"\n[{name}]")
    print(f"  CSV rows       : {rows:,}")
    print(f"  exists on disk : {exists_ct:,}")
    print(f"  hours ({used}) : {hrs:.2f}")

# ---------- load CSVs ----------
real_df = safe_read_csv(REAL_CSV)
fake_df = safe_read_csv(FAKE_CSV)
att_df  = safe_read_csv(ATT_CSV)

print(f"[ROOT] {ROOT} (exists={ROOT.exists()})")
print(f"[meta] real_csv={REAL_CSV.exists()} | fake_csv={FAKE_CSV.exists()} | attacks_csv={ATT_CSV.exists()}")

# ---------- master real/fake ----------
report_block("RAW: real (master_real.csv)", real_df)
report_block("RAW: fake (master_fake.csv)", fake_df)

# ---------- attacks (overall + split by label real/fake) ----------
report_block("ATTACKS: all (attacks.labeled.csv)", att_df)

if "label" in att_df.columns:
    # Normalize label to {real, fake} if possible
    lab = att_df["label"].astype(str).str.lower().str.strip()
    att_real = att_df[lab == "real"].copy()
    att_fake = att_df[lab == "fake"].copy()
    report_block("ATTACKS: real subset", att_real)
    report_block("ATTACKS: fake subset", att_fake)
else:
    print("\n[ATTACKS note] No 'label' column found; cannot split into real/fake subsets.")

Mounted at /content/drive
[ROOT] /content/drive/MyDrive/hindi_dfake (exists=True)
[meta] real_csv=True | fake_csv=True | attacks_csv=True

[RAW: real (master_real.csv)]
  CSV rows       : 25,965
  exists on disk : 25,965
  hours (csv_duration) : 53.13

[RAW: fake (master_fake.csv)]
  CSV rows       : 26,735
  exists on disk : 26,735
  hours (csv_duration) : 60.33

[ATTACKS: all (attacks.labeled.csv)]
  CSV rows       : 45,093
  exists on disk : 45,093
  hours (csv_duration) : 99.75

[ATTACKS: real subset]
  CSV rows       : 23,658
  exists on disk : 23,658
  hours (csv_duration) : 50.37

[ATTACKS: fake subset]
  CSV rows       : 21,435
  exists on disk : 21,435
  hours (csv_duration) : 49.38
